# Install dependencies

In [18]:
! pip install langchain==0.0.340 --quiet # Agent
# ! pip install langsmith==0.1.84 --quiet
!pip install -qU langchain-google-vertexai # gemini pro

! pip install sentence-transformers --quiet # cross encoder

! pip install numexpr --quiet # Math tool

! pip install chromadb==0.4.13 --quiet # chroma vectore store

! pip install google-cloud-bigquery[pandas] --quiet # BQ
! pip install --upgrade google-cloud-aiplatform --quiet # vertex AI
! pip install google-cloud-translate==2.0.1 --quiet # Translation
! pip install google-auth==2.15.0 --quiet
! pip install google-cloud-core==1.7.3 --quiet

! pip install fuzzywuzzy --quiet # Fuzzy search for translator

! pip install nltk --quiet # clean data for entity recognition

! pip install sqlalchemy psycopg2-binary pandas --quiet # Postgres

!pip install pyjwt # decode user token

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.5 requires langsmith<0.2.0,>=0.1.125, but you have langsmith 0.0.92 which is incompatible.
langchain-google-vertexai 1.0.10 requires langchain-core<0.3,>=0.2.33, but you have langchain-core 0.3.5 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.19.2 which is incompatible.
langchain 0.0.340 requires langsmith<0.1.0,>=0.0.63, but you have langsmith 0.1.126 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


# Logging

In [ ]:
import logging
# logging.basicConfig(level=logging.INFO)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Vertex configuration

In [2]:
VERTEX_PROJECT = "future-hubs"
VERTEX_REGION = "us-central1"

# Postgres configuration

In [3]:
import psycopg2
from psycopg2 import sql, OperationalError, DatabaseError

POSTGRES_CONN_PARAMS = {
    "database" : 'FH_Sales_TestAI',
    "user" : 'petroadmin',
    "password" : 'Pswd$FH123',
    "host" : '34.166.72.230',
    "port" : 5432,
    "schema" : 'SalesService'
}
# FH_Sales_Dev
def connect_db(database = POSTGRES_CONN_PARAMS['database'], user = POSTGRES_CONN_PARAMS['user'], password = POSTGRES_CONN_PARAMS['password'], host = POSTGRES_CONN_PARAMS['host'], port = POSTGRES_CONN_PARAMS['port']):
    try:
        logging.info(f"connecting to the database: {database}")
        logging.debug(f"database: {database}")
        logging.debug(f"host: {host}")
        logging.debug(f"port: {port}")

        conn_string = f"dbname={database} user={user} password={password} host={host} port={port}"
        conn = psycopg2.connect(conn_string)
        logging.info(f"Successfully connected to the database: {database}")
        return conn
    except psycopg2.DatabaseError as e:
        logging.error(f"Error connecting to database: {e}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        raise

# Authentication

In [4]:
import vertexai
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "keys_fh.json"
vertexai.init(project=VERTEX_PROJECT, location=VERTEX_REGION)


# Models

In [5]:
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI

CODE_BISON = ChatVertexAI(model_name="codechat-bison", max_output_tokens=1024)
INTERPRETOR_MODEL = ChatVertexAI(max_output_tokens=1024)
AGENT_MODEL agent_model = ChatVertexAI(max_output_tokens=1024)
CHAT_BISON = ChatVertexAI(model_name="chat-bison@002", max_output_tokens=1024)

TEXT_BISON = VertexAI( model_name="text-bison@002", max_output_tokens=1024)

In [ ]:
from langchain_google_vertexai import VertexAI
GEMINI_PRO = VertexAI(model_name="gemini-pro")

# ! pip install langchain-core langchain-google-vertexai --upgrade --quiet

# from langchain_google_vertexai import ChatVertexAI as GoogleChatVertexAI
# gemini_model = GoogleChatVertexAI(model_name="gemini-1.5-pro-001", max_output_tokens=1024) 


# Docs

### Descriptions

In [ ]:
TABLE_DESCRIPTIONS = {
    "AramcoStations": "This table contains information about Aramco stations. Each record represents a specific station with various attributes related to its identification, location, and administrative details.",
    "Bills": "This table stores billing information, capturing details of each bill issued, including the type of bill, transaction details, payment information, and associated entities like vehicles and tenants.",
    "DailySales": "This table contains records of daily sales, capturing detailed information about sales activities at various stations, including user and shift details.",
    "DispenserConfiguration": "This table contains configuration details for dispensers at various stations, including technical specifications and warranty information.",
    "FuelRequest": "This table records requests for fuel, including details of the requested fuel type, quantity, and delivery schedule.",
    "FuelType": "This table stores information about different types of fuel, capturing basic details such as the fuel type name and record metadata.",
    "JanitorData": "This table contains information related to janitorial data at various stations, including available amenities and services.",
    "Location": "This table contains location details, capturing the name and metadata related to the locations.",
    "Nozzles": "This table stores information about the nozzles used in fuel dispensers, capturing details such as nozzle numbers, associated dispensers, and fuel dispensed.",
    "Ports": "This table contains information about the ports at various stations, including their numbers and names.",
    "Regions": "This table contains details about different regions, including names and metadata related to the regions.",
    "Shifts": "This table stores information about shifts at various stations, including shift timings and metadata.",
    "StationMasters": "This table contains detailed information about fuel stations, including identification, location, and various metadata.",
    "Tenants": "This table contains information about tenants, including contact details and commercial registration information.",
    "TransactionDetails": "This table stores details about transactions or operations, including bill and fuel information.",
    "Users": "This table contains information about users, including personal details and account metadata."}

In [ ]:

COLUMN_DESCRIPTIONS = {"AramcoStations":
    {"Id": "Unique identifier for the Aramco station.",
    "TenantId": "Identifier for the tenant, referencing the tenants table.",
    "StationNumber": "Number assigned to the Aramco station.",
    "StationName": "Name of the Aramco station.",
    "Location": "Physical location of the Aramco station.",
    "IsDeleted": "Flag indicating if the station record is deleted.",
    "CreatedDate": "Timestamp of when the station record was created.",
    "ModifiedDate": "Timestamp of when the station record was last modified.",
    "CreatedBy": "Identifier of the user who created the station record.",
    "ModifiedBy": "Identifier of the user who last modified the station record."},
    "Bills":
    {"Id": "Unique identifier for the bill.",
    "BillTypeId": "Identifier for the type of bill.",
    "TransactionTypeId": "Identifier for the type of transaction.",
    "BilledTo": "Identifier of the entity the bill is issued to.",
    "VehicleNumber": "Vehicle number associated with the bill.",
    "CreatedDate": "Timestamp of when the bill was created.",
    "ModifiedDate": "Timestamp of when the bill was last modified.",
    "CreatedBy": "Identifier of the user who created the bill.",
    "ModifiedBy": "Identifier of the user who last modified the bill.",
    "IsDeleted": "Flag indicating if the bill record is deleted.",
    "MeansOfPaymentTypeId": "Identifier for the means of payment.",
    "StationId": "Identifier of the station associated with the bill.",
    "TenantId": "Identifier of the tenant associated with the bill.",
    "TotalBillWithVat": "Total amount of the bill including VAT.",
    "TotalBillWithoutVat": "Total amount of the bill excluding VAT.",
    "UserId": "Identifier of the user associated with the bill.",
    "VehicleNo": "Vehicle number associated with the bill, potentially a duplicate of VehicleNumber."},
    "DailySales":
    {"Id": "Unique identifier for the daily sales record.",
    "TenantId": "Identifier for the tenant.",
    "StationId": "Identifier for the station where the sales occurred.",
    "UserId": "Identifier for the user associated with the sales.",
    "ShiftId": "Identifier for the shift during which the sales were made.",
    "DispenserId": "Identifier for the dispenser used in the sales.",
    "OpeningBalance": "Opening balance of the sales for the day.",
    "ClosingBalance": "Closing balance of the sales for the day.",
    "TotalEmployeeSaleAmount": "Total amount of sales made by employees.",
    "TotalPosSaleAmount": "Total amount of sales made through the POS system.",
    "CreatedDate": "Date when the sales record was created.",
    "ModifiedDate": "Date when the sales record was last modified.",
    "CreatedBy": "Identifier of the user who created the sales record.",
    "ModifiedBy": "Identifier of the user who last modified the sales record.",
    "IsDeleted": "Flag indicating if the sales record is deleted."},
    "DispenserConfiguration":
    {"Id": "Unique identifier for the dispenser configuration.",
    "TenantId": "Identifier for the tenant.",
    "StationId": "Identifier for the station where the dispenser is located.",
    "PortId": "Identifier for the port associated with the dispenser.",
    "DispenserNumber": "Number assigned to the dispenser.",
    "DispenserSNo": "Serial number of the dispenser.",
    "BrandId": "Identifier for the brand of the dispenser.",
    "ModelId": "Identifier for the model of the dispenser.",
    "SpeedOfDelivery": "Speed of delivery for the dispenser.",
    "WarrantyStartDate": "Start date of the dispenser's warranty.",
    "WarrantyEndDate": "End date of the dispenser's warranty.",
    "SupplierId": "Identifier for the supplier of the dispenser.",
    "IsDeleted": "Flag indicating if the dispenser configuration is deleted.",
    "CreatedDate": "Timestamp of when the dispenser configuration was created.",
    "ModifiedDate": "Timestamp of when the dispenser configuration was last modified.",
    "CreatedBy": "Identifier of the user who created the dispenser configuration.",
    "ModifiedBy": "Identifier of the user who last modified the dispenser configuration."},
    "FuelRequest":
    {"int64_field_0": "Auxiliary field, potentially for internal use.",
    "Id": "Unique identifier for the fuel request.",
    "TenantId": "Identifier for the tenant making the request.",
    "AramcoStationld": "Identifier for the Aramco station.",
    "RequestNumber": "Number assigned to the fuel request.",
    "RequestedOn": "Date when the fuel request was made.",
    "SheduledDeliveryDate": "Scheduled date for the fuel delivery.",
    "RequestType": "Type of the fuel request.",
    "FuelTypeId": "Identifier for the type of fuel requested.",
    "Quantity": "Quantity of fuel requested.",
    "UnitPrice": "Unit price of the fuel.",
    "TotalPrice": "Total price of the requested fuel.",
    "Status": "Status of the fuel request.",
    "Comment": "Additional comments or notes about the fuel request.",
    "CreatedDate": "Timestamp of when the fuel request was created.",
    "ModifiedDate": "Timestamp of when the fuel request was last modified.",
    "CreatedBy": "Identifier of the user who created the fuel request.",
    "ModifiedBy": "Identifier of the user who last modified the fuel request.",
    "IsDeleted": "Flag indicating if the fuel request is deleted.",
    "ApprovedBy": "Identifier of the user who approved the fuel request.",
    "ApprovedOn": "Timestamp of when the fuel request was approved.",
    "IsApproved": "Flag indicating if the fuel request is approved."},
    "FuelType":
    {"Id": "Unique identifier for the fuel type.",
    "Name": "Name of the fuel type.",
    "IsDeleted": "Flag indicating if the fuel type record is deleted.",
    "CreatedDate": "Timestamp of when the fuel type record was created.",
    "ModifiedDate": "Timestamp of when the fuel type record was last modified.",
    "CreatedBy": "Identifier of the user who created the fuel type record.",
    "ModifiedBy": "Identifier of the user who last modified the fuel type record."},
    "JanitorData":
    {"fuel_types": "Types of fuel available at the station.",
    "location": "Location of the station.",
    "station_name": "Name of the station.",
    "restroom": "Indicates if restrooms are available at the station.",
    "parking": "Indicates if parking facilities are available at the station.",
    "charging_station": "Indicates if charging stations are available.",
    "air_pump": "Indicates if air pumps are available at the station.",
    "Fueling": "Indicates if fueling services are available.",
    "Oil Services": "Indicates if oil services are available.",
    "Convenience Store": "Indicates if a convenience store is present.",
    "ATM Machines": "Indicates if ATM machines are available.",
    "Car Wash": "Indicates if a car wash is available.",
    "Fast Food Restaurants": "Indicates if fast food restaurants are available."},
    "Location":
    {"Id": "Unique identifier for the location.",
    "Name": "Name of the location.",
    "IsDeleted": "Flag indicating if the location record is deleted.",
    "CreatedDate": "Timestamp of when the location record was created.",
    "ModifiedDate": "Timestamp of when the location record was last modified.",
    "CreatedBy": "Identifier of the user who created the location record.",
    "ModifiedBy": "Identifier of the user who last modified the location record."},
    "Nozzles":
    {"Id": "Unique identifier for the nozzle.",
    "NozzleNumber": "Number assigned to the nozzle.",
    "TenantId": "Identifier for the tenant.",
    "StationId": "Identifier for the station where the nozzle is located.",
    "TankId": "Identifier for the tank associated with the nozzle.",
    "DispenserId": "Identifier for the dispenser to which the nozzle is attached.",
    "TotalFuelDispensedVolume": "Total volume of fuel dispensed by the nozzle.",
    "LastInitialReading": "Last initial reading of the nozzle.",
    "IsDeleted": "Flag indicating if the nozzle record is deleted.",
    "CreatedDate": "Timestamp of when the nozzle record was created.",
    "ModifiedDate": "Timestamp of when the nozzle record was last modified.",
    "CreatedBy": "Identifier of the user who created the nozzle record.",
    "ModifiedBy": "Identifier of the user who last modified the nozzle record."},
    "Ports":
    {"Id": "Unique identifier for the port.",
    "Number": "Number assigned to the port.",
    "Name": "Name of the port.",
    "TenantId": "Identifier for the tenant.",
    "StationId": "Identifier for the station where the port is located.",
    "IsDeleted": "Flag indicating if the port record is deleted.",
    "CreatedDate": "Timestamp of when the port record was created.",
    "ModifiedDate": "Timestamp of when the port record was last modified.",
    "CreatedBy": "Identifier of the user who created the port record.",
    "ModifiedBy": "Identifier of the user who last modified the port record."},
    "Regions":
    {"Id": "Unique identifier for the region.",
    "Name": "Name of the region.",
    "IsDeleted": "Flag indicating if the region record is deleted.",
    "CreatedDate": "Timestamp of when the region record was created.",
    "ModifiedDate": "Timestamp of when the region record was last modified.",
    "CreatedBy": "Identifier of the user who created the region record.",
    "ModifiedBy": "Identifier of the user who last modified the region record.",
    "ArabicName": "Arabic name of the region."},
    "Shifts":
    {"Id": "Unique identifier for the shift.",
    "TenantId": "Identifier for the tenant.",
    "StationId": "Identifier for the station where the shift takes place.",
    "Name": "Name of the shift.",
    "FromTime": "Start time of the shift.",
    "ToTime": "End time of the shift.",
    "IsDeleted": "Flag indicating if the shift record is deleted.",
    "IsNormalShift": "Flag indicating if the shift is a normal shift.",
    "CreatedDate": "Timestamp of when the shift record was created.",
    "ModifiedDate": "Timestamp of when the shift record was last modified.",
    "CreatedBy": "Identifier of the user who created the shift record.",
    "ModifiedBy": "Identifier of the user who last modified the shift record."},
    "StationMasters":
    {"Id": "Unique identifier for the station.",
    "StationNumber": "Number assigned to the station.",
    "CRNumber": "Commercial Registration number of the station.",
    "MainOfficeBranchId": "Identifier for the main office branch.",
    "StationRelationShipId": "Identifier for the station relationship.",
    "CRExpiry": "Expiry date of the Commercial Registration.",
    "LocationId": "Identifier for the location.",
    "RegionId": "Identifier for the region.",
    "LatLong": "Latitude and Longitude coordinates.",
    "Name": "Name of the station.",
    "StationTypeId": "Identifier for the station type.",
    "LastTaqyeessInspection": "Date of the last Taqyeess inspection.",
    "TenantId": "Identifier for the tenant.",
    "IsDeleted": "Flag indicating if the station record is deleted.",
    "CreatedDate": "Timestamp of when the station record was created.",
    "ModifiedDate": "Timestamp of when the station record was last modified.",
    "CreatedBy": "Identifier of the user who created the station record.",
    "ModifiedBy": "Identifier of the user who last modified the station record.",
    "InternalInspectionReminderFrequency": "Frequency of internal inspection reminders.",
    "LastInternalInspection": "Date of the last internal inspection.",
    "TaqyeessInspectionReminderFrequency": "Frequency of Taqyeess inspection reminders.",
    "IsActive": "Flag indicating if the station is active.",
    "CityId": "Identifier for the city."},
    "Tenants":
    {"Id": "Unique identifier for the tenant.",
    "CRNumber": "Commercial Registration number of the tenant.",
    "CompanyName": "Name of the company.",
    "CRExpiryDate": "Expiry date of the Commercial Registration.",
    "PostalCode": "Postal code of the registered address.",
    "RegisteredAddress": "Full registered address of the tenant.",
    "ShortAddress": "Short form of the registered address.",
    "Street": "Street name of the registered address.",
    "AccountOwnerName": "Name of the account owner.",
    "AccountOwnerEmail": "Email address of the account owner.",
    "AccountOwnerPhoneNumber": "Phone number of the account owner.",
    "VAT": "VAT number of the tenant.",
    "IsQualified": "Flag indicating if the tenant is qualified.",
    "IsDeleted": "Flag indicating if the tenant record is deleted.",
    "CreatedDate": "Timestamp of when the tenant record was created.",
    "ModifiedDate": "Timestamp of when the tenant record was last modified.",
    "CreatedBy": "Identifier of the user who created the tenant record.",
    "ModifiedBy": "Identifier of the user who last modified the tenant record.",
    "AccountOwnerNID": "National ID of the account owner.",
    "AdditionalPhoneNumber": "Additional phone number for contact.",
    "Building": "Building name or number.",
    "City": "City name.",
    "District": "District name.",
    "NationalAdditionalPhoneNumber": "Additional national phone number.",
    "NationalBuilding": "National building name or number.",
    "NationalCity": "National city name.",
    "NationalDistrict": "National district name.",
    "NationalPostalCode": "National postal code.",
    "NationalShortAddress": "Short national address.",
    "NationalStreet": "National street name.",
    "AccountOwnerGender": "Gender of the account owner."},
    "TransactionDetails":
    {"Id": "Unique identifier for the transaction.",
    "BillId": "Identifier for the bill associated with the transaction.",
    "TransactionNumber": "Transaction number.",
    "FuelTypeId": "Identifier for the type of fuel.",
    "UnitCost": "Cost per unit of fuel.",
    "Quantity": "Quantity of fuel dispensed.",
    "TotalCost": "Total cost of the transaction.",
    "CreatedDate": "Date of the transaction creation.",
    "ModifiedDate": "Timestamp of when the transaction was last modified.",
    "CreatedBy": "Identifier of the user who created the transaction.",
    "ModifiedBy": "Identifier of the user who last modified the transaction.",
    "IsDeleted": "Flag indicating if the transaction record is deleted.",
    "NozzleId": "Identifier for the nozzle used in the transaction.",
    "StationId": "Identifier for the station where the transaction took place.",
    "TenantId": "Identifier for the tenant associated with the transaction."},
    "Users":
    {"Id": "Unique identifier for the user.",
    "TenantId": "Identifier for the tenant associated with the user.",
    "UserUniqueId": "Unique identifier for the user.",
    "FirstName": "First name of the user.",
    "MiddleName": "Middle name of the user.",
    "LastName": "Last name of the user.",
    "UserName": "Username of the user.",
    "Email": "Email address of the user.",
    "PhoneNumber": "Phone number of the user.",
    "IsDeleted": "Flag indicating if the user record is deleted.",
    "NID": "National ID of the user.",
    "Gender": "Gender of the user.",
    "ProfilePicture": "URL of the user's profile picture.",
    "CreatedDate": "Timestamp of when the user record was created.",
    "ModifiedDate": "Timestamp of when the user record was last modified.",
    "CreatedBy": "Identifier of the user who created the user record.",
    "ModifiedBy": "Identifier of the user who last modified the user record.",
    "IsActive": "Flag indicating if the user is active.",
    "ReportingToId": "Identifier of the user to whom this user reports."}
    }

## Table schema docs 

In [ ]:

def fetch_table_names(cursor, schema):
    """Fetch all table names in the given schema."""
    logging.info(f"Fetching table names from {schema}.")

    table_query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = %s
    """
    try:
        cursor.execute(table_query, (schema,))
        logging.info(f"Fetched table names from {schema}.")

        return cursor.fetchall()
    
    except psycopg2.Error as e:
        logging.error(f"Error fetching table names: {e}")
        raise

In [ ]:
def fetch_column_details(cursor, schema, table_name):
    """Fetch details of all columns in the specified table."""
    logging.info(f"Fetching column deatils from {schema}, {table_name}.")

    column_query = """
    SELECT
        c.column_name,
        c.data_type,
        tc.constraint_type,
        ccu.table_name AS foreign_table_name,
        ccu.column_name AS foreign_column_name
    FROM
        information_schema.columns AS c
    LEFT JOIN
        information_schema.key_column_usage AS kcu
        ON c.table_name = kcu.table_name
        AND c.column_name = kcu.column_name
        AND c.table_schema = kcu.table_schema
    LEFT JOIN
        information_schema.table_constraints AS tc
        ON kcu.constraint_name = tc.constraint_name
        AND kcu.table_name = tc.table_name
        AND kcu.table_schema = tc.table_schema
    LEFT JOIN
        information_schema.referential_constraints AS rc
        ON kcu.constraint_name = rc.constraint_name
    LEFT JOIN
        information_schema.constraint_column_usage AS ccu
        ON rc.unique_constraint_name = ccu.constraint_name
    WHERE
        c.table_schema = %s
        AND c.table_name = %s
    ORDER BY
        c.ordinal_position;
    """
    try:
        cursor.execute(column_query, (schema, table_name))
        return cursor.fetchall()
    except psycopg2.Error as e:
        logging.error(f"Error fetching column details for table {table_name}: {e}")
        raise

In [ ]:
TABLES_USED = [
    "AramcoStations", "Bills", "DailySales", "DispenserConfiguration", "FuelRequest",
    "FuelType", "JanitorData", "Location", "Nozzles", "Ports", "Regions", "Shifts",
    "StationMasters", "Tenants", "TransactionDetails", "Users"
]

def get_parent_child_table_schemas(database, user, password, host, port, schema):
    """Fetch and organize parent-child table schemas for a given schema in the DB."""
    logging.info(f"Fetching table schema from databse {database}.")
    logging.debug(f"database: {database}")
    logging.debug(f"host: {host}")
    logging.debug(f"port: {port}")

    child_docs = []
    parent_docs = []

    # Initialize connection to the database
    conn = None
    try:
        conn = connect_db(database, user, password, host, port)
        cursor = conn.cursor()

        # Fetch all table names in the schema
        tables = fetch_table_names(cursor, schema)

        for table in tables:
            table_name = table[0]
            if table_name in TABLES_USED:
                # Fetch column details for each table
                columns = fetch_column_details(cursor, schema, table_name)

                schema_fields = []
                column_description = ""

                for col in columns:
                    # Placeholder for column descriptions (if available)
                    if table_name in COLUMN_DESCRIPTIONS and col[0] in COLUMN_DESCRIPTIONS[table_name]:
                        column_description = COLUMN_DESCRIPTIONS[table_name][col[0]]

                    col_info = {
                        "column_name": col[0],
                        "data_type": col[1],
                        "constraint_type": col[2],
                        "foreign_table_name": col[3],
                        "foreign_column_name": col[4],
                        "column_description": column_description
                    }
                    schema_fields.append(col_info)

                    col_info_col = {
                        "column_name": col[0],
                        "column_description": column_description
                    }
                    child_doc = {
                        "table_name": table_name,
                        "schema": schema,
                        "column_details": col_info_col
                    }
                    child_docs.append(child_doc)

                # Placeholder for table descriptions (if available)
                table_description = ""
                if table_name in TABLE_DESCRIPTIONS:
                    table_description = TABLE_DESCRIPTIONS[table_name]

                parent_doc = {
                    "table_name": table_name,
                    "schema": schema,
                    "table_description": table_description,
                    "table_details": schema_fields
                }
                parent_docs.append(parent_doc)

    except Exception as e:
        logging.error(f"An error occurred in get_parent_child_table_schemas : {e}")
        raise
    finally:
        # Ensure that the connection is closed
        if conn:
            conn.close()
            logging.info("Database connection closed.")

    return parent_docs, child_docs

In [ ]:
parent_docs, child_docs =get_parent_child_table_schemas(database = POSTGRES_CONN_PARAMS['database'], user = POSTGRES_CONN_PARAMS['user'], password = POSTGRES_CONN_PARAMS['password'], host = POSTGRES_CONN_PARAMS['host'], port = POSTGRES_CONN_PARAMS['port'], schema= POSTGRES_CONN_PARAMS['schema'])
#     database='FH_Sales_TestAI',
#     user='petroadmin',
#     password='Pswd$FH123',
#     host='34.166.72.230',
#     port=5432,
#     schema='SalesService',
# )

In [ ]:
# parent_docs = [{'table_name': '__MyMigrationsHistory', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'MigrationId', 'data_type': 'character varying', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ProductVersion', 'data_type': 'character varying', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'table_description': 'This table stores details about transactions or operations, including bill and fuel information.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the transaction.'}, {'column_name': 'BillId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Bills', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the bill associated with the transaction.'}, {'column_name': 'TransactionNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Transaction number.'}, {'column_name': 'FuelTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'FuelType', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the type of fuel.'}, {'column_name': 'UnitCost', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Cost per unit of fuel.'}, {'column_name': 'Quantity', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Quantity of fuel dispensed.'}, {'column_name': 'TotalCost', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total cost of the transaction.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date of the transaction creation.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the transaction was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the transaction.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the transaction.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the transaction record is deleted.'}, {'column_name': 'NozzleId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Nozzles', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the nozzle used in the transaction.'}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the transaction took place.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant associated with the transaction.'}]}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'Users', 'schema': 'SalesService', 'table_description': 'This table contains information about users, including personal details and account metadata.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the user.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant associated with the user.'}, {'column_name': 'UserUniqueId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the user.'}, {'column_name': 'FirstName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'First name of the user.'}, {'column_name': 'MiddleName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Middle name of the user.'}, {'column_name': 'LastName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Last name of the user.'}, {'column_name': 'UserName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Username of the user.'}, {'column_name': 'Email', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Email address of the user.'}, {'column_name': 'PhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Phone number of the user.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the user record is deleted.'}, {'column_name': 'NID', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National ID of the user.'}, {'column_name': 'Gender', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Gender of the user.'}, {'column_name': 'ProfilePicture', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': "URL of the user's profile picture."}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the user record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the user record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the user record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the user record.'}, {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the user is active.'}, {'column_name': 'ReportingToId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user to whom this user reports.'}]}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'TransactionDoneBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TransactionDateTime', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TransactionAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'BalanceAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TransactionTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationLockerTransactionType', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'LockerAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'MaxLockerAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'table_description': 'This table contains detailed information about fuel stations, including identification, location, and various metadata.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the station.'}, {'column_name': 'StationNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the station.'}, {'column_name': 'CRNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Commercial Registration number of the station.'}, {'column_name': 'MainOfficeBranchId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the main office branch.'}, {'column_name': 'StationRelationShipId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the station relationship.'}, {'column_name': 'CRExpiry', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Expiry date of the Commercial Registration.'}, {'column_name': 'LocationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Location', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the location.'}, {'column_name': 'RegionId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Regions', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the region.'}, {'column_name': 'LatLong', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Latitude and Longitude coordinates.'}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the station.'}, {'column_name': 'StationTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationType', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station type.'}, {'column_name': 'LastTaqyeessInspection', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date of the last Taqyeess inspection.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the station record is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the station record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the station record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the station record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the station record.'}, {'column_name': 'InternalInspectionReminderFrequency', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Frequency of internal inspection reminders.'}, {'column_name': 'LastInternalInspection', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date of the last internal inspection.'}, {'column_name': 'TaqyeessInspectionReminderFrequency', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Frequency of Taqyeess inspection reminders.'}, {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the station is active.'}, {'column_name': 'CityId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the city.'}]}, {'table_name': 'Shifts', 'schema': 'SalesService', 'table_description': 'This table stores information about shifts at various stations, including shift timings and metadata.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the shift.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the tenant.'}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the shift takes place.'}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the shift.'}, {'column_name': 'FromTime', 'data_type': 'time without time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Start time of the shift.'}, {'column_name': 'ToTime', 'data_type': 'time without time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'End time of the shift.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the shift record is deleted.'}, {'column_name': 'IsNormalShift', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the shift is a normal shift.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the shift record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the shift record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the shift record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the shift record.'}]}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Email', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Phone', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'FuelType', 'schema': 'SalesService', 'table_description': 'This table stores information about different types of fuel, capturing basic details such as the fuel type name and record metadata.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the fuel type.'}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the fuel type.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the fuel type record is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the fuel type record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the fuel type record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the fuel type record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the fuel type record.'}]}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'FuelTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'FuelType', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'RetailPrice', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'WholesalePrice', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StartDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'EndDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'Tanks', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'TankMastersId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'TankMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'TankSupplierId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'TankSupplier', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'FuelTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'FuelType', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'WarrantyStartDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'WarrantyExpiryDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'UnusableVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'LastInspectionDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'InspectionReminderFrequency', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'InspectionReminderBeforeDays', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'DustVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'HumidityVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CurentFuelVolume', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Number', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'Ports', 'schema': 'SalesService', 'table_description': 'This table contains information about the ports at various stations, including their numbers and names.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the port.'}, {'column_name': 'Number', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the port.'}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the port.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the port is located.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the port record is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the port record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the port record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the port record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the port record.'}]}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'table_description': 'This table contains configuration details for dispensers at various stations, including technical specifications and warranty information.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the dispenser configuration.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the dispenser is located.'}, {'column_name': 'PortId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Ports', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the port associated with the dispenser.'}, {'column_name': 'DispenserNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the dispenser.'}, {'column_name': 'DispenserSNo', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Serial number of the dispenser.'}, {'column_name': 'BrandId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the brand of the dispenser.'}, {'column_name': 'ModelId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the model of the dispenser.'}, {'column_name': 'SpeedOfDelivery', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Speed of delivery for the dispenser.'}, {'column_name': 'WarrantyStartDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': "Start date of the dispenser's warranty."}, {'column_name': 'WarrantyEndDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': "End date of the dispenser's warranty."}, {'column_name': 'SupplierId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the supplier of the dispenser.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the dispenser configuration is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the dispenser configuration was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the dispenser configuration was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the dispenser configuration.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the dispenser configuration.'}]}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'table_description': 'This table stores information about the nozzles used in fuel dispensers, capturing details such as nozzle numbers, associated dispensers, and fuel dispensed.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the nozzle.'}, {'column_name': 'NozzleNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the nozzle.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the nozzle is located.'}, {'column_name': 'TankId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tanks', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tank associated with the nozzle.'}, {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the dispenser to which the nozzle is attached.'}, {'column_name': 'TotalFuelDispencedVolume', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the dispenser to which the nozzle is attached.'}, {'column_name': 'LastInitialReading', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Last initial reading of the nozzle.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the nozzle record is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the nozzle record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the nozzle record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the nozzle record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the nozzle record.'}]}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationLockerTransactionId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationLockerTransactions', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'CollectedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CollectedDateTime', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ExistingAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CollectedAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'BalanceAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDepositedtoBank', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'BankDepositedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CollectedAmtBasedOnSupervisor', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ExistingAmtBasedOnSupervisor', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsAgreedBySupervisor', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsCashSaleTally', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsPOSSaleTally', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Supervisor', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'Bills', 'schema': 'SalesService', 'table_description': 'This table stores billing information, capturing details of each bill issued, including the type of bill, transaction details, payment information, and associated entities like vehicles and tenants.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the bill.'}, {'column_name': 'BillTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'BillTypes', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the type of bill.'}, {'column_name': 'TransactionTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'TransactionTypes', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the type of transaction.'}, {'column_name': 'BilledTo', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the entity the bill is issued to.'}, {'column_name': 'VehicleNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Vehicle number associated with the bill.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the bill was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the bill was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the bill.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the bill.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the bill record is deleted.'}, {'column_name': 'MeansOfPaymentTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'MeansOfPaymentTypes', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the means of payment.'}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the station associated with the bill.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the tenant associated with the bill.'}, {'column_name': 'TotalBillWithVat', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of the bill including VAT.'}, {'column_name': 'TotalBillWithoutVat', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of the bill excluding VAT.'}, {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the user associated with the bill.'}, {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the user associated with the bill.'}]}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'OperationDoneBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'OperationDateTime', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'NozzleId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Nozzles', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'TankId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tanks', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'PreviousReading', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'InitialReading', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'FinalReading', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Difference', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'FuelTheftVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TestingOrInspectionVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'FuelSaleQuantity', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'FuelSaleDoneBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'DailySales', 'schema': 'SalesService', 'table_description': 'This table contains records of daily sales, capturing detailed information about sales activities at various stations, including user and shift details.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the daily sales record.'}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the tenant.'}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the sales occurred.'}, {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the user associated with the sales.'}, {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the shift during which the sales were made.'}, {'column_name': 'DailySaleId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the shift during which the sales were made.'}, {'column_name': 'OpeningBalance', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Opening balance of the sales for the day.'}, {'column_name': 'ClosingBalance', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Closing balance of the sales for the day.'}, {'column_name': 'TotalEmployeeSaleAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of sales made by employees.'}, {'column_name': 'TotalPosSaleAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of sales made through the POS system.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date when the sales record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date when the sales record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the sales record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the sales record.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the sales record is deleted.'}, {'column_name': 'SaleDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the sales record is deleted.'}, {'column_name': 'TotalSaleAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the sales record is deleted.'}]}, {'table_name': 'Tenants', 'schema': 'SalesService', 'table_description': 'This table contains information about tenants, including contact details and commercial registration information.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the tenant.'}, {'column_name': 'CRNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Commercial Registration number of the tenant.'}, {'column_name': 'CompanyName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the company.'}, {'column_name': 'CRExpiryDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Expiry date of the Commercial Registration.'}, {'column_name': 'PostalCode', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Postal code of the registered address.'}, {'column_name': 'RegisteredAddress', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Full registered address of the tenant.'}, {'column_name': 'ShortAddress', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Short form of the registered address.'}, {'column_name': 'Street', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Street name of the registered address.'}, {'column_name': 'AccountOwnerName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the account owner.'}, {'column_name': 'AccountOwnerEmail', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Email address of the account owner.'}, {'column_name': 'AccountOwnerPhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Phone number of the account owner.'}, {'column_name': 'VAT', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'VAT number of the tenant.'}, {'column_name': 'IsQualified', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the tenant is qualified.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the tenant record is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the tenant record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the tenant record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the tenant record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the tenant record.'}, {'column_name': 'AccountOwnerNID', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National ID of the account owner.'}, {'column_name': 'AdditionalPhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Additional phone number for contact.'}, {'column_name': 'Building', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Building name or number.'}, {'column_name': 'City', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'City name.'}, {'column_name': 'District', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'District name.'}, {'column_name': 'NationalAdditionalPhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Additional national phone number.'}, {'column_name': 'NationalBuilding', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National building name or number.'}, {'column_name': 'NationalCity', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National city name.'}, {'column_name': 'NationalDistrict', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National district name.'}, {'column_name': 'NationalPostalCode', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National postal code.'}, {'column_name': 'NationalShortAddress', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Short national address.'}, {'column_name': 'NationalStreet', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National street name.'}, {'column_name': 'AccountOwnerGender', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Gender of the account owner.'}]}, {'table_name': 'Location', 'schema': 'SalesService', 'table_description': 'This table contains location details, capturing the name and metadata related to the locations.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the location.'}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the location.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the location record is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the location record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the location record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the location record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the location record.'}]}, {'table_name': 'StationType', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'Regions', 'schema': 'SalesService', 'table_description': 'This table contains details about different regions, including names and metadata related to the regions.', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the region.'}, {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the region.'}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the region record is deleted.'}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the region record was created.'}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the region record was last modified.'}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the region record.'}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the region record.'}, {'column_name': 'ArabicName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Arabic name of the region.'}]}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'DailySalesId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'DispencerId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'UserRoleStationMapId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'table_description': '', 'table_details': [{'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Brand', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Model', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'Capacity', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'DoubleWalledTankInnerTankDiameter', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'DoubleWalledTankInnerTankLength', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'DoubleWalledTankOuterTankDiameter', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'DoubleWalledTankOuterTankLength', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'SingleWalledTankDiameter', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}, {'column_name': 'SingleWalledTankLength', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}]}]
# child_docs = [{'table_name': '__MyMigrationsHistory', 'schema': 'SalesService', 'column_details': {'column_name': 'MigrationId', 'data_type': 'character varying', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': '__MyMigrationsHistory', 'schema': 'SalesService', 'column_details': {'column_name': 'ProductVersion', 'data_type': 'character varying', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the transaction.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'BillId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Bills', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the bill associated with the transaction.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'TransactionNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Transaction number.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'FuelTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'FuelType', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the type of fuel.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'UnitCost', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Cost per unit of fuel.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'Quantity', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Quantity of fuel dispensed.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'TotalCost', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total cost of the transaction.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date of the transaction creation.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the transaction was last modified.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the transaction.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the transaction.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the transaction record is deleted.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'NozzleId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Nozzles', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the nozzle used in the transaction.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the transaction took place.'}}, {'table_name': 'TransactionDetails', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant associated with the transaction.'}}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactionType', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant associated with the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'UserUniqueId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'FirstName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'First name of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'MiddleName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Middle name of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'LastName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Last name of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'UserName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Username of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'Email', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Email address of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'PhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Phone number of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the user record is deleted.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'NID', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National ID of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'Gender', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Gender of the user.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'ProfilePicture', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': "URL of the user's profile picture."}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the user record was created.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the user record was last modified.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the user record.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the user record.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the user is active.'}}, {'table_name': 'Users', 'schema': 'SalesService', 'column_details': {'column_name': 'ReportingToId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user to whom this user reports.'}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'TransactionDoneBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'TransactionDateTime', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'TransactionAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'BalanceAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'TransactionTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationLockerTransactionType', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLockerTransactions', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'LockerAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationLocker', 'schema': 'SalesService', 'column_details': {'column_name': 'MaxLockerAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the station.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'StationNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the station.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'CRNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Commercial Registration number of the station.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'MainOfficeBranchId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the main office branch.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'StationRelationShipId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the station relationship.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'CRExpiry', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Expiry date of the Commercial Registration.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'LocationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Location', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the location.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'RegionId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Regions', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the region.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'LatLong', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Latitude and Longitude coordinates.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the station.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'StationTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationType', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station type.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'LastTaqyeessInspection', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date of the last Taqyeess inspection.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the station record is deleted.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the station record was created.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the station record was last modified.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the station record.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the station record.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'InternalInspectionReminderFrequency', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Frequency of internal inspection reminders.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'LastInternalInspection', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date of the last internal inspection.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'TaqyeessInspectionReminderFrequency', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Frequency of Taqyeess inspection reminders.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the station is active.'}}, {'table_name': 'StationMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'CityId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the city.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the shift.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the tenant.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the shift takes place.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the shift.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'FromTime', 'data_type': 'time without time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Start time of the shift.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'ToTime', 'data_type': 'time without time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'End time of the shift.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the shift record is deleted.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'IsNormalShift', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the shift is a normal shift.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the shift record was created.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the shift record was last modified.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the shift record.'}}, {'table_name': 'Shifts', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the shift record.'}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'Email', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'Phone', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankSupplier', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelType', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the fuel type.'}}, {'table_name': 'FuelType', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the fuel type.'}}, {'table_name': 'FuelType', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the fuel type record is deleted.'}}, {'table_name': 'FuelType', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the fuel type record was created.'}}, {'table_name': 'FuelType', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the fuel type record was last modified.'}}, {'table_name': 'FuelType', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the fuel type record.'}}, {'table_name': 'FuelType', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the fuel type record.'}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'FuelTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'FuelType', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'RetailPrice', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'WholesalePrice', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'StartDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'EndDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'FuelPrice', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'TankMastersId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'TankMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'TankSupplierId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'TankSupplier', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'FuelTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'FuelType', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'WarrantyStartDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'WarrantyExpiryDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'UnusableVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'LastInspectionDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'InspectionReminderFrequency', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'InspectionReminderBeforeDays', 'data_type': 'integer', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'DustVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'HumidityVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'CurentFuelVolume', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Tanks', 'schema': 'SalesService', 'column_details': {'column_name': 'Number', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the port.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'Number', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the port.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the port.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the port is located.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the port record is deleted.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the port record was created.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the port record was last modified.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the port record.'}}, {'table_name': 'Ports', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the port record.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the dispenser configuration.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the dispenser is located.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'PortId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Ports', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the port associated with the dispenser.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'DispenserNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the dispenser.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'DispenserSNo', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Serial number of the dispenser.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'BrandId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the brand of the dispenser.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'ModelId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the model of the dispenser.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'SpeedOfDelivery', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Speed of delivery for the dispenser.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'WarrantyStartDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': "Start date of the dispenser's warranty."}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'WarrantyEndDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': "End date of the dispenser's warranty."}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'SupplierId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the supplier of the dispenser.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the dispenser configuration is deleted.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the dispenser configuration was created.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the dispenser configuration was last modified.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the dispenser configuration.'}}, {'table_name': 'DispenserConfiguration', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the dispenser configuration.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the nozzle.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'NozzleNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Number assigned to the nozzle.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tenant.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the nozzle is located.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'TankId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tanks', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the tank associated with the nozzle.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the dispenser to which the nozzle is attached.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'TotalFuelDispencedVolume', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the dispenser to which the nozzle is attached.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'LastInitialReading', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Last initial reading of the nozzle.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the nozzle record is deleted.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the nozzle record was created.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the nozzle record was last modified.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the nozzle record.'}}, {'table_name': 'Nozzles', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the nozzle record.'}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'StationLockerTransactionId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationLockerTransactions', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'CollectedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'CollectedDateTime', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'ExistingAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'CollectedAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'BalanceAmt', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDepositedtoBank', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'BankDepositedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'CollectedAmtBasedOnSupervisor', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'ExistingAmtBasedOnSupervisor', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'IsAgreedBySupervisor', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'IsCashSaleTally', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'IsPOSSaleTally', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationCollections', 'schema': 'SalesService', 'column_details': {'column_name': 'Supervisor', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'BillTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'MeansOfPaymentTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TransactionTypes', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'BillTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'BillTypes', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the type of bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'TransactionTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'TransactionTypes', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the type of transaction.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'BilledTo', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the entity the bill is issued to.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'VehicleNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Vehicle number associated with the bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the bill was created.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the bill was last modified.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the bill record is deleted.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'MeansOfPaymentTypeId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'MeansOfPaymentTypes', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the means of payment.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the station associated with the bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the tenant associated with the bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'TotalBillWithVat', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of the bill including VAT.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'TotalBillWithoutVat', 'data_type': 'real', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of the bill excluding VAT.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the user associated with the bill.'}}, {'table_name': 'Bills', 'schema': 'SalesService', 'column_details': {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': 'Identifier of the user associated with the bill.'}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'OperationDoneBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'OperationDateTime', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'NozzleId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Nozzles', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'TankId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tanks', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'PreviousReading', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'InitialReading', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'FinalReading', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'Difference', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'FuelTheftVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'TestingOrInspectionVolume', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'FuelSaleQuantity', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'FuelSaleDoneBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'NozzleFuelOperations', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the daily sales record.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the tenant.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the station where the sales occurred.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the user associated with the sales.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': 'Identifier for the shift during which the sales were made.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'DailySaleId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier for the shift during which the sales were made.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'OpeningBalance', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Opening balance of the sales for the day.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'ClosingBalance', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Closing balance of the sales for the day.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'TotalEmployeeSaleAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of sales made by employees.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'TotalPosSaleAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Total amount of sales made through the POS system.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date when the sales record was created.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Date when the sales record was last modified.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the sales record.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the sales record.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the sales record is deleted.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'SaleDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the sales record is deleted.'}}, {'table_name': 'DailySales', 'schema': 'SalesService', 'column_details': {'column_name': 'TotalSaleAmount', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the sales record is deleted.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the tenant.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'CRNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Commercial Registration number of the tenant.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'CompanyName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the company.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'CRExpiryDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Expiry date of the Commercial Registration.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'PostalCode', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Postal code of the registered address.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'RegisteredAddress', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Full registered address of the tenant.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'ShortAddress', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Short form of the registered address.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'Street', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Street name of the registered address.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'AccountOwnerName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the account owner.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'AccountOwnerEmail', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Email address of the account owner.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'AccountOwnerPhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Phone number of the account owner.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'VAT', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'VAT number of the tenant.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'IsQualified', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the tenant is qualified.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the tenant record is deleted.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the tenant record was created.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the tenant record was last modified.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the tenant record.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the tenant record.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'AccountOwnerNID', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National ID of the account owner.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'AdditionalPhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Additional phone number for contact.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'Building', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Building name or number.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'City', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'City name.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'District', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'District name.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'NationalAdditionalPhoneNumber', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Additional national phone number.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'NationalBuilding', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National building name or number.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'NationalCity', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National city name.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'NationalDistrict', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National district name.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'NationalPostalCode', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National postal code.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'NationalShortAddress', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Short national address.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'NationalStreet', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'National street name.'}}, {'table_name': 'Tenants', 'schema': 'SalesService', 'column_details': {'column_name': 'AccountOwnerGender', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Gender of the account owner.'}}, {'table_name': 'Location', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the location.'}}, {'table_name': 'Location', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the location.'}}, {'table_name': 'Location', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the location record is deleted.'}}, {'table_name': 'Location', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the location record was created.'}}, {'table_name': 'Location', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the location record was last modified.'}}, {'table_name': 'Location', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the location record.'}}, {'table_name': 'Location', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the location record.'}}, {'table_name': 'StationType', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationType', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationType', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationType', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationType', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationType', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'StationType', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'IsActive', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserStationShiftMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Unique identifier for the region.'}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'Name', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Name of the region.'}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Flag indicating if the region record is deleted.'}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the region record was created.'}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Timestamp of when the region record was last modified.'}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who created the region record.'}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Identifier of the user who last modified the region record.'}}, {'table_name': 'Regions', 'schema': 'SalesService', 'column_details': {'column_name': 'ArabicName', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': 'Arabic name of the region.'}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'DailySalesId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'DispencerId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'DailySalesDispencerMap', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'TenantId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Tenants', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'UserRoleStationMapId', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'UserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Users', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'StationId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'StationMasters', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'ShiftId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'Shifts', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'DispenserId', 'data_type': 'text', 'constraint_type': 'FOREIGN KEY', 'foreign_table_name': 'DispenserConfiguration', 'foreign_column_name': 'Id', 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'UserDispenserMaps', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'Id', 'data_type': 'text', 'constraint_type': 'PRIMARY KEY', 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'Brand', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'Model', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'IsDeleted', 'data_type': 'boolean', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedDate', 'data_type': 'timestamp with time zone', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'CreatedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'ModifiedBy', 'data_type': 'text', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'Capacity', 'data_type': 'double precision', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'DoubleWalledTankInnerTankDiameter', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'DoubleWalledTankInnerTankLength', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'DoubleWalledTankOuterTankDiameter', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'DoubleWalledTankOuterTankLength', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'SingleWalledTankDiameter', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}, {'table_name': 'TankMasters', 'schema': 'SalesService', 'column_details': {'column_name': 'SingleWalledTankLength', 'data_type': 'numeric', 'constraint_type': None, 'foreign_table_name': None, 'foreign_column_name': None, 'column_description': ''}}]
# parent_docs
# child_docs

## Entity Docs

In [ ]:
import psycopg2
import pandas as pd
import re

NOUN_TABLE_COLUMN_DICT = {
    'AramcoStations': {'Location', 'StationName'},
    'Bills': {'BilledTo'},
    'Users': {'Email', 'FirstName', 'LastName', 'MiddleName'},
    'FuelType': {'Name'},
    'JanitorData': {'FuelTypes', 'Location', 'StationName'},
    'Location': {'Name'},
    'Tenants': {'AccountOwnerName', 'CompanyName', 'Building', 'City', 'District', 'NationalBuilding', 'NationalCity', 'NationalDistrict', 'NationalShortAddress', 'NationalStreet', 'RegisteredAddress', 'ShortAddress', 'Street'},
    'StationMasters': {'Name'},
    'Regions': {'ArabicName', 'Name'},
    'FuelRequest': {'ApprovedBy'},
    'Ports': {'Name'},
    'Shifts': {'Name'}
}


#########################
# identify noun column
#########################
def get_string_columns_with_samples(conn, table_name, schema):
    logging.info(f"Fetching column schema from and sample data from {table_name}.")
    logging.debug(f"table name: {table_name}")
    logging.debug(f"schema: {schema}")

    noun_columns_list = NOUN_TABLE_COLUMN_DICT.get(table_name, [])
    noun_data = []
    noun_map = {}

    if noun_columns_list:
        try:
            with conn.cursor() as cursor:
                for noun_column in noun_columns_list:
                    cursor.execute(f"""
                        SELECT "{noun_column}"
                        FROM "{schema}"."{table_name}"
                    """)
                    results = cursor.fetchall()

                    for row in results:
                        column_value = row[0]
                        if column_value:
                            noun_map[column_value] = {
                                "table": table_name,
                                "column": noun_column,
                                "schema": schema
                            }
                            noun_data.append(column_value)
        except psycopg2.Error as e:
            logging.error(f"Error identifying nouns for table {table_name}, column {noun_column}: {e}")

    return noun_data, noun_map



###########################################################################
# get all noun data that the LLM can refer to for making better decisions
###########################################################################
def get_all_noun_data():
    logging.info(f"Fetching all noun data.")

    try:
        conn = connect_db()
        schema = POSTGRES_CONN_PARAMS['schema']
        all_data = set()
        all_noun_map = {}

        try:
            with conn.cursor() as cursor:
                # Fetch all tables in the schema
                cursor.execute("""
                    SELECT table_name
                    FROM information_schema.tables
                    WHERE table_schema = %s
                """, (schema,))
                tables = cursor.fetchall()

                for (table_name,) in tables:

                    # Identify proper noun columns and map data
                    noun_data, noun_map = get_string_columns_with_samples(conn, table_name, schema)

                    if noun_data:
                        all_data.update(noun_data)
                        all_noun_map.update(noun_map)

        finally:
            conn.close()

        all_data_list = list(all_data)
        all_noun_map_list = [{k: v} for k, v in all_noun_map.items()]

        return all_data_list, all_noun_map_list

    except psycopg2.Error as e:
        logging.error(f"Error connecting to the database: {e}")
        return [], {}



In [ ]:
# all_noun_data, noun_maps = get_all_noun_data(TEXT_BISON)
noun_maps = [{'Shift-1': {'table': 'Shifts', 'column': 'Name', 'schema': 'SalesService'}}, {'port-1': {'table': 'Ports', 'column': 'Name', 'schema': 'SalesService'}}, {'port-2': {'table': 'Ports', 'column': 'Name', 'schema': 'SalesService'}}, {'port-3': {'table': 'Ports', 'column': 'Name', 'schema': 'SalesService'}}, {'Diesel': {'table': 'FuelType', 'column': 'Name', 'schema': 'SalesService'}}, {'Super 98': {'table': 'JanitorData', 'column': 'FuelTypes', 'schema': 'SalesService'}}, {'Special 95': {'table': 'JanitorData', 'column': 'FuelTypes', 'schema': 'SalesService'}}, {'EPlus 91': {'table': 'JanitorData', 'column': 'FuelTypes', 'schema': 'SalesService'}}, {'Al-Madina station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Qasr station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Najm station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Riyadh station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Dammam station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Khaleej station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Sharq station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Bahrain station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Fursan station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Jazira station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Waha station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Safwa station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Murooj station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Rabwa station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Sarh station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Nahda station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Fayhaa station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Quds station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Mansour station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Aseel station': {'table': 'JanitorData', 'column': 'StationName', 'schema': 'SalesService'}}, {'Jeddah': {'table': 'Location', 'column': 'Name', 'schema': 'SalesService'}}, {'Riyadh': {'table': 'Tenants', 'column': 'NationalCity', 'schema': 'SalesService'}}, {'Makkah': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Medina': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Dammam': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Sultanah': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Buraidah': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Tabuk': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Najran': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Al Kharj': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Dubai': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Abu Dhabi': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Sharjah': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Al Ai': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Ajman': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Ras Al Khaimah': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Fujairah': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Umm Al Quwain': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Khor Fakkan': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Kalba': {'table': 'JanitorData', 'column': 'Location', 'schema': 'SalesService'}}, {'Aramco station Amal': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Nasr': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Manara': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Fajr': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Hilal': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Noor': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Kawthar': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Rasheed': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Safa': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Huda': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Zahra': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Baraka': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Taqwa': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Mahd': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Azhar': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Aramco station Bustan': {'table': 'AramcoStations', 'column': 'StationName', 'schema': 'SalesService'}}, {'Al-Waha': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Safwa': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Murooj': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Rabwa': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Sarh': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Nahda': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Madina': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Qasr': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Najm': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Riyadh': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Dammam': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Khaleej': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Sharq': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Bahrain': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Al-Fursan': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Jazira': {'table': 'AramcoStations', 'column': 'Location', 'schema': 'SalesService'}}, {'Ahmed': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Ali': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Aisha': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Amira': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Amina': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Basim': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Fatima': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Hasan': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Hussein': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Ibrahim': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Jamal': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Jamilah': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Khalid': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Layla': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Mariam': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Mohammed': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Nabila': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Nadia': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Omar': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Rania': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Rasha': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Salim': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Salma': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Sami': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Sarah': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Yasmin': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Zainab': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Abdullah': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Abdulaziz': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Abdulrahman': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Adam': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Adnan': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Amal': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Anwar': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Asma': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Aziza': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Bilal': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Dina': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Farid': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Ghada': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Hala': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Hamid': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Hana': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Iman': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Ismail': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Jameel': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Kareem': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Lamia': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Lina': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Lubna': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Mahdi': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Malak': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Manal': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Mona': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Mustafa': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Nasser': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Noor': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Qasim': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Rahma': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Rami': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Reem': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Riham': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Saeed': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Samar': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Sanaa': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Selim': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Souad': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Tariq': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Thamer': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Waleed': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Warda': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Yara': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Yasin': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Youssef': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Zaid': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Zahra': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Zaki': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Zein': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Ziad': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Zuhair': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Abir': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Afaf': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Alia': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Ayoub': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Azhar': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Bahir': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Dalal': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Ehsan': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Eman': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Farah': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Fawzi': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'Haifa': {'table': 'Bills', 'column': 'BilledTo', 'schema': 'SalesService'}}, {'hamid.lubna@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'hana.mahdi@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'iman.malak@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'ismail.manal@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'jameel.mona@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'kareem.mustafa@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'lamia.nasser@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'lina.noor@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'lubna.qasim@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'mahdi.rahma@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'malak.rami@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'manal.reem@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'mona.riham@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'mustafa.saeed@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'nasser.samar@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'noor.sanaa@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'qasim.selim@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'rahma.souad@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'rami.tariq@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'reem.thamer@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'riham.waleed@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'saeed.warda@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'samar.yara@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'sanaa.yasin@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'selim.youssef@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'souad.zaid@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'tariq.zahra@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'thamer.zaki@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'waleed.zein@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'warda.ziad@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'yara.zuhair@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'yasin.abir@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'youssef.afaf@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'zaid.alia@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'zahra.amal@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'zaki.ayoub@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'zein.azhar@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'ziad.bahir@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'zuhair.dalal@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'abir.ehsan@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'afaf.eman@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'dalal.kawthar@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'ehsan.leen@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'eman.lubna@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'farah.marwa@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'fawzi.munir@example.com': {'table': 'FuelRequest', 'column': 'ApprovedBy', 'schema': 'SalesService'}}, {'123 Al-Nahda Street, Riyadh, Saudi Arabia': {'table': 'Tenants', 'column': 'RegisteredAddress', 'schema': 'SalesService'}}, {'Al-Takhassusi Street': {'table': 'Tenants', 'column': 'Street', 'schema': 'SalesService'}}, {'Riyadh, KSA': {'table': 'Tenants', 'column': 'NationalShortAddress', 'schema': 'SalesService'}}, {'Al-Olaya': {'table': 'Tenants', 'column': 'District', 'schema': 'SalesService'}}, {'Al-Jazeera Tower': {'table': 'Tenants', 'column': 'NationalBuilding', 'schema': 'SalesService'}}, {'Mohammad Ali Al-Khalidi': {'table': 'Tenants', 'column': 'AccountOwnerName', 'schema': 'SalesService'}}, {'Al-Rawi Trading Co.': {'table': 'Tenants', 'column': 'CompanyName', 'schema': 'SalesService'}}, {'ahmed.al-farsi@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'mohammed.al-sayed@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'ali.al-mansour@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'hassan.al-amin@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'omar.al-harbi@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'abdullah.al-rashid@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'khalid.al-hashemi@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'youssef.al-qahtani@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'ibrahim.al-sharif@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'salem.al-zahrani@example.com': {'table': 'Users', 'column': 'Email', 'schema': 'SalesService'}}, {'Al-Farsi': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Sayed': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Mansour': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Amin': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Harbi': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Rashid': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Hashemi': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Qahtani': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Sharif': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Al-Zahrani': {'table': 'Users', 'column': 'LastName', 'schema': 'SalesService'}}, {'Hassan': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Salem': {'table': 'Users', 'column': 'FirstName', 'schema': 'SalesService'}}, {'Samra petrol station': {'table': 'StationMasters', 'column': 'Name', 'schema': 'SalesService'}}, {'north': {'table': 'Regions', 'column': 'Name', 'schema': 'SalesService'}}, {'ahmed ali': {'table': 'Regions', 'column': 'ArabicName', 'schema': 'SalesService'}}, {'95': {'table': 'FuelType', 'column': 'Name', 'schema': 'SalesService'}}, {'91': {'table': 'FuelType', 'column': 'Name', 'schema': 'SalesService'}}]
all_noun_data = ['port-2', 'Aramco station Baraka', 'Super 98', 'Farid', 'mustafa.saeed@example.com', 'Buraidah', 'Al-Waha', 'yara.zuhair@example.com', 'sanaa.yasin@example.com', 'Tariq', 'ahmed.al-farsi@example.com', 'Salem', 'Al-Madina', 'lubna.qasim@example.com', '123 Al-Nahda Street, Riyadh, Saudi Arabia', 'thamer.zaki@example.com', 'Zahra', 'Ajman', 'Khor Fakkan', 'Al-Bahrain station', 'Al-Amin', 'Yara', 'iman.malak@example.com', 'Umm Al Quwain', 'Qasim', 'Shift-1', 'Farah', 'Al-Harbi', 'lina.noor@example.com', 'Asma', 'Al-Fayhaa station', 'Al-Farsi', 'Riham', 'Al-Sharif', 'Al-Qahtani', 'Al-Olaya', 'Souad', 'Zainab', 'Riyadh', 'Nabila', 'Al-Rawi Trading Co.', 'Mohammed', 'Medina', 'Al-Qasr', 'Jazira', 'Nasser', 'Fawzi', 'youssef.al-qahtani@example.com', 'malak.rami@example.com', 'salem.al-zahrani@example.com', 'port-1', 'Lamia', 'eman.lubna@example.com', 'Zaid', 'Al-Rashid', 'abir.ehsan@example.com', 'Dubai', 'Al-Fursan', 'mona.riham@example.com', 'nasser.samar@example.com', 'port-3', 'mohammed.al-sayed@example.com', 'ali.al-mansour@example.com', 'Tabuk', 'Mona', 'rahma.souad@example.com', 'Adnan', 'Zein', 'Rabwa', 'mahdi.rahma@example.com', 'rami.tariq@example.com', 'north', 'Al-Aseel station', 'Sharq', 'youssef.afaf@example.com', 'Hasan', 'ziad.bahir@example.com', 'Al-Takhassusi Street', 'samar.yara@example.com', 'Al Ai', 'Rasha', 'Selim', 'Sarah', 'Al-Safwa station', 'Yasmin', 'Ali', 'Waleed', 'Al-Bahrain', 'Rami', 'Al-Hashemi', 'Anwar', 'Youssef', 'souad.zaid@example.com', 'farah.marwa@example.com', 'Hassan', 'Aramco station Safa', 'Aramco station Noor', 'Aramco station Mahd', '95', 'noor.sanaa@example.com', 'Abdullah', 'Al-Sarh station', 'Safwa', 'Amira', 'omar.al-harbi@example.com', 'Jamilah', 'Al-Zahrani', 'Aramco station Zahra', 'ibrahim.al-sharif@example.com', 'Ras Al Khaimah', 'Sultanah', 'Samar', 'Alia', 'abdullah.al-rashid@example.com', 'Nadia', 'Hamid', 'Kalba', 'Al-Mansour station', 'Al-Nahda', 'Fatima', 'Adam', 'Aramco station Manara', 'Al-Qasr station', 'Yasin', 'Warda', '91', 'Al-Quds station', 'qasim.selim@example.com', 'Al-Madina station', 'selim.youssef@example.com', 'Dammam', 'Jamal', 'Aramco station Fajr', 'Ismail', 'Mohammad Ali Al-Khalidi', 'Aramco station Kawthar', 'Khalid', 'waleed.zein@example.com', 'Dalal', 'Al-Khaleej station', 'manal.reem@example.com', 'riham.waleed@example.com', 'Mariam', 'Al-Waha station', 'Makkah', 'Fujairah', 'khalid.al-hashemi@example.com', 'hamid.lubna@example.com', 'Sharjah', 'Aisha', 'Ayoub', 'Sami', 'Malak', 'Reem', 'hassan.al-amin@example.com', 'reem.thamer@example.com', 'Jameel', 'Aramco station Taqwa', 'Ghada', 'Al-Sharq station', 'hana.mahdi@example.com', 'Bahir', 'zaki.ayoub@example.com', 'Ehsan', 'Aziza', 'Najm', 'EPlus 91', 'ehsan.leen@example.com', 'Special 95', 'zein.azhar@example.com', 'Afaf', 'Diesel', 'kareem.mustafa@example.com', 'Azhar', 'Hana', 'Aramco station Azhar', 'afaf.eman@example.com', 'Lubna', 'Jeddah', 'Aramco station Nasr', 'Aramco station Bustan', 'Ibrahim', 'Aramco station Hilal', 'Al-Jazeera Tower', 'Abdulaziz', 'Noor', 'Al-Jazira station', 'Al Kharj', 'Al-Murooj', 'Al-Riyadh', 'Aramco station Amal', 'Samra petrol station', 'zaid.alia@example.com', 'Rahma', 'Riyadh, KSA', 'Abir', 'Ahmed', 'Aramco station Huda', 'warda.ziad@example.com', 'jameel.mona@example.com', 'Hussein', 'tariq.zahra@example.com', 'Omar', 'Amina', 'Najran', 'yasin.abir@example.com', 'Al-Murooj station', 'Layla', 'ismail.manal@example.com', 'dalal.kawthar@example.com', 'Kareem', 'Al-Sarh', 'Ziad', 'Al-Fursan station', 'Abdulrahman', 'Manal', 'Iman', 'Mustafa', 'ahmed ali', 'Al-Dammam station', 'Mahdi', 'Rania', 'fawzi.munir@example.com', 'Sanaa', 'Saeed', 'Al-Riyadh station', 'Al-Rabwa station', 'Al-Nahda station', 'Al-Dammam', 'Bilal', 'Hala', 'Lina', 'Al-Najm station', 'zahra.amal@example.com', 'Amal', 'Al-Mansour', 'Al-Khaleej', 'saeed.warda@example.com', 'Zaki', 'Al-Sayed', 'Abu Dhabi', 'Zuhair', 'Eman', 'Aramco station Rasheed', 'Basim', 'Thamer', 'Haifa', 'Salim', 'zuhair.dalal@example.com', 'Salma', 'Dina', 'lamia.nasser@example.com']

In [ ]:
# print(all_noun_data)

In [ ]:
# noun_maps = [{'Al-Waha': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Murooj': {'table': 'aramco-station', 'column': 'Location'}}, {'Rabwa': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Sarh': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Nahda': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Madina': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Qasr': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Riyadh': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Dammam': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Khaleej': {'table': 'aramco-station', 'column': 'Location'}}, {'Sharq': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Bahrain': {'table': 'aramco-station', 'column': 'Location'}}, {'Jazira': {'table': 'aramco-station', 'column': 'Location'}}, {'Safwa': {'table': 'aramco-station', 'column': 'Location'}}, {'Najm': {'table': 'aramco-station', 'column': 'Location'}}, {'Al-Fursan': {'table': 'aramco-station', 'column': 'Location'}}, {'Aramco station Amal': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Manara': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Fajr': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Hilal': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Noor': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Kawthar': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Rasheed': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Huda': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Zahra': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Baraka': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Taqwa': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Mahd': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Bustan': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Nasr': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Safa': {'table': 'aramco-station', 'column': 'StationName'}}, {'Aramco station Azhar': {'table': 'aramco-station', 'column': 'StationName'}}, {'4Y6h8z131z75pWIr': {'table': 'bills', 'column': 'VehicleNumber'}}, {'8m7E7G9Z66Tw25IA': {'table': 'bills', 'column': 'VehicleNumber'}}, {'uGex454XQ98e25b2': {'table': 'bills', 'column': 'VehicleNumber'}}, {'nu49hU1Vn054fh34': {'table': 'bills', 'column': 'VehicleNumber'}}, {'zi3DR73Q05d3w03L': {'table': 'bills', 'column': 'VehicleNumber'}}, {'oY53q55RkW0i3K17': {'table': 'bills', 'column': 'VehicleNumber'}}, {'ET564XL393t0S0ZA': {'table': 'bills', 'column': 'VehicleNumber'}}, {'52s891CZS0X8J4Dr': {'table': 'bills', 'column': 'VehicleNumber'}}, {'W2M2O7MX6J0q57F4': {'table': 'bills', 'column': 'VehicleNumber'}}, {'1CaGp270C5900Asw': {'table': 'bills', 'column': 'VehicleNumber'}}, {'98yG06zI836ks2JY': {'table': 'bills', 'column': 'VehicleNumber'}}, {'F9y1Yoc283H33Ae2': {'table': 'bills', 'column': 'VehicleNumber'}}, {'96OEh99uro82r62a': {'table': 'bills', 'column': 'VehicleNumber'}}, {'r57h71b10h88FzlL': {'table': 'bills', 'column': 'VehicleNumber'}}, {'eAY64453Enf7Dg14': {'table': 'bills', 'column': 'VehicleNumber'}}, {'djM12a9G9790sb9f': {'table': 'bills', 'column': 'VehicleNumber'}}, {'6J755E72u8Lqgi5M': {'table': 'bills', 'column': 'VehicleNumber'}}, {'5Jr190L24WzUUS96': {'table': 'bills', 'column': 'VehicleNumber'}}, {'5e7b58M08BGkGM03': {'table': 'bills', 'column': 'VehicleNumber'}}, {'91i2qv930Z0BpT7Q': {'table': 'bills', 'column': 'VehicleNumber'}}, {'W8210bE255lgs2KR': {'table': 'bills', 'column': 'VehicleNumber'}}, {'f9i53o33byu2H98E': {'table': 'bills', 'column': 'VehicleNumber'}}, {'i3S7T4v3o0Ty45u7': {'table': 'bills', 'column': 'VehicleNumber'}}, {'Ol45J1w2zpp9y419': {'table': 'bills', 'column': 'VehicleNumber'}}, {'DH858xQ2bUA1M192': {'table': 'bills', 'column': 'VehicleNumber'}}, {'FSg201eN24vPI416': {'table': 'bills', 'column': 'VehicleNumber'}}, {'5Xl43A04jx7lSB91': {'table': 'bills', 'column': 'VehicleNumber'}}, {'6Q3C0eJg47fn49A3': {'table': 'bills', 'column': 'VehicleNumber'}}, {'11QX835Dg9N6c0Jb': {'table': 'bills', 'column': 'VehicleNumber'}}, {'jS1HB61b669P6r8t': {'table': 'bills', 'column': 'VehicleNumber'}}, {'47xe6c9QOSkz5358': {'table': 'bills', 'column': 'VehicleNumber'}}, {'4aJ27zCm01Z7t6m8': {'table': 'bills', 'column': 'VehicleNumber'}}, {'Y4eqf664rw5xE865': {'table': 'bills', 'column': 'VehicleNumber'}}, {'7tyu38kg8TQ1247Z': {'table': 'bills', 'column': 'VehicleNumber'}}, {'2Y1R4a11YMz132SE': {'table': 'bills', 'column': 'VehicleNumber'}}, {'2Tfx60Q6073gse0H': {'table': 'bills', 'column': 'VehicleNumber'}}, {'s2NA2LsY78z0771A': {'table': 'bills', 'column': 'VehicleNumber'}}, {'54r8K2oN3PH3A1H3': {'table': 'bills', 'column': 'VehicleNumber'}}, {'X07H3sFm028e90jq': {'table': 'bills', 'column': 'VehicleNumber'}}, {'sr9A5h7K137n8G6l': {'table': 'bills', 'column': 'VehicleNumber'}}, {'3UEfDc33K0q0g346': {'table': 'bills', 'column': 'VehicleNumber'}}, {'z78N9t49h8q2Rd7o': {'table': 'bills', 'column': 'VehicleNumber'}}, {'T374I41K9rme7oK2': {'table': 'bills', 'column': 'VehicleNumber'}}, {'24YW6215n65iQgaS': {'table': 'bills', 'column': 'VehicleNumber'}}, {'9uZ9H11X3E2u7U8d': {'table': 'bills', 'column': 'VehicleNumber'}}, {'6b52akdf0b0r4s39': {'table': 'bills', 'column': 'VehicleNumber'}}, {'Z5MG3o2GoA25572p': {'table': 'bills', 'column': 'VehicleNumber'}}, {'u4vF15aIX8V6u894': {'table': 'bills', 'column': 'VehicleNumber'}}, {'99301diz7u3B4MgC': {'table': 'bills', 'column': 'VehicleNumber'}}, {'6rg5okO6t45W3q91': {'table': 'bills', 'column': 'VehicleNumber'}}, {'ROV1S19373K9VT0W': {'table': 'bills', 'column': 'VehicleNumber'}}, {'0R660631MBM5keIx': {'table': 'bills', 'column': 'VehicleNumber'}}, {'H4l1Nc7S8382n4zD': {'table': 'bills', 'column': 'VehicleNumber'}}, {'1B8e46w1L5fXY8M0': {'table': 'bills', 'column': 'VehicleNumber'}}, {'f6p6D1QG13j172sO': {'table': 'bills', 'column': 'VehicleNumber'}}, {'Qhm5xJ359LR20s86': {'table': 'bills', 'column': 'VehicleNumber'}}, {'4673J0MH9lYH87jR': {'table': 'bills', 'column': 'VehicleNumber'}}, {'wKPc8X636ypz7580': {'table': 'bills', 'column': 'VehicleNumber'}}, {'Q5R1cIZ6m75h98A9': {'table': 'bills', 'column': 'VehicleNumber'}}, {'R462ZHa59rv1l49T': {'table': 'bills', 'column': 'VehicleNumber'}}, {'247Ry0CKk92N8ND5': {'table': 'bills', 'column': 'VehicleNumber'}}, {'qZ8074F3IM90H3cV': {'table': 'bills', 'column': 'VehicleNumber'}}, {'6PdRb1z7q831A8I1': {'table': 'bills', 'column': 'VehicleNumber'}}, {'p6P16258Ckx7L0Kl': {'table': 'bills', 'column': 'VehicleNumber'}}, {'Q986LG08HuzE3p18': {'table': 'bills', 'column': 'VehicleNumber'}}, {'R20J9u8n496gUK8g': {'table': 'bills', 'column': 'VehicleNumber'}}, {'m3V81K1457HD6yFV': {'table': 'bills', 'column': 'VehicleNumber'}}, {'91qq0UV14DL42IE5': {'table': 'bills', 'column': 'VehicleNumber'}}, {'4NJu90bVf7l749y8': {'table': 'bills', 'column': 'VehicleNumber'}}, {'i08N207lFgw5i00i': {'table': 'bills', 'column': 'VehicleNumber'}}, {'HS6C9Ky7p754SJ41': {'table': 'bills', 'column': 'VehicleNumber'}}, {'4Dgd5770AfqdH901': {'table': 'bills', 'column': 'VehicleNumber'}}, {'YxI96Y606729stYT': {'table': 'bills', 'column': 'VehicleNumber'}}, {'8G96wBCm4i5152CB': {'table': 'bills', 'column': 'VehicleNumber'}}, {'c66Q97UM7y755xdd': {'table': 'bills', 'column': 'VehicleNumber'}}, {'7ds0J91N7TBj642n': {'table': 'bills', 'column': 'VehicleNumber'}}, {'84trW13bN1NQL423': {'table': 'bills', 'column': 'VehicleNumber'}}, {'qK05dF6iGX66B745': {'table': 'bills', 'column': 'VehicleNumber'}}, {'7593ni40Fnto3f3A': {'table': 'bills', 'column': 'VehicleNumber'}}, {'s612237J0TF0BXGf': {'table': 'bills', 'column': 'VehicleNumber'}}, {'P0a2Am70C2c70re1': {'table': 'bills', 'column': 'VehicleNumber'}}, {'4L6G27aPdM94Lu93': {'table': 'bills', 'column': 'VehicleNumber'}}, {'6x4rU4L47Z59V3PQ': {'table': 'bills', 'column': 'VehicleNumber'}}, {'V64R8b89O9jj08rM': {'table': 'bills', 'column': 'VehicleNumber'}}, {'54p0jW7445Ao7Mve': {'table': 'bills', 'column': 'VehicleNumber'}}, {'7lx18tIx262Et10J': {'table': 'bills', 'column': 'VehicleNumber'}}, {'g9Q37Q0nwLC1478P': {'table': 'bills', 'column': 'VehicleNumber'}}, {'6ny26U33jn0qi6Y4': {'table': 'bills', 'column': 'VehicleNumber'}}, {'9b2xV05K1E068Vsy': {'table': 'bills', 'column': 'VehicleNumber'}}, {'5t9bNVd4ro68178D': {'table': 'bills', 'column': 'VehicleNumber'}}, {'7dGhB4r12a280H9W': {'table': 'bills', 'column': 'VehicleNumber'}}, {'dh29LfN358I5Ts98': {'table': 'bills', 'column': 'VehicleNumber'}}, {'m34H49q52C78jApH': {'table': 'bills', 'column': 'VehicleNumber'}}, {'Farid': {'table': 'bills', 'column': 'BilledTo'}}, {'Hamid': {'table': 'bills', 'column': 'BilledTo'}}, {'Lamia': {'table': 'bills', 'column': 'BilledTo'}}, {'Samar': {'table': 'bills', 'column': 'BilledTo'}}, {'Selim': {'table': 'bills', 'column': 'BilledTo'}}, {'Yasin': {'table': 'bills', 'column': 'BilledTo'}}, {'Amal': {'table': 'bills', 'column': 'BilledTo'}}, {'Ibrahim': {'table': 'users', 'column': 'FirstName'}}, {'Mariam': {'table': 'bills', 'column': 'BilledTo'}}, {'Azhar': {'table': 'bills', 'column': 'BilledTo'}}, {'Ahmed': {'table': 'users', 'column': 'FirstName'}}, {'Ali': {'table': 'users', 'column': 'FirstName'}}, {'Jamilah': {'table': 'bills', 'column': 'BilledTo'}}, {'Bilal': {'table': 'bills', 'column': 'BilledTo'}}, {'Sanaa': {'table': 'bills', 'column': 'BilledTo'}}, {'Layla': {'table': 'bills', 'column': 'BilledTo'}}, {'Zainab': {'table': 'bills', 'column': 'BilledTo'}}, {'Lubna': {'table': 'bills', 'column': 'BilledTo'}}, {'Dalal': {'table': 'bills', 'column': 'BilledTo'}}, {'Khalid': {'table': 'users', 'column': 'FirstName'}}, {'Salim': {'table': 'bills', 'column': 'BilledTo'}}, {'Anwar': {'table': 'bills', 'column': 'BilledTo'}}, {'Ehsan': {'table': 'bills', 'column': 'BilledTo'}}, {'Mohammed': {'table': 'users', 'column': 'FirstName'}}, {'Nadia': {'table': 'bills', 'column': 'BilledTo'}}, {'Sami': {'table': 'bills', 'column': 'BilledTo'}}, {'Basim': {'table': 'bills', 'column': 'BilledTo'}}, {'Abdulrahman': {'table': 'bills', 'column': 'BilledTo'}}, {'Asma': {'table': 'bills', 'column': 'BilledTo'}}, {'Dina': {'table': 'bills', 'column': 'BilledTo'}}, {'Ismail': {'table': 'bills', 'column': 'BilledTo'}}, {'Omar': {'table': 'users', 'column': 'FirstName'}}, {'Yara': {'table': 'bills', 'column': 'BilledTo'}}, {'Ziad': {'table': 'bills', 'column': 'BilledTo'}}, {'Hasan': {'table': 'bills', 'column': 'BilledTo'}}, {'Jamal': {'table': 'bills', 'column': 'BilledTo'}}, {'Rasha': {'table': 'bills', 'column': 'BilledTo'}}, {'Salma': {'table': 'bills', 'column': 'BilledTo'}}, {'Mahdi': {'table': 'bills', 'column': 'BilledTo'}}, {'Fawzi': {'table': 'bills', 'column': 'BilledTo'}}, {'Farah': {'table': 'bills', 'column': 'BilledTo'}}, {'Amira': {'table': 'bills', 'column': 'BilledTo'}}, {'Nabila': {'table': 'bills', 'column': 'BilledTo'}}, {'Adam': {'table': 'bills', 'column': 'BilledTo'}}, {'Ghada': {'table': 'bills', 'column': 'BilledTo'}}, {'Iman': {'table': 'bills', 'column': 'BilledTo'}}, {'Qasim': {'table': 'bills', 'column': 'BilledTo'}}, {'Yasmin': {'table': 'bills', 'column': 'BilledTo'}}, {'Abdullah': {'table': 'users', 'column': 'FirstName'}}, {'Abdulaziz': {'table': 'bills', 'column': 'BilledTo'}}, {'Kareem': {'table': 'bills', 'column': 'BilledTo'}}, {'Noor': {'table': 'bills', 'column': 'BilledTo'}}, {'Riham': {'table': 'bills', 'column': 'BilledTo'}}, {'Tariq': {'table': 'bills', 'column': 'BilledTo'}}, {'Waleed': {'table': 'bills', 'column': 'BilledTo'}}, {'Haifa': {'table': 'bills', 'column': 'BilledTo'}}, {'Mustafa': {'table': 'bills', 'column': 'BilledTo'}}, {'Rami': {'table': 'bills', 'column': 'BilledTo'}}, {'Zaid': {'table': 'bills', 'column': 'BilledTo'}}, {'Zaki': {'table': 'bills', 'column': 'BilledTo'}}, {'Alia': {'table': 'bills', 'column': 'BilledTo'}}, {'Amina': {'table': 'bills', 'column': 'BilledTo'}}, {'Jameel': {'table': 'bills', 'column': 'BilledTo'}}, {'Bahir': {'table': 'bills', 'column': 'BilledTo'}}, {'Eman': {'table': 'bills', 'column': 'BilledTo'}}, {'Fatima': {'table': 'bills', 'column': 'BilledTo'}}, {'Hala': {'table': 'bills', 'column': 'BilledTo'}}, {'Nasser': {'table': 'bills', 'column': 'BilledTo'}}, {'Thamer': {'table': 'bills', 'column': 'BilledTo'}}, {'Sarah': {'table': 'bills', 'column': 'BilledTo'}}, {'Hana': {'table': 'bills', 'column': 'BilledTo'}}, {'Malak': {'table': 'bills', 'column': 'BilledTo'}}, {'Warda': {'table': 'bills', 'column': 'BilledTo'}}, {'Zuhair': {'table': 'bills', 'column': 'BilledTo'}}, {'Hussein': {'table': 'bills', 'column': 'BilledTo'}}, {'Rania': {'table': 'bills', 'column': 'BilledTo'}}, {'Mona': {'table': 'bills', 'column': 'BilledTo'}}, {'Saeed': {'table': 'bills', 'column': 'BilledTo'}}, {'Youssef': {'table': 'users', 'column': 'FirstName'}}, {'Zahra': {'table': 'bills', 'column': 'BilledTo'}}, {'Afaf': {'table': 'bills', 'column': 'BilledTo'}}, {'Adnan': {'table': 'bills', 'column': 'BilledTo'}}, {'Manal': {'table': 'bills', 'column': 'BilledTo'}}, {'Rahma': {'table': 'bills', 'column': 'BilledTo'}}, {'Abir': {'table': 'bills', 'column': 'BilledTo'}}, {'Lina': {'table': 'bills', 'column': 'BilledTo'}}, {'Reem': {'table': 'bills', 'column': 'BilledTo'}}, {'Zein': {'table': 'bills', 'column': 'BilledTo'}}, {'Aisha': {'table': 'bills', 'column': 'BilledTo'}}, {'Aziza': {'table': 'bills', 'column': 'BilledTo'}}, {'Souad': {'table': 'bills', 'column': 'BilledTo'}}, {'Ayoub': {'table': 'bills', 'column': 'BilledTo'}}, {'reem.thamer@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'riham.waleed@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'saeed.warda@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'samar.yara@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'sanaa.yasin@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'selim.youssef@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'souad.zaid@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'warda.ziad@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'yasin.abir@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'zahra.amal@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'zaki.ayoub@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'abir.ehsan@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'hamid.lubna@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'hana.mahdi@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'iman.malak@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'ismail.manal@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'jameel.mona@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'kareem.mustafa@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'lamia.nasser@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'lina.noor@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'lubna.qasim@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'mahdi.rahma@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'malak.rami@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'mona.riham@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'mustafa.saeed@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'rahma.souad@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'nasser.samar@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'manal.reem@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'noor.sanaa@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'qasim.selim@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'rami.tariq@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'tariq.zahra@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'farah.marwa@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'thamer.zaki@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'waleed.zein@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'yara.zuhair@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'youssef.afaf@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'zaid.alia@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'zein.azhar@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'ziad.bahir@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'zuhair.dalal@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'afaf.eman@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'dalal.kawthar@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'ehsan.leen@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'eman.lubna@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'fawzi.munir@example.com': {'table': 'fuel-request', 'column': 'ApprovedBy'}}, {'95': {'table': 'fuel-type', 'column': 'Name'}}, {'91': {'table': 'fuel-type', 'column': 'Name'}}, {'Diesel': {'table': 'janitor-data', 'column': 'fuel_types'}}, {'EPlus 91': {'table': 'janitor-data', 'column': 'fuel_types'}}, {'Special 95': {'table': 'janitor-data', 'column': 'fuel_types'}}, {'Super 98': {'table': 'janitor-data', 'column': 'fuel_types'}}, {'Sultanah': {'table': 'janitor-data', 'column': 'location'}}, {'Buraidah': {'table': 'janitor-data', 'column': 'location'}}, {'Tabuk': {'table': 'janitor-data', 'column': 'location'}}, {'Najran': {'table': 'janitor-data', 'column': 'location'}}, {'Abu Dhabi': {'table': 'janitor-data', 'column': 'location'}}, {'Ajman': {'table': 'janitor-data', 'column': 'location'}}, {'Ras Al Khaimah': {'table': 'janitor-data', 'column': 'location'}}, {'Fujairah': {'table': 'janitor-data', 'column': 'location'}}, {'Umm Al Quwain': {'table': 'janitor-data', 'column': 'location'}}, {'Jeddah': {'table': 'location', 'column': 'Name'}}, {'Riyadh': {'table': 'tenant', 'column': 'NationalCity'}}, {'Medina': {'table': 'janitor-data', 'column': 'location'}}, {'Al Kharj': {'table': 'janitor-data', 'column': 'location'}}, {'Kalba': {'table': 'janitor-data', 'column': 'location'}}, {'Makkah': {'table': 'janitor-data', 'column': 'location'}}, {'Dammam': {'table': 'janitor-data', 'column': 'location'}}, {'Dubai': {'table': 'janitor-data', 'column': 'location'}}, {'Sharjah': {'table': 'janitor-data', 'column': 'location'}}, {'Al Ai': {'table': 'janitor-data', 'column': 'location'}}, {'Khor Fakkan': {'table': 'janitor-data', 'column': 'location'}}, {'Al-Khaleej station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Sharq station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Bahrain station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Fursan station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Safwa station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Sarh station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Nahda station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Fayhaa station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Quds station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Madina station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Qasr station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Riyadh station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Jazira station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Aseel station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Najm station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Dammam station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Waha station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Murooj station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Rabwa station': {'table': 'janitor-data', 'column': 'station_name'}}, {'Al-Mansour station': {'table': 'janitor-data', 'column': 'station_name'}}, {'port-1': {'table': 'ports', 'column': 'Name'}}, {'port-3': {'table': 'ports', 'column': 'Name'}}, {'port-2': {'table': 'ports', 'column': 'Name'}}, {'ahmed ali': {'table': 'region', 'column': 'ArabicName'}}, {'north': {'table': 'region', 'column': 'Name'}}, {'Shift-1': {'table': 'shifts', 'column': 'Name'}}, {'Samra petrol station': {'table': 'station-master', 'column': 'Name'}}, {'Al-Rawi Trading Co.': {'table': 'tenant', 'column': 'CompanyName'}}, {'Al-Olaya': {'table': 'tenant', 'column': 'District'}}, {'Riyadh, KSA': {'table': 'tenant', 'column': 'NationalShortAddress'}}, {'Mohammad Ali Al-Khalidi': {'table': 'tenant', 'column': 'AccountOwnerName'}}, {'123 Al-Nahda Street, Riyadh, Saudi Arabia': {'table': 'tenant', 'column': 'RegisteredAddress'}}, {'Al-Takhassusi Street': {'table': 'tenant', 'column': 'NationalStreet'}}, {'Al-Jazeera Tower': {'table': 'tenant', 'column': 'Building'}}, {'ahmed.al-farsi@example.com': {'table': 'users', 'column': 'Email'}}, {'mohammed.al-sayed@example.com': {'table': 'users', 'column': 'Email'}}, {'ali.al-mansour@example.com': {'table': 'users', 'column': 'Email'}}, {'hassan.al-amin@example.com': {'table': 'users', 'column': 'Email'}}, {'omar.al-harbi@example.com': {'table': 'users', 'column': 'Email'}}, {'abdullah.al-rashid@example.com': {'table': 'users', 'column': 'Email'}}, {'khalid.al-hashemi@example.com': {'table': 'users', 'column': 'Email'}}, {'youssef.al-qahtani@example.com': {'table': 'users', 'column': 'Email'}}, {'ibrahim.al-sharif@example.com': {'table': 'users', 'column': 'Email'}}, {'salem.al-zahrani@example.com': {'table': 'users', 'column': 'Email'}}, {'Hassan': {'table': 'users', 'column': 'FirstName'}}, {'Salem': {'table': 'users', 'column': 'FirstName'}}, {'Al-Farsi': {'table': 'users', 'column': 'LastName'}}, {'Al-Sayed': {'table': 'users', 'column': 'LastName'}}, {'Al-Mansour': {'table': 'users', 'column': 'LastName'}}, {'Al-Amin': {'table': 'users', 'column': 'LastName'}}, {'Al-Harbi': {'table': 'users', 'column': 'LastName'}}, {'Al-Rashid': {'table': 'users', 'column': 'LastName'}}, {'Al-Hashemi': {'table': 'users', 'column': 'LastName'}}, {'Al-Qahtani': {'table': 'users', 'column': 'LastName'}}, {'Al-Sharif': {'table': 'users', 'column': 'LastName'}}, {'Al-Zahrani': {'table': 'users', 'column': 'LastName'}}]

# all_noun_data = ['f6p6D1QG13j172sO', 'Samar', 'Al-Qasr', 'Khalid', 'F9y1Yoc283H33Ae2', 'mahdi.rahma@example.com', 'zi3DR73Q05d3w03L', 'Mohammad Ali Al-Khalidi', 'eman.lubna@example.com', 'Hussein', 'nu49hU1Vn054fh34', 'Al-Fayhaa station', 'jameel.mona@example.com', 'waleed.zein@example.com', 'Asma', 'Super 98', 'Samra petrol station', '6PdRb1z7q831A8I1', 'youssef.al-qahtani@example.com', '99301diz7u3B4MgC', '7lx18tIx262Et10J', 'X07H3sFm028e90jq', '7ds0J91N7TBj642n', 'Omar', 'Al-Jazira station', 'Al-Murooj station', 'Basim', 'Aramco station Zahra', 'Al-Nahda station', 'Ol45J1w2zpp9y419', '9uZ9H11X3E2u7U8d', '47xe6c9QOSkz5358', 'Al-Takhassusi Street', 'Eman', 'Aramco station Baraka', 'Aisha', 'nasser.samar@example.com', 'Al-Nahda', 'Rabwa', 'Ras Al Khaimah', 'zaid.alia@example.com', 'Al-Farsi', '2Tfx60Q6073gse0H', 'Safwa', 'lubna.qasim@example.com', 'Fujairah', 'g9Q37Q0nwLC1478P', 'ziad.bahir@example.com', 'Qhm5xJ359LR20s86', 'R462ZHa59rv1l49T', '4aJ27zCm01Z7t6m8', '3UEfDc33K0q0g346', 'Ismail', 'yasin.abir@example.com', 'Jamilah', '4Y6h8z131z75pWIr', 'W8210bE255lgs2KR', '5e7b58M08BGkGM03', 'Al-Khaleej', 'Farid', '1CaGp270C5900Asw', 'Diesel', 'Al-Qasr station', 'omar.al-harbi@example.com', 'Amal', 'm34H49q52C78jApH', 'malak.rami@example.com', 'Sami', '84trW13bN1NQL423', 'T374I41K9rme7oK2', 'manal.reem@example.com', 'salem.al-zahrani@example.com', 'Al-Sharq station', 'Hasan', 'ahmed.al-farsi@example.com', 'Al-Madina station', '123 Al-Nahda Street, Riyadh, Saudi Arabia', 'Al-Dammam', 'Al-Madina', 'fawzi.munir@example.com', 'f9i53o33byu2H98E', 'Makkah', 'Thamer', 'Riyadh', 'Al-Rashid', 'Jamal', 'Reem', 'rami.tariq@example.com', 'Ayoub', '11QX835Dg9N6c0Jb', '7dGhB4r12a280H9W', 'Mohammed', 'u4vF15aIX8V6u894', 'Aramco station Hilal', 'Al Ai', 'hassan.al-amin@example.com', 'Aramco station Nasr', 'Qasim', 'Al-Dammam station', 'Al-Bahrain station', 'qasim.selim@example.com', 'Manal', 'Rami', 'Aramco station Rasheed', 'Al-Rawi Trading Co.', '0R660631MBM5keIx', 'Al-Rabwa station', 'Farah', 'Kareem', 'Sanaa', 'reem.thamer@example.com', 'Mustafa', 'selim.youssef@example.com', 'Al-Fursan station', 'Bilal', 'Zein', 'Al-Bahrain', 'wKPc8X636ypz7580', 'iman.malak@example.com', 'zahra.amal@example.com', 'samar.yara@example.com', 'Medina', 'lina.noor@example.com', 'Yara', 'Shift-1', 'port-1', 'Ali', '7593ni40Fnto3f3A', 'Jameel', 'abir.ehsan@example.com', 'Aramco station Amal', 'Q986LG08HuzE3p18', 'r57h71b10h88FzlL', 'Buraidah', 'Sarah', 'Al-Sharif', 'ROV1S19373K9VT0W', 'Riham', 'Abu Dhabi', 'Abdulrahman', 'Al-Riyadh', 'youssef.afaf@example.com', 'warda.ziad@example.com', '24YW6215n65iQgaS', 'Haifa', '6ny26U33jn0qi6Y4', 'Al-Khaleej station', 'FSg201eN24vPI416', 'yara.zuhair@example.com', 'Nadia', 'YxI96Y606729stYT', 'Y4eqf664rw5xE865', 'port-2', '4L6G27aPdM94Lu93', '8m7E7G9Z66Tw25IA', '6rg5okO6t45W3q91', 'Ajman', '52s891CZS0X8J4Dr', 'Al-Sarh', 'Afaf', '7tyu38kg8TQ1247Z', 'Lina', 'lamia.nasser@example.com', '96OEh99uro82r62a', 'qK05dF6iGX66B745', 'Sharjah', 'W2M2O7MX6J0q57F4', 'Dalal', '2Y1R4a11YMz132SE', 'Al-Sayed', 'Bahir', 'Abir', '5Jr190L24WzUUS96', 'Najran', 'Najm', 'Jazira', 'Al-Olaya', 'oY53q55RkW0i3K17', 'sr9A5h7K137n8G6l', 'Aramco station Noor', '6Q3C0eJg47fn49A3', 'Nabila', 'Riyadh, KSA', '98yG06zI836ks2JY', '1B8e46w1L5fXY8M0', 'Aramco station Huda', 'Youssef', 'souad.zaid@example.com', 'ali.al-mansour@example.com', 'riham.waleed@example.com', 'Zahra', 'jS1HB61b669P6r8t', 'farah.marwa@example.com', 'Noor', 'zaki.ayoub@example.com', 'Hana', 'Al-Qahtani', 'mustafa.saeed@example.com', 'Al-Zahrani', 'H4l1Nc7S8382n4zD', 'qZ8074F3IM90H3cV', 'afaf.eman@example.com', 'c66Q97UM7y755xdd', 'Warda', 'mohammed.al-sayed@example.com', 'Sultanah', 'Yasin', 'i3S7T4v3o0Ty45u7', 'Aramco station Safa', 'Tabuk', '5t9bNVd4ro68178D', 'Al-Aseel station', 'Al Kharj', 'Aziza', 'Hala', 'Anwar', 'dalal.kawthar@example.com', '247Ry0CKk92N8ND5', 'dh29LfN358I5Ts98', 'R20J9u8n496gUK8g', 'Iman', 'Aramco station Kawthar', 'hamid.lubna@example.com', 'Ziad', 'Lamia', 'Tariq', 'Aramco station Taqwa', 'Amira', 'ehsan.leen@example.com', 'Aramco station Fajr', 'Rasha', 'Saeed', 'Adnan', 'HS6C9Ky7p754SJ41', '95', 'Al-Sarh station', 'Al-Harbi', 'Dubai', 'Mariam', '4NJu90bVf7l749y8', 'Abdulaziz', '6J755E72u8Lqgi5M', 'rahma.souad@example.com', 'Q5R1cIZ6m75h98A9', 'Sharq', '4673J0MH9lYH87jR', 'Mahdi', 'Fawzi', 'Umm Al Quwain', 'Zuhair', 'Al-Mansour', 'Lubna', 'V64R8b89O9jj08rM', 'Abdullah', 'Kalba', 'Al-Riyadh station', 's612237J0TF0BXGf', 'Rahma', 'Hamid', 'Malak', 'Aramco station Azhar', 'Khor Fakkan', 'Al-Quds station', 'Alia', 'Ibrahim', 'Aramco station Mahd', 'djM12a9G9790sb9f', 'Dammam', 'noor.sanaa@example.com', 'uGex454XQ98e25b2', 'Fatima', 'Al-Murooj', 'Dina', 'Aramco station Bustan', 'kareem.mustafa@example.com', 'Z5MG3o2GoA25572p', 'Layla', 'Ehsan', 'Waleed', 'Rania', '4Dgd5770AfqdH901', 'ahmed ali', 'Al-Waha station', 'Zaid', 'Al-Hashemi', 'Selim', 'zuhair.dalal@example.com', 'z78N9t49h8q2Rd7o', 'Zainab', 'ibrahim.al-sharif@example.com', 'Yasmin', 'P0a2Am70C2c70re1', 'Hassan', 'Ahmed', 'Ghada', 'Adam', 'Salim', 'DH858xQ2bUA1M192', 'Al-Fursan', 'Azhar', 'Al-Jazeera Tower', 'hana.mahdi@example.com', 'Al-Mansour station', 'Nasser', 'Al-Waha', '91i2qv930Z0BpT7Q', 's2NA2LsY78z0771A', '8G96wBCm4i5152CB', '54r8K2oN3PH3A1H3', '9b2xV05K1E068Vsy', 'Al-Najm station', 'saeed.warda@example.com', 'Aramco station Manara', 'Salma', 'khalid.al-hashemi@example.com', 'north', 'ET564XL393t0S0ZA', 'sanaa.yasin@example.com', 'Al-Amin', 'zein.azhar@example.com', 'Jeddah', 'EPlus 91', 'abdullah.al-rashid@example.com', '54p0jW7445Ao7Mve', '91', 'Amina', 'tariq.zahra@example.com', '6b52akdf0b0r4s39', 'thamer.zaki@example.com', '5Xl43A04jx7lSB91', 'Special 95', 'm3V81K1457HD6yFV', 'Zaki', 'ismail.manal@example.com', 'Al-Safwa station', '91qq0UV14DL42IE5', 'i08N207lFgw5i00i', 'eAY64453Enf7Dg14', 'port-3', '6x4rU4L47Z59V3PQ', 'Souad', 'Salem', 'Mona', 'mona.riham@example.com', 'p6P16258Ckx7L0Kl']
# # noun_maps
# # all_noun_data

## Translated Docs

In [ ]:
from google.cloud import translate_v2 as translate

######################
"""
Basic Translator 
1. For getting initial translation of the user query 
2. For getting Entity translations for fuzzy and vector search # NEEDS human verification
above will be provided as context to LLM in advanced translator

IP:
1. Text to be translated 
2. Taregt language to be translate to

OP:
Translated text

"""
######################
def translate_text(text: str, tgt: str) -> str:
    try:
        logging.debug(f"translating text: {text[:50]}... to {tgt}")
        # Initialize Google Cloud Translation client
        translate_client = translate.Client()

        # Ensure text is in string format
        if isinstance(text, bytes):
            text = text.decode("utf-8")

        # Perform translation
        result = translate_client.translate(text, target_language=tgt)
        translated_text = result.get("translatedText", "")

        # Log successful translation
        logging.info(f"Successfully translated text to: {translated_text[:50]}...")

        return translated_text
    except Exception as e:
        # Log error
        logging.error(f"Error translating text: {text[:50]}... to {tgt} - {e}")
        return ""
######################
"""
Translate Entity:(SRF)
Provides tranlsation of entities. 
Used for mapping translated entites to entities in database(en)

IP:
1. List of entities 
2. Taregt language to be translate to

OP:
1. List of dicts [{en:ar}, {en:ar}] 
2. list of translated entities
"""
######################

def get_entity_translations(datas, tgt_lang):
    logging.info("Creating entity : translations map")
    logging.debug(f"Target language : {tgt_lang}")

    translate_map_list = []
    translated_texts_list = []

    for data in datas:
        if not data:
            continue

        try:
            # Translate text and build map
            translated_texts = translate_text(data, tgt_lang)
            if translated_texts:
                translate_map = {data: translated_texts}
                translated_texts_list.append(translated_texts)
                translate_map_list.append(translate_map)
            else:
                logging.warning(f"Translation failed for text: {data[:50]}...")

        except Exception as e:
            # Log error during translation
            logging.error(f"Error processing translation for {data[:50]}...: {e}")
    logging.info("Completed entity : translations map")
    return translate_map_list, translated_texts_list



In [ ]:
# entity_translate_map, translated_texts_list = get_entity_translations(all_noun_data, "ar")

In [ ]:
entity_translate_map = [{'f6p6D1QG13j172sO': 'f6p6D1QG13j172sO'}, {'Samar': 'إنتاج'}, {'Al-Qasr': 'Al-Qasr'}, {'Khalid': 'خالد'}, {'F9y1Yoc283H33Ae2': 'F9y1Yoc283H33Ae2'}, {'mahdi.rahma@example.com': 'mahdi.rahma@example.com'}, {'zi3DR73Q05d3w03L': 'zi3DR73Q05d3w03L'}, {'Mohammad Ali Al-Khalidi': 'Mohammad Ali Al-Khalidi'}, {'eman.lubna@example.com': 'eman.lubna@example.com'}, {'Hussein': 'حسين'}, {'nu49hU1Vn054fh34': 'نو49hU1Vn054fh34'}, {'Al-Fayhaa station': 'محطة الفيحاء'}, {'jameel.mona@example.com': 'jameel.mona@example.com'}, {'waleed.zein@example.com': 'waleed.zein@example.com'}, {'Asma': 'الربو'}, {'Super 98': 'سوبر 98'}, {'Samra petrol station': 'محطة بنزين سمرة'}, {'6PdRb1z7q831A8I1': '6PdRb1z7q831A8I1'}, {'youssef.al-qahtani@example.com': 'youssef.al-qahtani@example.com'}, {'99301diz7u3B4MgC': '99301diz7u3B4MgC'}, {'7lx18tIx262Et10J': '7lx18tIx262Et10J'}, {'X07H3sFm028e90jq': 'X07H3sFm028e90jq'}, {'7ds0J91N7TBj642n': '7ds0J91N7TBj642n'}, {'Omar': 'عمر'}, {'Al-Jazira station': 'محطة الجزيرة'}, {'Al-Murooj station': 'محطة المروج'}, {'Basim': 'باسم'}, {'Aramco station Zahra': 'محطة ارامكو الزهراء'}, {'Al-Nahda station': 'Al-Nahda station'}, {'Ol45J1w2zpp9y419': 'Ol45J1w2zpp9y419'}, {'9uZ9H11X3E2u7U8d': '9uZ9H11X3E2u7U8d'}, {'47xe6c9QOSkz5358': '47xe6c9QOSkz5358'}, {'Al-Takhassusi Street': 'شارع التخصصي'}, {'Eman': 'يعطي'}, {'Aramco station Baraka': 'محطة ارامكو البركة'}, {'Aisha': 'عائشة'}, {'nasser.samar@example.com': 'nasser.samar@example.com'}, {'Al-Nahda': 'Al-Nahda'}, {'Rabwa': 'رابا'}, {'Ras Al Khaimah': 'Ras Al Khaimah'}, {'zaid.alia@example.com': 'zaid.alia@example.com'}, {'Al-Farsi': 'الفارسي'}, {'2Tfx60Q6073gse0H': '2Tfx60Q6073gse0H'}, {'Safwa': 'بالطبع'}, {'lubna.qasim@example.com': 'lubna.qasim@example.com'}, {'Fujairah': 'الفجيرة'}, {'g9Q37Q0nwLC1478P': 'g9Q37Q0nwLC1478P'}, {'ziad.bahir@example.com': 'ziad.bahir@example.com'}, {'Qhm5xJ359LR20s86': 'Qhm5xJ359LR20s86'}, {'R462ZHa59rv1l49T': 'R462ZHa59rv1l49T'}, {'4aJ27zCm01Z7t6m8': '4aJ27zCm01Z7t6m8'}, {'3UEfDc33K0q0g346': '3UEfDc33K0q0g346'}, {'Ismail': 'اسماعيل'}, {'yasin.abir@example.com': 'yasin.abir@example.com'}, {'Jamilah': 'جميلة'}, {'4Y6h8z131z75pWIr': '4Y6h8z131z75pWIr'}, {'W8210bE255lgs2KR': 'W8210bE255lgs2KR'}, {'5e7b58M08BGkGM03': '5e7b58M08BGkGM03'}, {'Al-Khaleej': 'الخليج'}, {'Farid': 'فريد'}, {'1CaGp270C5900Asw': '1CaGp270C5900Asw'}, {'Diesel': 'ديزل'}, {'Al-Qasr station': 'محطة القصر'}, {'omar.al-harbi@example.com': 'omar.al-harbi@example.com'}, {'Amal': 'صدقة'}, {'m34H49q52C78jApH': 'm34H49q52C78jApH'}, {'malak.rami@example.com': 'malak.rami@example.com'}, {'Sami': 'سامي'}, {'84trW13bN1NQL423': '84trW13bN1NQL423'}, {'T374I41K9rme7oK2': 'T374I41K9rme7oK2'}, {'manal.reem@example.com': 'manal.reem@example.com'}, {'salem.al-zahrani@example.com': 'Salem.al-zahrani@example.com'}, {'Al-Sharq station': 'محطة الشرق'}, {'Hasan': 'حسن'}, {'ahmed.al-farsi@example.com': 'ahmed.al-farsi@example.com'}, {'Al-Madina station': 'محطة المدينة المنورة'}, {'123 Al-Nahda Street, Riyadh, Saudi Arabia': '123 Al-Nahda Street, Riyadh, Saudi Arabia'}, {'Al-Dammam': 'Al-Dammam'}, {'Al-Madina': 'Al-Madina'}, {'fawzi.munir@example.com': 'fawzi.munir@example.com'}, {'f9i53o33byu2H98E': 'f9i53o33byu2H98E'}, {'Makkah': 'مكه'}, {'Thamer': 'ثامر'}, {'Riyadh': 'الرياض'}, {'Al-Rashid': 'Al-Rashid'}, {'Jamal': 'جمال'}, {'Reem': 'ريم'}, {'rami.tariq@example.com': 'rami.tariq@example.com'}, {'Ayoub': 'أيوب'}, {'11QX835Dg9N6c0Jb': '11QX835Dg9N6c0Jb'}, {'7dGhB4r12a280H9W': '7dGhB4r12a280H9W'}, {'Mohammed': 'محمد'}, {'u4vF15aIX8V6u894': 'u4vF15aIX8V6u894'}, {'Aramco station Hilal': 'محطة أرامكو الهلال'}, {'Al Ai': 'آل عاي'}, {'hassan.al-amin@example.com': 'hassan.al-amin@example.com'}, {'Aramco station Nasr': 'محطة ارامكو النصر'}, {'Qasim': 'قاسم'}, {'Al-Dammam station': 'محطة الدمام'}, {'Al-Bahrain station': 'محطة البحرين'}, {'qasim.selim@example.com': 'qasim.selim@example.com'}, {'Manal': 'منال'}, {'Rami': 'رامي'}, {'Aramco station Rasheed': 'محطة ارامكو الرشيد'}, {'Al-Rawi Trading Co.': 'شركة الراوي التجارية'}, {'0R660631MBM5keIx': '0R660631MBM5keIx'}, {'Al-Rabwa station': 'محطة الربوة'}, {'Farah': 'فرح'}, {'Kareem': 'كريم'}, {'Sanaa': 'فن'}, {'reem.thamer@example.com': 'reem.thamer@example.com'}, {'Mustafa': 'مصطفى'}, {'selim.youssef@example.com': 'selim.youssef@example.com'}, {'Al-Fursan station': 'محطة الفرسان'}, {'Bilal': 'بلال'}, {'Zein': 'أيّ'}, {'Al-Bahrain': 'Al-Bahrain'}, {'wKPc8X636ypz7580': 'wKPc8X636ypz7580'}, {'iman.malak@example.com': 'iman.malak@example.com'}, {'zahra.amal@example.com': 'zahra.amal@example.com'}, {'samar.yara@example.com': 'samar.yara@example.com'}, {'Medina': 'المدينة المنورة'}, {'lina.noor@example.com': 'lina.noor@example.com'}, {'Yara': 'أطفال'}, {'Shift-1': 'التحول-1'}, {'port-1': 'المنفذ 1'}, {'Ali': 'أو'}, {'7593ni40Fnto3f3A': '7593ni40Fnto3f3A'}, {'Jameel': 'جميل'}, {'abir.ehsan@example.com': 'abir.ehsan@example.com'}, {'Aramco station Amal': 'محطة أرامكو أمل'}, {'Q986LG08HuzE3p18': 'Q986LG08HuzE3p18'}, {'r57h71b10h88FzlL': 'r57h71b10h88FzlL'}, {'Buraidah': 'بريدة'}, {'Sarah': 'ساره'}, {'Al-Sharif': 'Al-Sharif'}, {'ROV1S19373K9VT0W': 'ROV1S19373K9VT0W'}, {'Riham': 'ريهام'}, {'Abu Dhabi': 'Abu Dhabi'}, {'Abdulrahman': 'عبدالرحمن'}, {'Al-Riyadh': 'Al-Riyadh'}, {'youssef.afaf@example.com': 'youssef.afaf@example.com'}, {'warda.ziad@example.com': 'warda.ziad@example.com'}, {'24YW6215n65iQgaS': '24YW6215n65iQgaS'}, {'Haifa': 'حيفا'}, {'6ny26U33jn0qi6Y4': '6ny26U33jn0qi6Y4'}, {'Al-Khaleej station': 'محطة الخليج'}, {'FSg201eN24vPI416': 'FSg201eN24vPI416'}, {'yara.zuhair@example.com': 'yara.zuhair@example.com'}, {'Nadia': 'ناديه'}, {'YxI96Y606729stYT': 'YxI96Y606729stYT'}, {'Y4eqf664rw5xE865': 'Y4eqf664rw5xE865'}, {'port-2': 'المنفذ 2'}, {'4L6G27aPdM94Lu93': '4L6G27aPdM94Lu93'}, {'8m7E7G9Z66Tw25IA': '8m7E7G9Z66Tw25IA'}, {'6rg5okO6t45W3q91': '6rg5okO6t45W3q91'}, {'Ajman': 'عجمان'}, {'52s891CZS0X8J4Dr': '52s891CZS0X8J4 د'}, {'Al-Sarh': 'الصرح'}, {'Afaf': 'اللغات'}, {'7tyu38kg8TQ1247Z': '7tyu38kg8TQ1247Z'}, {'Lina': 'لينا'}, {'lamia.nasser@example.com': 'lamia.nasser@example.com'}, {'96OEh99uro82r62a': '96OEh99uro82r62a'}, {'qK05dF6iGX66B745': 'qK05dF6iGX66B745'}, {'Sharjah': 'الشارقة'}, {'W2M2O7MX6J0q57F4': 'W2M2O7MX6J0q57F4'}, {'Dalal': 'بلدان'}, {'2Y1R4a11YMz132SE': '2Y1R4a11YMz132SE'}, {'Al-Sayed': 'Al-Sayed'}, {'Bahir': 'باهر'}, {'Abir': 'عبير'}, {'5Jr190L24WzUUS96': '5Jr190L24WzUUS96'}, {'Najran': 'نجران'}, {'Najm': 'بخاصة'}, {'Jazira': 'جزيرة'}, {'Al-Olaya': 'Al-Olaya'}, {'oY53q55RkW0i3K17': 'oY53q55RkW0i3K17'}, {'sr9A5h7K137n8G6l': 'sr9A5h7K137n8G6l'}, {'Aramco station Noor': 'محطة أرامكو نور'}, {'6Q3C0eJg47fn49A3': '6Q3C0eJg47fn49A3'}, {'Nabila': 'نبيلة'}, {'Riyadh, KSA': 'الرياض، المملكة العربية السعودية'}, {'98yG06zI836ks2JY': '98yG06zI836ks2JY'}, {'1B8e46w1L5fXY8M0': '1B8e46w1L5fXY8M0'}, {'Aramco station Huda': 'محطة أرامكو الهدى'}, {'Youssef': 'يوسف'}, {'souad.zaid@example.com': 'souad.zaid@example.com'}, {'ali.al-mansour@example.com': 'ali.al-mansour@example.com'}, {'riham.waleed@example.com': 'riham.waleed@example.com'}, {'Zahra': 'الزهراء'}, {'jS1HB61b669P6r8t': 'jS1HB61b669P6r8t'}, {'farah.marwa@example.com': 'فرح.ماروا@example.com'}, {'Noor': 'نور'}, {'zaki.ayoub@example.com': 'zaki.ayoub@example.com'}, {'Hana': 'عمل'}, {'Al-Qahtani': 'Al-Qahtani'}, {'mustafa.saeed@example.com': 'mustafa.saeed@example.com'}, {'Al-Zahrani': 'Al-Zahrani'}, {'H4l1Nc7S8382n4zD': 'H4l1Nc7S8382n4zD'}, {'qZ8074F3IM90H3cV': 'qZ8074F3IM90H3cV'}, {'afaf.eman@example.com': 'afaf.eman@example.com'}, {'c66Q97UM7y755xdd': 'c66Q97UM7y755xdd'}, {'Warda': 'وردة'}, {'mohammed.al-sayed@example.com': 'mohammed.al-sayed@example.com'}, {'Sultanah': 'سلطانة'}, {'Yasin': 'ياسين'}, {'i3S7T4v3o0Ty45u7': 'i3S7T4v3o0Ty45u7'}, {'Aramco station Safa': 'محطة ارامكو الصفا'}, {'Tabuk': 'تبوك'}, {'5t9bNVd4ro68178D': '5t9bNVd4ro68178D'}, {'Al-Aseel station': 'محطة الأصيل'}, {'Al Kharj': 'Al Kharj'}, {'Aziza': 'عزيزي'}, {'Hala': 'لذا'}, {'Anwar': 'أنور'}, {'dalal.kawthar@example.com': 'dalal.kawthar@example.com'}, {'247Ry0CKk92N8ND5': '247Ry0CKk92N8ND5'}, {'dh29LfN358I5Ts98': 'dh29LfN358I5Ts98'}, {'R20J9u8n496gUK8g': 'R20J9u8n496gUK8g'}, {'Iman': 'إيمان'}, {'Aramco station Kawthar': 'Aramco station Kawthar'}, {'hamid.lubna@example.com': 'hamid.lubna@example.com'}, {'Ziad': 'زياد'}, {'Lamia': 'لمياء'}, {'Tariq': 'طارق'}, {'Aramco station Taqwa': 'محطة أرامكو التقوى'}, {'Amira': 'أميرة'}, {'ehsan.leen@example.com': 'ehsan.leen@example.com'}, {'Aramco station Fajr': 'محطة أرامكو الفجر'}, {'Rasha': 'روسيا'}, {'Saeed': 'سعيد'}, {'Adnan': 'عدنان'}, {'HS6C9Ky7p754SJ41': 'HS6C9Ky7p754SJ41'}, {'95': '95'}, {'Al-Sarh station': 'محطة الصرح'}, {'Al-Harbi': 'Al-Harbi'}, {'Dubai': 'دبي'}, {'Mariam': 'مريم'}, {'4NJu90bVf7l749y8': '4NJu90bVf7l749y8'}, {'Abdulaziz': 'عبد العزيز'}, {'6J755E72u8Lqgi5M': '6J755E72u8Lqgi5M'}, {'rahma.souad@example.com': 'rahma.souad@example.com'}, {'Q5R1cIZ6m75h98A9': 'Q5R1cIZ6m75h98A9'}, {'Sharq': 'شرق'}, {'4673J0MH9lYH87jR': '4673J0MH9lYH87jR'}, {'Mahdi': 'مهدي'}, {'Fawzi': 'فوزي'}, {'Umm Al Quwain': 'ام القيوين'}, {'Zuhair': 'كوكب الزهرة'}, {'Al-Mansour': 'Al-Mansour'}, {'Lubna': 'لبنى'}, {'V64R8b89O9jj08rM': 'V64R8b89O9jj08rM'}, {'Abdullah': 'عبد الله'}, {'Kalba': 'لغة'}, {'Al-Riyadh station': 'محطة الرياض'}, {'s612237J0TF0BXGf': 's612237J0TF0BXGf'}, {'Rahma': 'رحمة'}, {'Hamid': 'حميد'}, {'Malak': 'ملاك'}, {'Aramco station Azhar': 'محطة أرامكو الأزهر'}, {'Khor Fakkan': 'خورفكان'}, {'Al-Quds station': 'محطة القدس'}, {'Alia': 'علياء'}, {'Ibrahim': 'ابراهيم'}, {'Aramco station Mahd': 'محطة أرامكو المهد'}, {'djM12a9G9790sb9f': 'djM12a9G9790sb9f'}, {'Dammam': 'الدمام'}, {'noor.sanaa@example.com': 'noor.sanaa@example.com'}, {'uGex454XQ98e25b2': 'uGex454XQ98e25b2'}, {'Fatima': 'فاطمة'}, {'Al-Murooj': 'المروس'}, {'Dina': 'حقير'}, {'Aramco station Bustan': 'محطة أرامكو البستان'}, {'kareem.mustafa@example.com': 'kareem.mustafa@example.com'}, {'Z5MG3o2GoA25572p': 'Z5MG3o2GoA25572p'}, {'Layla': 'ليلى'}, {'Ehsan': 'كياسة'}, {'Waleed': 'وليد'}, {'Rania': 'رانيا'}, {'4Dgd5770AfqdH901': '4Dgd5770AfqdH901'}, {'ahmed ali': 'ahmed ali'}, {'Al-Waha station': 'محطة الواحة'}, {'Zaid': 'زيد'}, {'Al-Hashemi': 'Al-Hashemi'}, {'Selim': 'سليم'}, {'zuhair.dalal@example.com': 'zuhair.dalal@example.com'}, {'z78N9t49h8q2Rd7o': 'z78N9t49h8q2Rd7o'}, {'Zainab': 'زينب'}, {'ibrahim.al-sharif@example.com': 'ibrahim.al-sharif@example.com'}, {'Yasmin': 'ياسمين'}, {'P0a2Am70C2c70re1': 'P0a2Am70C2c70re1'}, {'Hassan': 'حسان'}, {'Ahmed': 'أحمد'}, {'Ghada': 'غداً'}, {'Adam': 'آدم'}, {'Salim': 'سليم'}, {'DH858xQ2bUA1M192': 'DH858xQ2bUA1M192'}, {'Al-Fursan': 'Al-Fursan'}, {'Azhar': 'أزهر'}, {'Al-Jazeera Tower': 'برج الجزيرة'}, {'hana.mahdi@example.com': 'hana.mahdi@example.com'}, {'Al-Mansour station': 'محطة المنصور'}, {'Nasser': 'ناصر'}, {'Al-Waha': 'Al-Waha'}, {'91i2qv930Z0BpT7Q': '91i2qv930Z0BpT7Q'}, {'s2NA2LsY78z0771A': 's2NA2LsY78z0771A'}, {'8G96wBCm4i5152CB': '8G96wBCm4i5152CB'}, {'54r8K2oN3PH3A1H3': '54r8K2oN3PH3A1H3'}, {'9b2xV05K1E068Vsy': '9b2xV05K1E068Vsy'}, {'Al-Najm station': 'محطة النجم'}, {'saeed.warda@example.com': 'saeed.warda@example.com'}, {'Aramco station Manara': 'محطة أرامكو المنارة'}, {'Salma': 'سلمى'}, {'khalid.al-hashemi@example.com': 'khalid.al-hashemi@example.com'}, {'north': 'شمال'}, {'ET564XL393t0S0ZA': 'ET564XL393t0S0ZA'}, {'sanaa.yasin@example.com': 'sanaa.yasin@example.com'}, {'Al-Amin': 'Al-Amin'}, {'zein.azhar@example.com': 'zein.azhar@example.com'}, {'Jeddah': 'جدة'}, {'EPlus 91': 'إي بلس 91'}, {'abdullah.al-rashid@example.com': 'abdallah.al-rashid@example.com'}, {'54p0jW7445Ao7Mve': '54p0jW7445Ao7Mve'}, {'91': '91'}, {'Amina': 'آمين'}, {'tariq.zahra@example.com': 'tariq.zahra@example.com'}, {'6b52akdf0b0r4s39': '6b52akdf0b0r4s39'}, {'thamer.zaki@example.com': 'thamer.zaki@example.com'}, {'5Xl43A04jx7lSB91': '5Xl43A04jx7lSB91'}, {'Special 95': 'خاص 95'}, {'m3V81K1457HD6yFV': 'm3V81K1457HD6yFV'}, {'Zaki': 'أسد'}, {'ismail.manal@example.com': 'ismail.manal@example.com'}, {'Al-Safwa station': 'Al-Safwa station'}, {'91qq0UV14DL42IE5': '91qq0UV14DL42IE5'}, {'i08N207lFgw5i00i': 'i08N207lFgw5i00i'}, {'eAY64453Enf7Dg14': 'eAY64453Enf7Dg14'}, {'port-3': 'المنفذ 3'}, {'6x4rU4L47Z59V3PQ': '6x4rU4L47Z59V3PQ'}, {'Souad': 'سعاد'}, {'Salem': 'سالم'}, {'Mona': 'منى'}, {'mona.riham@example.com': 'mona.riham@example.com'}, {'p6P16258Ckx7L0Kl': 'p6P16258Ckx7L0Kl'}]
translated_texts_list = ['f6p6D1QG13j172sO', 'إنتاج', 'Al-Qasr', 'خالد', 'F9y1Yoc283H33Ae2', 'mahdi.rahma@example.com', 'zi3DR73Q05d3w03L', 'Mohammad Ali Al-Khalidi', 'eman.lubna@example.com', 'حسين', 'نو49hU1Vn054fh34', 'محطة الفيحاء', 'jameel.mona@example.com', 'waleed.zein@example.com', 'الربو', 'سوبر 98', 'محطة بنزين سمرة', '6PdRb1z7q831A8I1', 'youssef.al-qahtani@example.com', '99301diz7u3B4MgC', '7lx18tIx262Et10J', 'X07H3sFm028e90jq', '7ds0J91N7TBj642n', 'عمر', 'محطة الجزيرة', 'محطة المروج', 'باسم', 'محطة ارامكو الزهراء', 'Al-Nahda station', 'Ol45J1w2zpp9y419', '9uZ9H11X3E2u7U8d', '47xe6c9QOSkz5358', 'شارع التخصصي', 'يعطي', 'محطة ارامكو البركة', 'عائشة', 'nasser.samar@example.com', 'Al-Nahda', 'رابا', 'Ras Al Khaimah', 'zaid.alia@example.com', 'الفارسي', '2Tfx60Q6073gse0H', 'بالطبع', 'lubna.qasim@example.com', 'الفجيرة', 'g9Q37Q0nwLC1478P', 'ziad.bahir@example.com', 'Qhm5xJ359LR20s86', 'R462ZHa59rv1l49T', '4aJ27zCm01Z7t6m8', '3UEfDc33K0q0g346', 'اسماعيل', 'yasin.abir@example.com', 'جميلة', '4Y6h8z131z75pWIr', 'W8210bE255lgs2KR', '5e7b58M08BGkGM03', 'فريد', '1CaGp270C5900Asw', 'ديزل', 'محطة القصر', 'omar.al-harbi@example.com', 'صدقة', 'm34H49q52C78jApH', 'malak.rami@example.com', 'سامي', '84trW13bN1NQL423', 'T374I41K9rme7oK2', 'manal.reem@example.com', 'Salem.al-zahrani@example.com', 'محطة الشرق', 'حسن', 'ahmed.al-farsi@example.com', 'محطة المدينة المنورة', '123 Al-Nahda Street, Riyadh, Saudi Arabia', 'Al-Dammam', 'Al-Madina', 'fawzi.munir@example.com', 'f9i53o33byu2H98E', 'مكه', 'ثامر', 'الرياض', 'Al-Rashid', 'جمال', 'ريم', 'rami.tariq@example.com', 'أيوب', '11QX835Dg9N6c0Jb', '7dGhB4r12a280H9W', 'محمد', 'u4vF15aIX8V6u894', 'محطة أرامكو الهلال', 'آل عاي', 'hassan.al-amin@example.com', 'محطة ارامكو النصر', 'قاسم', 'محطة الدمام', 'محطة البحرين', 'qasim.selim@example.com', 'منال', 'رامي', 'محطة ارامكو الرشيد', 'شركة الراوي التجارية', '0R660631MBM5keIx', 'محطة الربوة', 'فرح', 'كريم', 'فن', 'reem.thamer@example.com', 'مصطفى', 'selim.youssef@example.com', 'محطة الفرسان', 'بلال', 'أيّ', 'Al-Bahrain', 'wKPc8X636ypz7580', 'iman.malak@example.com', 'zahra.amal@example.com', 'samar.yara@example.com', 'المدينة المنورة', 'lina.noor@example.com', 'أطفال', 'التحول-1', 'المنفذ 1', 'أو', '7593ni40Fnto3f3A', 'جميل', 'abir.ehsan@example.com', 'محطة أرامكو أمل', 'Q986LG08HuzE3p18', 'r57h71b10h88FzlL', 'بريدة', 'ساره', 'Al-Sharif', 'ROV1S19373K9VT0W', 'ريهام', 'Abu Dhabi', 'عبدالرحمن', 'Al-Riyadh', 'youssef.afaf@example.com', 'warda.ziad@example.com', '24YW6215n65iQgaS', 'حيفا', '6ny26U33jn0qi6Y4', 'محطة الخليج', 'FSg201eN24vPI416', 'yara.zuhair@example.com', 'ناديه', 'YxI96Y606729stYT', 'Y4eqf664rw5xE865', 'المنفذ 2', '4L6G27aPdM94Lu93', '8m7E7G9Z66Tw25IA', '6rg5okO6t45W3q91', 'عجمان', '52s891CZS0X8J4 د', 'الصرح', 'اللغات', '7tyu38kg8TQ1247Z', 'لينا', 'lamia.nasser@example.com', '96OEh99uro82r62a', 'qK05dF6iGX66B745', 'الشارقة', 'W2M2O7MX6J0q57F4', 'بلدان', '2Y1R4a11YMz132SE', 'Al-Sayed', 'باهر', 'عبير', '5Jr190L24WzUUS96', 'نجران', 'بخاصة', 'جزيرة', 'Al-Olaya', 'oY53q55RkW0i3K17', 'sr9A5h7K137n8G6l', 'محطة أرامكو نور', '6Q3C0eJg47fn49A3', 'نبيلة', 'الرياض، المملكة العربية السعودية', '98yG06zI836ks2JY', '1B8e46w1L5fXY8M0', 'محطة أرامكو الهدى', 'يوسف', 'souad.zaid@example.com', 'ali.al-mansour@example.com', 'riham.waleed@example.com', 'الزهراء', 'jS1HB61b669P6r8t', 'فرح.ماروا@example.com', 'نور', 'zaki.ayoub@example.com', 'عمل', 'Al-Qahtani', 'mustafa.saeed@example.com', 'Al-Zahrani', 'H4l1Nc7S8382n4zD', 'qZ8074F3IM90H3cV', 'afaf.eman@example.com', 'c66Q97UM7y755xdd', 'وردة', 'mohammed.al-sayed@example.com', 'سلطانة', 'ياسين', 'i3S7T4v3o0Ty45u7', 'محطة ارامكو الصفا', 'تبوك', '5t9bNVd4ro68178D', 'محطة الأصيل', 'Al Kharj', 'عزيزي', 'لذا', 'أنور', 'dalal.kawthar@example.com', '247Ry0CKk92N8ND5', 'dh29LfN358I5Ts98', 'R20J9u8n496gUK8g', 'إيمان', 'Aramco station Kawthar', 'hamid.lubna@example.com', 'زياد', 'لمياء', 'طارق', 'محطة أرامكو التقوى', 'أميرة', 'ehsan.leen@example.com', 'محطة أرامكو الفجر', 'روسيا', 'سعيد', 'عدنان', 'HS6C9Ky7p754SJ41', '95', 'محطة الصرح', 'Al-Harbi', 'دبي', 'مريم', '4NJu90bVf7l749y8', 'عبد العزيز', '6J755E72u8Lqgi5M', 'rahma.souad@example.com', 'Q5R1cIZ6m75h98A9', 'شرق', '4673J0MH9lYH87jR', 'مهدي', 'فوزي', 'ام القيوين', 'كوكب الزهرة', 'Al-Mansour', 'لبنى', 'V64R8b89O9jj08rM', 'عبد الله', 'لغة', 'محطة الرياض', 's612237J0TF0BXGf', 'رحمة', 'حميد', 'ملاك', 'محطة أرامكو الأزهر', 'خورفكان', 'محطة القدس', 'علياء', 'ابراهيم', 'محطة أرامكو المهد', 'djM12a9G9790sb9f', 'الدمام', 'noor.sanaa@example.com', 'uGex454XQ98e25b2', 'فاطمة', 'المروس', 'حقير', 'محطة أرامكو البستان', 'kareem.mustafa@example.com', 'Z5MG3o2GoA25572p', 'ليلى', 'كياسة', 'وليد', 'رانيا', '4Dgd5770AfqdH901', 'ahmed ali', 'محطة الواحة', 'زيد', 'Al-Hashemi', 'سليم', 'zuhair.dalal@example.com', 'z78N9t49h8q2Rd7o', 'زينب', 'ibrahim.al-sharif@example.com', 'ياسمين', 'P0a2Am70C2c70re1', 'حسان', 'أحمد', 'غداً', 'آدم', 'سليم', 'DH858xQ2bUA1M192', 'Al-Fursan', 'أزهر', 'برج الجزيرة', 'hana.mahdi@example.com', 'محطة المنصور', 'ناصر', 'Al-Waha', '91i2qv930Z0BpT7Q', 's2NA2LsY78z0771A', '8G96wBCm4i5152CB', '54r8K2oN3PH3A1H3', '9b2xV05K1E068Vsy', 'محطة النجم', 'saeed.warda@example.com', 'محطة أرامكو المنارة', 'سلمى', 'khalid.al-hashemi@example.com', 'شمال', 'ET564XL393t0S0ZA', 'sanaa.yasin@example.com', 'Al-Amin', 'zein.azhar@example.com', 'جدة', 'إي بلس 91', 'abdallah.al-rashid@example.com', '54p0jW7445Ao7Mve', '91', 'آمين', 'tariq.zahra@example.com', '6b52akdf0b0r4s39', 'thamer.zaki@example.com', '5Xl43A04jx7lSB91', 'خاص 95', 'm3V81K1457HD6yFV', 'أسد', 'ismail.manal@example.com', 'Al-Safwa station', '91qq0UV14DL42IE5', 'i08N207lFgw5i00i', 'eAY64453Enf7Dg14', 'المنفذ 3', '6x4rU4L47Z59V3PQ', 'سعاد', 'سالم', 'منى', 'mona.riham@example.com', 'p6P16258Ckx7L0Kl', 'الخليج']


## Entity to split words docs

In [ ]:
######################
"""
Creats words to entity map: 
Splits given entity into words and maps each word to its entity

IP:
1. List of entities

OP:
1. Dict {entity: [words, in, entity], entity: [words, in, entity], ... }
2. list of split words [words, in, all, entity]
"""
######################

def get_entity_words_map(entities_list):
    logging.info("Splittig entity and mapping each word to its Entity")

    word_entity_map = {}
    all_split_words = set()

    for text in entities_list:
        if not text:  # Check for empty strings
            logging.warning("Encountered empty string in entities_list.")
            continue

        try:
            # Split text into words and update maps
            split_texts = text.split()
            all_split_words.update(split_texts)
            word_entity_map[text] = split_texts

            logging.info(f"Processed entity: '{text}' -> Words: {split_texts}")

        except Exception as e:
            logging.error(f"Error processing entity '{text}': {e}")

    return word_entity_map, all_split_words

In [ ]:
##################################
# Entity context for sql query
##################################
en_word_entity_map, en_all_split_words = get_entity_words_map(all_noun_data)


# en_word_entity_map = {'f6p6D1QG13j172sO': ['f6p6D1QG13j172sO'], 'Samar': ['Samar'], 'Al-Qasr': ['Al-Qasr'], 'Khalid': ['Khalid'], 'F9y1Yoc283H33Ae2': ['F9y1Yoc283H33Ae2'], 'mahdi.rahma@example.com': ['mahdi.rahma@example.com'], 'zi3DR73Q05d3w03L': ['zi3DR73Q05d3w03L'], 'Mohammad Ali Al-Khalidi': ['Mohammad', 'Ali', 'Al-Khalidi'], 'eman.lubna@example.com': ['eman.lubna@example.com'], 'Hussein': ['Hussein'], 'nu49hU1Vn054fh34': ['nu49hU1Vn054fh34'], 'Al-Fayhaa station': ['Al-Fayhaa', 'station'], 'jameel.mona@example.com': ['jameel.mona@example.com'], 'waleed.zein@example.com': ['waleed.zein@example.com'], 'Asma': ['Asma'], 'Super 98': ['Super', '98'], 'Samra petrol station': ['Samra', 'petrol', 'station'], '6PdRb1z7q831A8I1': ['6PdRb1z7q831A8I1'], 'youssef.al-qahtani@example.com': ['youssef.al-qahtani@example.com'], '99301diz7u3B4MgC': ['99301diz7u3B4MgC'], '7lx18tIx262Et10J': ['7lx18tIx262Et10J'], 'X07H3sFm028e90jq': ['X07H3sFm028e90jq'], '7ds0J91N7TBj642n': ['7ds0J91N7TBj642n'], 'Omar': ['Omar'], 'Al-Jazira station': ['Al-Jazira', 'station'], 'Al-Murooj station': ['Al-Murooj', 'station'], 'Basim': ['Basim'], 'Aramco station Zahra': ['Aramco', 'station', 'Zahra'], 'Al-Nahda station': ['Al-Nahda', 'station'], 'Ol45J1w2zpp9y419': ['Ol45J1w2zpp9y419'], '9uZ9H11X3E2u7U8d': ['9uZ9H11X3E2u7U8d'], '47xe6c9QOSkz5358': ['47xe6c9QOSkz5358'], 'Al-Takhassusi Street': ['Al-Takhassusi', 'Street'], 'Eman': ['Eman'], 'Aramco station Baraka': ['Aramco', 'station', 'Baraka'], 'Aisha': ['Aisha'], 'nasser.samar@example.com': ['nasser.samar@example.com'], 'Al-Nahda': ['Al-Nahda'], 'Rabwa': ['Rabwa'], 'Ras Al Khaimah': ['Ras', 'Al', 'Khaimah'], 'zaid.alia@example.com': ['zaid.alia@example.com'], 'Al-Farsi': ['Al-Farsi'], '2Tfx60Q6073gse0H': ['2Tfx60Q6073gse0H'], 'Safwa': ['Safwa'], 'lubna.qasim@example.com': ['lubna.qasim@example.com'], 'Fujairah': ['Fujairah'], 'g9Q37Q0nwLC1478P': ['g9Q37Q0nwLC1478P'], 'ziad.bahir@example.com': ['ziad.bahir@example.com'], 'Qhm5xJ359LR20s86': ['Qhm5xJ359LR20s86'], 'R462ZHa59rv1l49T': ['R462ZHa59rv1l49T'], '4aJ27zCm01Z7t6m8': ['4aJ27zCm01Z7t6m8'], '3UEfDc33K0q0g346': ['3UEfDc33K0q0g346'], 'Ismail': ['Ismail'], 'yasin.abir@example.com': ['yasin.abir@example.com'], 'Jamilah': ['Jamilah'], '4Y6h8z131z75pWIr': ['4Y6h8z131z75pWIr'], 'W8210bE255lgs2KR': ['W8210bE255lgs2KR'], '5e7b58M08BGkGM03': ['5e7b58M08BGkGM03'], 'Al-Khaleej': ['Al-Khaleej'], 'Farid': ['Farid'], '1CaGp270C5900Asw': ['1CaGp270C5900Asw'], 'Diesel': ['Diesel'], 'Al-Qasr station': ['Al-Qasr', 'station'], 'omar.al-harbi@example.com': ['omar.al-harbi@example.com'], 'Amal': ['Amal'], 'm34H49q52C78jApH': ['m34H49q52C78jApH'], 'malak.rami@example.com': ['malak.rami@example.com'], 'Sami': ['Sami'], '84trW13bN1NQL423': ['84trW13bN1NQL423'], 'T374I41K9rme7oK2': ['T374I41K9rme7oK2'], 'manal.reem@example.com': ['manal.reem@example.com'], 'salem.al-zahrani@example.com': ['salem.al-zahrani@example.com'], 'Al-Sharq station': ['Al-Sharq', 'station'], 'Hasan': ['Hasan'], 'ahmed.al-farsi@example.com': ['ahmed.al-farsi@example.com'], 'Al-Madina station': ['Al-Madina', 'station'], '123 Al-Nahda Street, Riyadh, Saudi Arabia': ['123', 'Al-Nahda', 'Street,', 'Riyadh,', 'Saudi', 'Arabia'], 'Al-Dammam': ['Al-Dammam'], 'Al-Madina': ['Al-Madina'], 'fawzi.munir@example.com': ['fawzi.munir@example.com'], 'f9i53o33byu2H98E': ['f9i53o33byu2H98E'], 'Makkah': ['Makkah'], 'Thamer': ['Thamer'], 'Riyadh': ['Riyadh'], 'Al-Rashid': ['Al-Rashid'], 'Jamal': ['Jamal'], 'Reem': ['Reem'], 'rami.tariq@example.com': ['rami.tariq@example.com'], 'Ayoub': ['Ayoub'], '11QX835Dg9N6c0Jb': ['11QX835Dg9N6c0Jb'], '7dGhB4r12a280H9W': ['7dGhB4r12a280H9W'], 'Mohammed': ['Mohammed'], 'u4vF15aIX8V6u894': ['u4vF15aIX8V6u894'], 'Aramco station Hilal': ['Aramco', 'station', 'Hilal'], 'Al Ai': ['Al', 'Ai'], 'hassan.al-amin@example.com': ['hassan.al-amin@example.com'], 'Aramco station Nasr': ['Aramco', 'station', 'Nasr'], 'Qasim': ['Qasim'], 'Al-Dammam station': ['Al-Dammam', 'station'], 'Al-Bahrain station': ['Al-Bahrain', 'station'], 'qasim.selim@example.com': ['qasim.selim@example.com'], 'Manal': ['Manal'], 'Rami': ['Rami'], 'Aramco station Rasheed': ['Aramco', 'station', 'Rasheed'], 'Al-Rawi Trading Co.': ['Al-Rawi', 'Trading', 'Co.'], '0R660631MBM5keIx': ['0R660631MBM5keIx'], 'Al-Rabwa station': ['Al-Rabwa', 'station'], 'Farah': ['Farah'], 'Kareem': ['Kareem'], 'Sanaa': ['Sanaa'], 'reem.thamer@example.com': ['reem.thamer@example.com'], 'Mustafa': ['Mustafa'], 'selim.youssef@example.com': ['selim.youssef@example.com'], 'Al-Fursan station': ['Al-Fursan', 'station'], 'Bilal': ['Bilal'], 'Zein': ['Zein'], 'Al-Bahrain': ['Al-Bahrain'], 'wKPc8X636ypz7580': ['wKPc8X636ypz7580'], 'iman.malak@example.com': ['iman.malak@example.com'], 'zahra.amal@example.com': ['zahra.amal@example.com'], 'samar.yara@example.com': ['samar.yara@example.com'], 'Medina': ['Medina'], 'lina.noor@example.com': ['lina.noor@example.com'], 'Yara': ['Yara'], 'Shift-1': ['Shift-1'], 'port-1': ['port-1'], 'Ali': ['Ali'], '7593ni40Fnto3f3A': ['7593ni40Fnto3f3A'], 'Jameel': ['Jameel'], 'abir.ehsan@example.com': ['abir.ehsan@example.com'], 'Aramco station Amal': ['Aramco', 'station', 'Amal'], 'Q986LG08HuzE3p18': ['Q986LG08HuzE3p18'], 'r57h71b10h88FzlL': ['r57h71b10h88FzlL'], 'Buraidah': ['Buraidah'], 'Sarah': ['Sarah'], 'Al-Sharif': ['Al-Sharif'], 'ROV1S19373K9VT0W': ['ROV1S19373K9VT0W'], 'Riham': ['Riham'], 'Abu Dhabi': ['Abu', 'Dhabi'], 'Abdulrahman': ['Abdulrahman'], 'Al-Riyadh': ['Al-Riyadh'], 'youssef.afaf@example.com': ['youssef.afaf@example.com'], 'warda.ziad@example.com': ['warda.ziad@example.com'], '24YW6215n65iQgaS': ['24YW6215n65iQgaS'], 'Haifa': ['Haifa'], '6ny26U33jn0qi6Y4': ['6ny26U33jn0qi6Y4'], 'Al-Khaleej station': ['Al-Khaleej', 'station'], 'FSg201eN24vPI416': ['FSg201eN24vPI416'], 'yara.zuhair@example.com': ['yara.zuhair@example.com'], 'Nadia': ['Nadia'], 'YxI96Y606729stYT': ['YxI96Y606729stYT'], 'Y4eqf664rw5xE865': ['Y4eqf664rw5xE865'], 'port-2': ['port-2'], '4L6G27aPdM94Lu93': ['4L6G27aPdM94Lu93'], '8m7E7G9Z66Tw25IA': ['8m7E7G9Z66Tw25IA'], '6rg5okO6t45W3q91': ['6rg5okO6t45W3q91'], 'Ajman': ['Ajman'], '52s891CZS0X8J4Dr': ['52s891CZS0X8J4Dr'], 'Al-Sarh': ['Al-Sarh'], 'Afaf': ['Afaf'], '7tyu38kg8TQ1247Z': ['7tyu38kg8TQ1247Z'], 'Lina': ['Lina'], 'lamia.nasser@example.com': ['lamia.nasser@example.com'], '96OEh99uro82r62a': ['96OEh99uro82r62a'], 'qK05dF6iGX66B745': ['qK05dF6iGX66B745'], 'Sharjah': ['Sharjah'], 'W2M2O7MX6J0q57F4': ['W2M2O7MX6J0q57F4'], 'Dalal': ['Dalal'], '2Y1R4a11YMz132SE': ['2Y1R4a11YMz132SE'], 'Al-Sayed': ['Al-Sayed'], 'Bahir': ['Bahir'], 'Abir': ['Abir'], '5Jr190L24WzUUS96': ['5Jr190L24WzUUS96'], 'Najran': ['Najran'], 'Najm': ['Najm'], 'Jazira': ['Jazira'], 'Al-Olaya': ['Al-Olaya'], 'oY53q55RkW0i3K17': ['oY53q55RkW0i3K17'], 'sr9A5h7K137n8G6l': ['sr9A5h7K137n8G6l'], 'Aramco station Noor': ['Aramco', 'station', 'Noor'], '6Q3C0eJg47fn49A3': ['6Q3C0eJg47fn49A3'], 'Nabila': ['Nabila'], 'Riyadh, KSA': ['Riyadh,', 'KSA'], '98yG06zI836ks2JY': ['98yG06zI836ks2JY'], '1B8e46w1L5fXY8M0': ['1B8e46w1L5fXY8M0'], 'Aramco station Huda': ['Aramco', 'station', 'Huda'], 'Youssef': ['Youssef'], 'souad.zaid@example.com': ['souad.zaid@example.com'], 'ali.al-mansour@example.com': ['ali.al-mansour@example.com'], 'riham.waleed@example.com': ['riham.waleed@example.com'], 'Zahra': ['Zahra'], 'jS1HB61b669P6r8t': ['jS1HB61b669P6r8t'], 'farah.marwa@example.com': ['farah.marwa@example.com'], 'Noor': ['Noor'], 'zaki.ayoub@example.com': ['zaki.ayoub@example.com'], 'Hana': ['Hana'], 'Al-Qahtani': ['Al-Qahtani'], 'mustafa.saeed@example.com': ['mustafa.saeed@example.com'], 'Al-Zahrani': ['Al-Zahrani'], 'H4l1Nc7S8382n4zD': ['H4l1Nc7S8382n4zD'], 'qZ8074F3IM90H3cV': ['qZ8074F3IM90H3cV'], 'afaf.eman@example.com': ['afaf.eman@example.com'], 'c66Q97UM7y755xdd': ['c66Q97UM7y755xdd'], 'Warda': ['Warda'], 'mohammed.al-sayed@example.com': ['mohammed.al-sayed@example.com'], 'Sultanah': ['Sultanah'], 'Yasin': ['Yasin'], 'i3S7T4v3o0Ty45u7': ['i3S7T4v3o0Ty45u7'], 'Aramco station Safa': ['Aramco', 'station', 'Safa'], 'Tabuk': ['Tabuk'], '5t9bNVd4ro68178D': ['5t9bNVd4ro68178D'], 'Al-Aseel station': ['Al-Aseel', 'station'], 'Al Kharj': ['Al', 'Kharj'], 'Aziza': ['Aziza'], 'Hala': ['Hala'], 'Anwar': ['Anwar'], 'dalal.kawthar@example.com': ['dalal.kawthar@example.com'], '247Ry0CKk92N8ND5': ['247Ry0CKk92N8ND5'], 'dh29LfN358I5Ts98': ['dh29LfN358I5Ts98'], 'R20J9u8n496gUK8g': ['R20J9u8n496gUK8g'], 'Iman': ['Iman'], 'Aramco station Kawthar': ['Aramco', 'station', 'Kawthar'], 'hamid.lubna@example.com': ['hamid.lubna@example.com'], 'Ziad': ['Ziad'], 'Lamia': ['Lamia'], 'Tariq': ['Tariq'], 'Aramco station Taqwa': ['Aramco', 'station', 'Taqwa'], 'Amira': ['Amira'], 'ehsan.leen@example.com': ['ehsan.leen@example.com'], 'Aramco station Fajr': ['Aramco', 'station', 'Fajr'], 'Rasha': ['Rasha'], 'Saeed': ['Saeed'], 'Adnan': ['Adnan'], 'HS6C9Ky7p754SJ41': ['HS6C9Ky7p754SJ41'], '95': ['95'], 'Al-Sarh station': ['Al-Sarh', 'station'], 'Al-Harbi': ['Al-Harbi'], 'Dubai': ['Dubai'], 'Mariam': ['Mariam'], '4NJu90bVf7l749y8': ['4NJu90bVf7l749y8'], 'Abdulaziz': ['Abdulaziz'], '6J755E72u8Lqgi5M': ['6J755E72u8Lqgi5M'], 'rahma.souad@example.com': ['rahma.souad@example.com'], 'Q5R1cIZ6m75h98A9': ['Q5R1cIZ6m75h98A9'], 'Sharq': ['Sharq'], '4673J0MH9lYH87jR': ['4673J0MH9lYH87jR'], 'Mahdi': ['Mahdi'], 'Fawzi': ['Fawzi'], 'Umm Al Quwain': ['Umm', 'Al', 'Quwain'], 'Zuhair': ['Zuhair'], 'Al-Mansour': ['Al-Mansour'], 'Lubna': ['Lubna'], 'V64R8b89O9jj08rM': ['V64R8b89O9jj08rM'], 'Abdullah': ['Abdullah'], 'Kalba': ['Kalba'], 'Al-Riyadh station': ['Al-Riyadh', 'station'], 's612237J0TF0BXGf': ['s612237J0TF0BXGf'], 'Rahma': ['Rahma'], 'Hamid': ['Hamid'], 'Malak': ['Malak'], 'Aramco station Azhar': ['Aramco', 'station', 'Azhar'], 'Khor Fakkan': ['Khor', 'Fakkan'], 'Al-Quds station': ['Al-Quds', 'station'], 'Alia': ['Alia'], 'Ibrahim': ['Ibrahim'], 'Aramco station Mahd': ['Aramco', 'station', 'Mahd'], 'djM12a9G9790sb9f': ['djM12a9G9790sb9f'], 'Dammam': ['Dammam'], 'noor.sanaa@example.com': ['noor.sanaa@example.com'], 'uGex454XQ98e25b2': ['uGex454XQ98e25b2'], 'Fatima': ['Fatima'], 'Al-Murooj': ['Al-Murooj'], 'Dina': ['Dina'], 'Aramco station Bustan': ['Aramco', 'station', 'Bustan'], 'kareem.mustafa@example.com': ['kareem.mustafa@example.com'], 'Z5MG3o2GoA25572p': ['Z5MG3o2GoA25572p'], 'Layla': ['Layla'], 'Ehsan': ['Ehsan'], 'Waleed': ['Waleed'], 'Rania': ['Rania'], '4Dgd5770AfqdH901': ['4Dgd5770AfqdH901'], 'ahmed ali': ['ahmed', 'ali'], 'Al-Waha station': ['Al-Waha', 'station'], 'Zaid': ['Zaid'], 'Al-Hashemi': ['Al-Hashemi'], 'Selim': ['Selim'], 'zuhair.dalal@example.com': ['zuhair.dalal@example.com'], 'z78N9t49h8q2Rd7o': ['z78N9t49h8q2Rd7o'], 'Zainab': ['Zainab'], 'ibrahim.al-sharif@example.com': ['ibrahim.al-sharif@example.com'], 'Yasmin': ['Yasmin'], 'P0a2Am70C2c70re1': ['P0a2Am70C2c70re1'], 'Hassan': ['Hassan'], 'Ahmed': ['Ahmed'], 'Ghada': ['Ghada'], 'Adam': ['Adam'], 'Salim': ['Salim'], 'DH858xQ2bUA1M192': ['DH858xQ2bUA1M192'], 'Al-Fursan': ['Al-Fursan'], 'Azhar': ['Azhar'], 'Al-Jazeera Tower': ['Al-Jazeera', 'Tower'], 'hana.mahdi@example.com': ['hana.mahdi@example.com'], 'Al-Mansour station': ['Al-Mansour', 'station'], 'Nasser': ['Nasser'], 'Al-Waha': ['Al-Waha'], '91i2qv930Z0BpT7Q': ['91i2qv930Z0BpT7Q'], 's2NA2LsY78z0771A': ['s2NA2LsY78z0771A'], '8G96wBCm4i5152CB': ['8G96wBCm4i5152CB'], '54r8K2oN3PH3A1H3': ['54r8K2oN3PH3A1H3'], '9b2xV05K1E068Vsy': ['9b2xV05K1E068Vsy'], 'Al-Najm station': ['Al-Najm', 'station'], 'saeed.warda@example.com': ['saeed.warda@example.com'], 'Aramco station Manara': ['Aramco', 'station', 'Manara'], 'Salma': ['Salma'], 'khalid.al-hashemi@example.com': ['khalid.al-hashemi@example.com'], 'north': ['north'], 'ET564XL393t0S0ZA': ['ET564XL393t0S0ZA'], 'sanaa.yasin@example.com': ['sanaa.yasin@example.com'], 'Al-Amin': ['Al-Amin'], 'zein.azhar@example.com': ['zein.azhar@example.com'], 'Jeddah': ['Jeddah'], 'EPlus 91': ['EPlus', '91'], 'abdullah.al-rashid@example.com': ['abdullah.al-rashid@example.com'], '54p0jW7445Ao7Mve': ['54p0jW7445Ao7Mve'], '91': ['91'], 'Amina': ['Amina'], 'tariq.zahra@example.com': ['tariq.zahra@example.com'], '6b52akdf0b0r4s39': ['6b52akdf0b0r4s39'], 'thamer.zaki@example.com': ['thamer.zaki@example.com'], '5Xl43A04jx7lSB91': ['5Xl43A04jx7lSB91'], 'Special 95': ['Special', '95'], 'm3V81K1457HD6yFV': ['m3V81K1457HD6yFV'], 'Zaki': ['Zaki'], 'ismail.manal@example.com': ['ismail.manal@example.com'], 'Al-Safwa station': ['Al-Safwa', 'station'], '91qq0UV14DL42IE5': ['91qq0UV14DL42IE5'], 'i08N207lFgw5i00i': ['i08N207lFgw5i00i'], 'eAY64453Enf7Dg14': ['eAY64453Enf7Dg14'], 'port-3': ['port-3'], '6x4rU4L47Z59V3PQ': ['6x4rU4L47Z59V3PQ'], 'Souad': ['Souad'], 'Salem': ['Salem'], 'Mona': ['Mona'], 'mona.riham@example.com': ['mona.riham@example.com'], 'p6P16258Ckx7L0Kl': ['p6P16258Ckx7L0Kl']}
# en_all_split_words = {'Al-Safwa', 'Riyadh,', 'f6p6D1QG13j172sO', 'Samar', '98', 'Al-Qasr', 'Safa', 'Khalid', 'F9y1Yoc283H33Ae2', 'mahdi.rahma@example.com', 'Baraka', 'zi3DR73Q05d3w03L', 'eman.lubna@example.com', 'Hussein', 'nu49hU1Vn054fh34', 'jameel.mona@example.com', 'waleed.zein@example.com', 'Asma', '6PdRb1z7q831A8I1', 'youssef.al-qahtani@example.com', 'Al-Quds', '99301diz7u3B4MgC', '7lx18tIx262Et10J', 'X07H3sFm028e90jq', '7ds0J91N7TBj642n', 'Omar', 'Basim', 'Ol45J1w2zpp9y419', '9uZ9H11X3E2u7U8d', 'Saudi', '47xe6c9QOSkz5358', 'Eman', 'Aisha', 'nasser.samar@example.com', 'Al-Nahda', 'Rabwa', 'zaid.alia@example.com', 'Arabia', 'Al-Farsi', '2Tfx60Q6073gse0H', 'Safwa', 'lubna.qasim@example.com', 'Fujairah', 'g9Q37Q0nwLC1478P', 'ziad.bahir@example.com', 'Qhm5xJ359LR20s86', 'R462ZHa59rv1l49T', '4aJ27zCm01Z7t6m8', '3UEfDc33K0q0g346', 'Ismail', 'yasin.abir@example.com', 'Jamilah', '4Y6h8z131z75pWIr', 'W8210bE255lgs2KR', '5e7b58M08BGkGM03', 'Al-Khaleej', 'Farid', '1CaGp270C5900Asw', 'Diesel', 'omar.al-harbi@example.com', 'Amal', 'm34H49q52C78jApH', 'malak.rami@example.com', 'Sami', '84trW13bN1NQL423', 'T374I41K9rme7oK2', 'Ai', 'manal.reem@example.com', 'salem.al-zahrani@example.com', 'Abu', 'Hasan', 'ahmed.al-farsi@example.com', 'Al-Dammam', 'Al-Madina', 'fawzi.munir@example.com', 'f9i53o33byu2H98E', 'Makkah', 'Thamer', 'Bustan', 'Riyadh', 'Al-Rashid', 'Jamal', 'Reem', 'rami.tariq@example.com', 'Ayoub', '11QX835Dg9N6c0Jb', '7dGhB4r12a280H9W', 'Kharj', 'Mohammed', 'u4vF15aIX8V6u894', 'hassan.al-amin@example.com', 'Qasim', 'qasim.selim@example.com', 'Manal', 'Rami', 'Al-Rawi', '0R660631MBM5keIx', 'Farah', 'Kareem', 'Sanaa', 'reem.thamer@example.com', 'Mustafa', 'selim.youssef@example.com', 'Bilal', 'Zein', 'Al-Bahrain', 'wKPc8X636ypz7580', 'iman.malak@example.com', 'zahra.amal@example.com', 'samar.yara@example.com', 'Al-Rabwa', 'Medina', 'lina.noor@example.com', 'Yara', 'Kawthar', 'Shift-1', 'port-1', 'Super', 'Ali', '7593ni40Fnto3f3A', 'Jameel', 'abir.ehsan@example.com', 'Fakkan', 'Q986LG08HuzE3p18', 'r57h71b10h88FzlL', 'Buraidah', 'Sarah', 'Al-Sharif', 'ROV1S19373K9VT0W', 'Riham', 'Abdulrahman', 'Al-Riyadh', 'Al-Jazeera', 'youssef.afaf@example.com', 'warda.ziad@example.com', '24YW6215n65iQgaS', 'Haifa', '6ny26U33jn0qi6Y4', 'FSg201eN24vPI416', 'yara.zuhair@example.com', 'Nadia', 'YxI96Y606729stYT', 'Y4eqf664rw5xE865', 'port-2', '4L6G27aPdM94Lu93', '8m7E7G9Z66Tw25IA', '6rg5okO6t45W3q91', 'Ajman', '52s891CZS0X8J4Dr', 'Al-Sarh', 'Afaf', '7tyu38kg8TQ1247Z', 'Lina', 'lamia.nasser@example.com', '96OEh99uro82r62a', 'qK05dF6iGX66B745', 'Sharjah', 'Al-Aseel', 'W2M2O7MX6J0q57F4', 'Dalal', '2Y1R4a11YMz132SE', 'Al-Sayed', 'Ras', 'Bahir', 'Abir', '5Jr190L24WzUUS96', 'Najran', 'Najm', 'Jazira', 'Al-Olaya', 'Dhabi', 'oY53q55RkW0i3K17', 'sr9A5h7K137n8G6l', '6Q3C0eJg47fn49A3', 'Nabila', '98yG06zI836ks2JY', 'Al-Takhassusi', '1B8e46w1L5fXY8M0', 'KSA', 'Youssef', 'souad.zaid@example.com', 'Hilal', 'ali.al-mansour@example.com', 'riham.waleed@example.com', 'Zahra', 'Al-Najm', 'jS1HB61b669P6r8t', 'Co.', 'farah.marwa@example.com', 'Noor', 'zaki.ayoub@example.com', 'Hana', 'Al-Qahtani', 'mustafa.saeed@example.com', 'Al-Zahrani', 'H4l1Nc7S8382n4zD', 'qZ8074F3IM90H3cV', 'afaf.eman@example.com', 'c66Q97UM7y755xdd', 'Aramco', 'Warda', 'mohammed.al-sayed@example.com', 'Nasr', 'Sultanah', 'Mahd', 'Yasin', 'i3S7T4v3o0Ty45u7', 'Tabuk', '5t9bNVd4ro68178D', 'Aziza', 'Al-Sharq', 'Mohammad', 'Hala', 'Anwar', 'dalal.kawthar@example.com', '247Ry0CKk92N8ND5', 'Khaimah', 'dh29LfN358I5Ts98', 'R20J9u8n496gUK8g', 'Iman', 'hamid.lubna@example.com', 'Ziad', 'Lamia', 'Al', 'Tariq', 'Amira', 'ehsan.leen@example.com', 'Rasha', 'Saeed', 'ali', 'Adnan', 'HS6C9Ky7p754SJ41', '95', 'Al-Harbi', 'Dubai', 'Mariam', '4NJu90bVf7l749y8', 'Abdulaziz', '6J755E72u8Lqgi5M', 'rahma.souad@example.com', 'Q5R1cIZ6m75h98A9', 'Sharq', '4673J0MH9lYH87jR', 'Mahdi', 'Fawzi', 'Zuhair', 'Al-Mansour', 'Lubna', 'V64R8b89O9jj08rM', 'Abdullah', 'Kalba', 's612237J0TF0BXGf', 'Rahma', 'Hamid', 'Street', 'Malak', 'Alia', 'Al-Fayhaa', 'Ibrahim', 'Taqwa', 'Street,', 'djM12a9G9790sb9f', 'Dammam', 'noor.sanaa@example.com', 'uGex454XQ98e25b2', 'Fatima', 'Al-Murooj', 'station', 'Dina', 'Manara', 'kareem.mustafa@example.com', 'Quwain', 'Z5MG3o2GoA25572p', 'petrol', 'Layla', 'Ehsan', 'Al-Jazira', 'Waleed', 'Rania', '4Dgd5770AfqdH901', 'EPlus', 'Zaid', 'Al-Hashemi', 'Selim', 'zuhair.dalal@example.com', 'z78N9t49h8q2Rd7o', 'Zainab', 'ibrahim.al-sharif@example.com', 'Yasmin', 'P0a2Am70C2c70re1', 'Samra', 'Hassan', 'Ahmed', 'Ghada', 'Adam', 'Fajr', 'Salim', 'Tower', '123', 'DH858xQ2bUA1M192', 'Al-Fursan', 'Rasheed', 'ahmed', 'Azhar', 'hana.mahdi@example.com', 'Nasser', 'Al-Waha', '91i2qv930Z0BpT7Q', 's2NA2LsY78z0771A', '8G96wBCm4i5152CB', '54r8K2oN3PH3A1H3', '9b2xV05K1E068Vsy', 'saeed.warda@example.com', 'Trading', 'Salma', 'khalid.al-hashemi@example.com', 'north', 'Khor', 'ET564XL393t0S0ZA', 'sanaa.yasin@example.com', 'Al-Amin', 'zein.azhar@example.com', 'Jeddah', 'Al-Khalidi', 'abdullah.al-rashid@example.com', '54p0jW7445Ao7Mve', 'Umm', '91', 'Amina', 'tariq.zahra@example.com', '6b52akdf0b0r4s39', 'thamer.zaki@example.com', '5Xl43A04jx7lSB91', 'm3V81K1457HD6yFV', 'Zaki', 'ismail.manal@example.com', 'Special', '91qq0UV14DL42IE5', 'i08N207lFgw5i00i', 'Huda', 'eAY64453Enf7Dg14', 'port-3', '6x4rU4L47Z59V3PQ', 'Souad', 'Salem', 'Mona', 'mona.riham@example.com', 'p6P16258Ckx7L0Kl'}


In [ ]:
# en_word_entity_map

In [ ]:
# Extract name form email (NOT USED)
def extract_name_from_email(email):
    name = re.sub(r'@.*', '', email)
    name = re.sub(r'[^a-zA-Z0-9]', '', name)
    return name

In [ ]:
##################################
# Entity context for traslation
##################################
# translated_texts_dict, all_translated_words = get_entity_words_map(translated_texts_list)


translated_texts_dict = {'f6p6D1QG13j172sO': ['f6p6D1QG13j172sO'], 'إنتاج': ['إنتاج'], 'Al-Qasr': ['Al-Qasr'], 'خالد': ['خالد'], 'F9y1Yoc283H33Ae2': ['F9y1Yoc283H33Ae2'], 'mahdi.rahma@example.com': ['mahdi.rahma@example.com'], 'zi3DR73Q05d3w03L': ['zi3DR73Q05d3w03L'], 'Mohammad Ali Al-Khalidi': ['Mohammad', 'Ali', 'Al-Khalidi'], 'eman.lubna@example.com': ['eman.lubna@example.com'], 'حسين': ['حسين'], 'نو49hU1Vn054fh34': ['نو49hU1Vn054fh34'], 'محطة الفيحاء': ['محطة', 'الفيحاء'], 'jameel.mona@example.com': ['jameel.mona@example.com'], 'waleed.zein@example.com': ['waleed.zein@example.com'], 'الربو': ['الربو'], 'سوبر 98': ['سوبر', '98'], 'محطة بنزين سمرة': ['محطة', 'بنزين', 'سمرة'], '6PdRb1z7q831A8I1': ['6PdRb1z7q831A8I1'], 'youssef.al-qahtani@example.com': ['youssef.al-qahtani@example.com'], '99301diz7u3B4MgC': ['99301diz7u3B4MgC'], '7lx18tIx262Et10J': ['7lx18tIx262Et10J'], 'X07H3sFm028e90jq': ['X07H3sFm028e90jq'], '7ds0J91N7TBj642n': ['7ds0J91N7TBj642n'], 'عمر': ['عمر'], 'محطة الجزيرة': ['محطة', 'الجزيرة'], 'محطة المروج': ['محطة', 'المروج'], 'باسم': ['باسم'], 'محطة ارامكو الزهراء': ['محطة', 'ارامكو', 'الزهراء'], 'Al-Nahda station': ['Al-Nahda', 'station'], 'Ol45J1w2zpp9y419': ['Ol45J1w2zpp9y419'], '9uZ9H11X3E2u7U8d': ['9uZ9H11X3E2u7U8d'], '47xe6c9QOSkz5358': ['47xe6c9QOSkz5358'], 'شارع التخصصي': ['شارع', 'التخصصي'], 'يعطي': ['يعطي'], 'محطة ارامكو البركة': ['محطة', 'ارامكو', 'البركة'], 'عائشة': ['عائشة'], 'nasser.samar@example.com': ['nasser.samar@example.com'], 'Al-Nahda': ['Al-Nahda'], 'رابا': ['رابا'], 'Ras Al Khaimah': ['Ras', 'Al', 'Khaimah'], 'zaid.alia@example.com': ['zaid.alia@example.com'], 'الفارسي': ['الفارسي'], '2Tfx60Q6073gse0H': ['2Tfx60Q6073gse0H'], 'بالطبع': ['بالطبع'], 'lubna.qasim@example.com': ['lubna.qasim@example.com'], 'الفجيرة': ['الفجيرة'], 'g9Q37Q0nwLC1478P': ['g9Q37Q0nwLC1478P'], 'ziad.bahir@example.com': ['ziad.bahir@example.com'], 'Qhm5xJ359LR20s86': ['Qhm5xJ359LR20s86'], 'R462ZHa59rv1l49T': ['R462ZHa59rv1l49T'], '4aJ27zCm01Z7t6m8': ['4aJ27zCm01Z7t6m8'], '3UEfDc33K0q0g346': ['3UEfDc33K0q0g346'], 'اسماعيل': ['اسماعيل'], 'yasin.abir@example.com': ['yasin.abir@example.com'], 'جميلة': ['جميلة'], '4Y6h8z131z75pWIr': ['4Y6h8z131z75pWIr'], 'W8210bE255lgs2KR': ['W8210bE255lgs2KR'], '5e7b58M08BGkGM03': ['5e7b58M08BGkGM03'], 'Al-Khaleej': ['الخليج'], 'فريد': ['فريد'], '1CaGp270C5900Asw': ['1CaGp270C5900Asw'], 'ديزل': ['ديزل'], 'محطة القصر': ['محطة', 'القصر'], 'omar.al-harbi@example.com': ['omar.al-harbi@example.com'], 'صدقة': ['صدقة'], 'm34H49q52C78jApH': ['m34H49q52C78jApH'], 'malak.rami@example.com': ['malak.rami@example.com'], 'سامي': ['سامي'], '84trW13bN1NQL423': ['84trW13bN1NQL423'], 'T374I41K9rme7oK2': ['T374I41K9rme7oK2'], 'manal.reem@example.com': ['manal.reem@example.com'], 'Salem.al-zahrani@example.com': ['Salem.al-zahrani@example.com'], 'محطة الشرق': ['محطة', 'الشرق'], 'حسن': ['حسن'], 'ahmed.al-farsi@example.com': ['ahmed.al-farsi@example.com'], 'محطة المدينة المنورة': ['محطة', 'المدينة', 'المنورة'], '123 Al-Nahda Street, Riyadh, Saudi Arabia': ['123', 'Al-Nahda', 'Street,', 'Riyadh,', 'Saudi', 'Arabia'], 'Al-Dammam': ['Al-Dammam'], 'Al-Madina': ['Al-Madina'], 'fawzi.munir@example.com': ['fawzi.munir@example.com'], 'f9i53o33byu2H98E': ['f9i53o33byu2H98E'], 'مكه': ['مكه'], 'ثامر': ['ثامر'], 'الرياض': ['الرياض'], 'Al-Rashid': ['Al-Rashid'], 'جمال': ['جمال'], 'ريم': ['ريم'], 'rami.tariq@example.com': ['rami.tariq@example.com'], 'أيوب': ['أيوب'], '11QX835Dg9N6c0Jb': ['11QX835Dg9N6c0Jb'], '7dGhB4r12a280H9W': ['7dGhB4r12a280H9W'], 'محمد': ['محمد'], 'u4vF15aIX8V6u894': ['u4vF15aIX8V6u894'], 'محطة أرامكو الهلال': ['محطة', 'أرامكو', 'الهلال'], 'آل عاي': ['آل', 'عاي'], 'hassan.al-amin@example.com': ['hassan.al-amin@example.com'], 'محطة ارامكو النصر': ['محطة', 'ارامكو', 'النصر'], 'قاسم': ['قاسم'], 'محطة الدمام': ['محطة', 'الدمام'], 'محطة البحرين': ['محطة', 'البحرين'], 'qasim.selim@example.com': ['qasim.selim@example.com'], 'منال': ['منال'], 'رامي': ['رامي'], 'محطة ارامكو الرشيد': ['محطة', 'ارامكو', 'الرشيد'], 'شركة الراوي التجارية': ['شركة', 'الراوي', 'التجارية'], '0R660631MBM5keIx': ['0R660631MBM5keIx'], 'محطة الربوة': ['محطة', 'الربوة'], 'فرح': ['فرح'], 'كريم': ['كريم'], 'فن': ['فن'], 'reem.thamer@example.com': ['reem.thamer@example.com'], 'مصطفى': ['مصطفى'], 'selim.youssef@example.com': ['selim.youssef@example.com'], 'محطة الفرسان': ['محطة', 'الفرسان'], 'بلال': ['بلال'], 'أيّ': ['أيّ'], 'Al-Bahrain': ['Al-Bahrain'], 'wKPc8X636ypz7580': ['wKPc8X636ypz7580'], 'iman.malak@example.com': ['iman.malak@example.com'], 'zahra.amal@example.com': ['zahra.amal@example.com'], 'samar.yara@example.com': ['samar.yara@example.com'], 'المدينة المنورة': ['المدينة', 'المنورة'], 'lina.noor@example.com': ['lina.noor@example.com'], 'أطفال': ['أطفال'], 'التحول-1': ['التحول-1'], 'المنفذ 1': ['المنفذ', '1'], 'أو': ['أو'], '7593ni40Fnto3f3A': ['7593ni40Fnto3f3A'], 'جميل': ['جميل'], 'abir.ehsan@example.com': ['abir.ehsan@example.com'], 'محطة أرامكو أمل': ['محطة', 'أرامكو', 'أمل'], 'Q986LG08HuzE3p18': ['Q986LG08HuzE3p18'], 'r57h71b10h88FzlL': ['r57h71b10h88FzlL'], 'بريدة': ['بريدة'], 'ساره': ['ساره'], 'Al-Sharif': ['Al-Sharif'], 'ROV1S19373K9VT0W': ['ROV1S19373K9VT0W'], 'ريهام': ['ريهام'], 'Abu Dhabi': ['Abu', 'Dhabi'], 'عبدالرحمن': ['عبدالرحمن'], 'Al-Riyadh': ['Al-Riyadh'], 'youssef.afaf@example.com': ['youssef.afaf@example.com'], 'warda.ziad@example.com': ['warda.ziad@example.com'], '24YW6215n65iQgaS': ['24YW6215n65iQgaS'], 'حيفا': ['حيفا'], '6ny26U33jn0qi6Y4': ['6ny26U33jn0qi6Y4'], 'محطة الخليج': ['محطة', 'الخليج'], 'FSg201eN24vPI416': ['FSg201eN24vPI416'], 'yara.zuhair@example.com': ['yara.zuhair@example.com'], 'ناديه': ['ناديه'], 'YxI96Y606729stYT': ['YxI96Y606729stYT'], 'Y4eqf664rw5xE865': ['Y4eqf664rw5xE865'], 'المنفذ 2': ['المنفذ', '2'], '4L6G27aPdM94Lu93': ['4L6G27aPdM94Lu93'], '8m7E7G9Z66Tw25IA': ['8m7E7G9Z66Tw25IA'], '6rg5okO6t45W3q91': ['6rg5okO6t45W3q91'], 'عجمان': ['عجمان'], '52s891CZS0X8J4 د': ['52s891CZS0X8J4', 'د'], 'الصرح': ['الصرح'], 'اللغات': ['اللغات'], '7tyu38kg8TQ1247Z': ['7tyu38kg8TQ1247Z'], 'لينا': ['لينا'], 'lamia.nasser@example.com': ['lamia.nasser@example.com'], '96OEh99uro82r62a': ['96OEh99uro82r62a'], 'qK05dF6iGX66B745': ['qK05dF6iGX66B745'], 'الشارقة': ['الشارقة'], 'W2M2O7MX6J0q57F4': ['W2M2O7MX6J0q57F4'], 'بلدان': ['بلدان'], '2Y1R4a11YMz132SE': ['2Y1R4a11YMz132SE'], 'Al-Sayed': ['Al-Sayed'], 'باهر': ['باهر'], 'عبير': ['عبير'], '5Jr190L24WzUUS96': ['5Jr190L24WzUUS96'], 'نجران': ['نجران'], 'بخاصة': ['بخاصة'], 'جزيرة': ['جزيرة'], 'Al-Olaya': ['Al-Olaya'], 'oY53q55RkW0i3K17': ['oY53q55RkW0i3K17'], 'sr9A5h7K137n8G6l': ['sr9A5h7K137n8G6l'], 'محطة أرامكو نور': ['محطة', 'أرامكو', 'نور'], '6Q3C0eJg47fn49A3': ['6Q3C0eJg47fn49A3'], 'نبيلة': ['نبيلة'], 'الرياض، المملكة العربية السعودية': ['الرياض،', 'المملكة', 'العربية', 'السعودية'], '98yG06zI836ks2JY': ['98yG06zI836ks2JY'], '1B8e46w1L5fXY8M0': ['1B8e46w1L5fXY8M0'], 'محطة أرامكو الهدى': ['محطة', 'أرامكو', 'الهدى'], 'يوسف': ['يوسف'], 'souad.zaid@example.com': ['souad.zaid@example.com'], 'ali.al-mansour@example.com': ['ali.al-mansour@example.com'], 'riham.waleed@example.com': ['riham.waleed@example.com'], 'الزهراء': ['الزهراء'], 'jS1HB61b669P6r8t': ['jS1HB61b669P6r8t'], 'فرح.ماروا@example.com': ['فرح.ماروا@example.com'], 'نور': ['نور'], 'zaki.ayoub@example.com': ['zaki.ayoub@example.com'], 'عمل': ['عمل'], 'Al-Qahtani': ['Al-Qahtani'], 'mustafa.saeed@example.com': ['mustafa.saeed@example.com'], 'Al-Zahrani': ['Al-Zahrani'], 'H4l1Nc7S8382n4zD': ['H4l1Nc7S8382n4zD'], 'qZ8074F3IM90H3cV': ['qZ8074F3IM90H3cV'], 'afaf.eman@example.com': ['afaf.eman@example.com'], 'c66Q97UM7y755xdd': ['c66Q97UM7y755xdd'], 'وردة': ['وردة'], 'mohammed.al-sayed@example.com': ['mohammed.al-sayed@example.com'], 'سلطانة': ['سلطانة'], 'ياسين': ['ياسين'], 'i3S7T4v3o0Ty45u7': ['i3S7T4v3o0Ty45u7'], 'محطة ارامكو الصفا': ['محطة', 'ارامكو', 'الصفا'], 'تبوك': ['تبوك'], '5t9bNVd4ro68178D': ['5t9bNVd4ro68178D'], 'محطة الأصيل': ['محطة', 'الأصيل'], 'Al Kharj': ['Al', 'Kharj'], 'عزيزي': ['عزيزي'], 'لذا': ['لذا'], 'أنور': ['أنور'], 'dalal.kawthar@example.com': ['dalal.kawthar@example.com'], '247Ry0CKk92N8ND5': ['247Ry0CKk92N8ND5'], 'dh29LfN358I5Ts98': ['dh29LfN358I5Ts98'], 'R20J9u8n496gUK8g': ['R20J9u8n496gUK8g'], 'إيمان': ['إيمان'], 'Aramco station Kawthar': ['Aramco', 'station', 'Kawthar'], 'hamid.lubna@example.com': ['hamid.lubna@example.com'], 'زياد': ['زياد'], 'لمياء': ['لمياء'], 'طارق': ['طارق'], 'محطة أرامكو التقوى': ['محطة', 'أرامكو', 'التقوى'], 'أميرة': ['أميرة'], 'ehsan.leen@example.com': ['ehsan.leen@example.com'], 'محطة أرامكو الفجر': ['محطة', 'أرامكو', 'الفجر'], 'روسيا': ['روسيا'], 'سعيد': ['سعيد'], 'عدنان': ['عدنان'], 'HS6C9Ky7p754SJ41': ['HS6C9Ky7p754SJ41'], '95': ['95'], 'محطة الصرح': ['محطة', 'الصرح'], 'Al-Harbi': ['Al-Harbi'], 'دبي': ['دبي'], 'مريم': ['مريم'], '4NJu90bVf7l749y8': ['4NJu90bVf7l749y8'], 'عبد العزيز': ['عبد', 'العزيز'], '6J755E72u8Lqgi5M': ['6J755E72u8Lqgi5M'], 'rahma.souad@example.com': ['rahma.souad@example.com'], 'Q5R1cIZ6m75h98A9': ['Q5R1cIZ6m75h98A9'], 'شرق': ['شرق'], '4673J0MH9lYH87jR': ['4673J0MH9lYH87jR'], 'مهدي': ['مهدي'], 'فوزي': ['فوزي'], 'ام القيوين': ['ام', 'القيوين'], 'كوكب الزهرة': ['كوكب', 'الزهرة'], 'Al-Mansour': ['Al-Mansour'], 'لبنى': ['لبنى'], 'V64R8b89O9jj08rM': ['V64R8b89O9jj08rM'], 'عبد الله': ['عبد', 'الله'], 'لغة': ['لغة'], 'محطة الرياض': ['محطة', 'الرياض'], 's612237J0TF0BXGf': ['s612237J0TF0BXGf'], 'رحمة': ['رحمة'], 'حميد': ['حميد'], 'ملاك': ['ملاك'], 'محطة أرامكو الأزهر': ['محطة', 'أرامكو', 'الأزهر'], 'خورفكان': ['خورفكان'], 'محطة القدس': ['محطة', 'القدس'], 'علياء': ['علياء'], 'ابراهيم': ['ابراهيم'], 'محطة أرامكو المهد': ['محطة', 'أرامكو', 'المهد'], 'djM12a9G9790sb9f': ['djM12a9G9790sb9f'], 'الدمام': ['الدمام'], 'noor.sanaa@example.com': ['noor.sanaa@example.com'], 'uGex454XQ98e25b2': ['uGex454XQ98e25b2'], 'فاطمة': ['فاطمة'], 'المروس': ['المروس'], 'حقير': ['حقير'], 'محطة أرامكو البستان': ['محطة', 'أرامكو', 'البستان'], 'kareem.mustafa@example.com': ['kareem.mustafa@example.com'], 'Z5MG3o2GoA25572p': ['Z5MG3o2GoA25572p'], 'ليلى': ['ليلى'], 'كياسة': ['كياسة'], 'وليد': ['وليد'], 'رانيا': ['رانيا'], '4Dgd5770AfqdH901': ['4Dgd5770AfqdH901'], 'ahmed ali': ['ahmed', 'ali'], 'محطة الواحة': ['محطة', 'الواحة'], 'زيد': ['زيد'], 'Al-Hashemi': ['Al-Hashemi'], 'سليم': ['سليم'], 'zuhair.dalal@example.com': ['zuhair.dalal@example.com'], 'z78N9t49h8q2Rd7o': ['z78N9t49h8q2Rd7o'], 'زينب': ['زينب'], 'ibrahim.al-sharif@example.com': ['ibrahim.al-sharif@example.com'], 'ياسمين': ['ياسمين'], 'P0a2Am70C2c70re1': ['P0a2Am70C2c70re1'], 'حسان': ['حسان'], 'أحمد': ['أحمد'], 'غداً': ['غداً'], 'آدم': ['آدم'], 'DH858xQ2bUA1M192': ['DH858xQ2bUA1M192'], 'Al-Fursan': ['Al-Fursan'], 'أزهر': ['أزهر'], 'برج الجزيرة': ['برج', 'الجزيرة'], 'hana.mahdi@example.com': ['hana.mahdi@example.com'], 'محطة المنصور': ['محطة', 'المنصور'], 'ناصر': ['ناصر'], 'Al-Waha': ['Al-Waha'], '91i2qv930Z0BpT7Q': ['91i2qv930Z0BpT7Q'], 's2NA2LsY78z0771A': ['s2NA2LsY78z0771A'], '8G96wBCm4i5152CB': ['8G96wBCm4i5152CB'], '54r8K2oN3PH3A1H3': ['54r8K2oN3PH3A1H3'], '9b2xV05K1E068Vsy': ['9b2xV05K1E068Vsy'], 'محطة النجم': ['محطة', 'النجم'], 'saeed.warda@example.com': ['saeed.warda@example.com'], 'محطة أرامكو المنارة': ['محطة', 'أرامكو', 'المنارة'], 'سلمى': ['سلمى'], 'khalid.al-hashemi@example.com': ['khalid.al-hashemi@example.com'], 'شمال': ['شمال'], 'ET564XL393t0S0ZA': ['ET564XL393t0S0ZA'], 'sanaa.yasin@example.com': ['sanaa.yasin@example.com'], 'Al-Amin': ['Al-Amin'], 'zein.azhar@example.com': ['zein.azhar@example.com'], 'جدة': ['جدة'], 'إي بلس 91': ['إي', 'بلس', '91'], 'abdallah.al-rashid@example.com': ['abdallah.al-rashid@example.com'], '54p0jW7445Ao7Mve': ['54p0jW7445Ao7Mve'], '91': ['91'], 'آمين': ['آمين'], 'tariq.zahra@example.com': ['tariq.zahra@example.com'], '6b52akdf0b0r4s39': ['6b52akdf0b0r4s39'], 'thamer.zaki@example.com': ['thamer.zaki@example.com'], '5Xl43A04jx7lSB91': ['5Xl43A04jx7lSB91'], 'خاص 95': ['خاص', '95'], 'm3V81K1457HD6yFV': ['m3V81K1457HD6yFV'], 'أسد': ['أسد'], 'ismail.manal@example.com': ['ismail.manal@example.com'], 'Al-Safwa station': ['Al-Safwa', 'station'], '91qq0UV14DL42IE5': ['91qq0UV14DL42IE5'], 'i08N207lFgw5i00i': ['i08N207lFgw5i00i'], 'eAY64453Enf7Dg14': ['eAY64453Enf7Dg14'], 'المنفذ 3': ['المنفذ', '3'], '6x4rU4L47Z59V3PQ': ['6x4rU4L47Z59V3PQ'], 'سعاد': ['سعاد'], 'سالم': ['سالم'], 'منى': ['منى'], 'mona.riham@example.com': ['mona.riham@example.com'], 'p6P16258Ckx7L0Kl': ['p6P16258Ckx7L0Kl']}

all_translated_words = {'Al-Safwa', 'Riyadh,', 'f6p6D1QG13j172sO', '98', 'Al-Qasr', 'التقوى', 'عدنان', 'F9y1Yoc283H33Ae2', 'mahdi.rahma@example.com', 'zi3DR73Q05d3w03L', 'eman.lubna@example.com', 'الشرق', 'jameel.mona@example.com', 'waleed.zein@example.com', 'الله', 'لبنى', 'سلطانة', '6PdRb1z7q831A8I1', 'youssef.al-qahtani@example.com', '99301diz7u3B4MgC', '7lx18tIx262Et10J', 'أميرة', 'X07H3sFm028e90jq', '7ds0J91N7TBj642n', 'بلدان', 'ياسين', 'رابا', 'Ol45J1w2zpp9y419', '9uZ9H11X3E2u7U8d', 'Saudi', '47xe6c9QOSkz5358', 'ام', 'اسماعيل', 'nasser.samar@example.com', 'السعودية', 'Al-Nahda', 'خورفكان', 'zaid.alia@example.com', 'Arabia', 'آدم', '2Tfx60Q6073gse0H', 'lubna.qasim@example.com', 'المدينة', 'لينا', 'شارع', 'مصطفى', 'g9Q37Q0nwLC1478P', 'لغة', '2', 'بلال', 'ziad.bahir@example.com', 'Qhm5xJ359LR20s86', 'الجزيرة', 'باسم', 'R462ZHa59rv1l49T', 'الزهرة', '4aJ27zCm01Z7t6m8', '3UEfDc33K0q0g346', 'yasin.abir@example.com', '4Y6h8z131z75pWIr', 'W8210bE255lgs2KR', '5e7b58M08BGkGM03', 'خاص', 'الخليج', 'الصفا', 'إنتاج', '1CaGp270C5900Asw', 'omar.al-harbi@example.com', 'ريم', 'abdallah.al-rashid@example.com', 'أمل', 'm34H49q52C78jApH', 'malak.rami@example.com', '84trW13bN1NQL423', 'T374I41K9rme7oK2', 'manal.reem@example.com', 'Abu', 'ahmed.al-farsi@example.com', 'سلمى', 'Al-Dammam', 'Al-Madina', 'fawzi.munir@example.com', 'د', 'f9i53o33byu2H98E', 'ساره', 'مكه', 'Al-Rashid', 'جمال', 'البركة', 'حقير', 'rami.tariq@example.com', 'فوزي', '11QX835Dg9N6c0Jb', '7dGhB4r12a280H9W', 'Kharj', 'u4vF15aIX8V6u894', 'hassan.al-amin@example.com', 'محطة', 'القصر', 'qasim.selim@example.com', 'كوكب', 'ياسمين', 'محمد', 'فاطمة', 'عبد', 'العزيز', 'التخصصي', 'الرياض', 'سمرة', 'بلس', '0R660631MBM5keIx', 'reem.thamer@example.com', 'يوسف', 'selim.youssef@example.com', 'Al-Bahrain', 'wKPc8X636ypz7580', 'iman.malak@example.com', 'الرشيد', 'آمين', 'zahra.amal@example.com', 'رحمة', 'samar.yara@example.com', 'عائشة', 'lina.noor@example.com', 'كياسة', 'نجران', 'إيمان', 'منال', 'Kawthar', 'التجارية', 'Ali', '7593ni40Fnto3f3A', 'abir.ehsan@example.com', 'نور', 'Q986LG08HuzE3p18', 'r57h71b10h88FzlL', 'Al-Sharif', 'المنصور', 'ROV1S19373K9VT0W', 'أرامكو', 'Al-Riyadh', 'المهد', 'عبير', 'youssef.afaf@example.com', 'warda.ziad@example.com', '24YW6215n65iQgaS', 'مريم', 'جدة', 'الأزهر', 'أو', '6ny26U33jn0qi6Y4', 'رامي', 'FSg201eN24vPI416', 'yara.zuhair@example.com', 'YxI96Y606729stYT', 'Y4eqf664rw5xE865', '4L6G27aPdM94Lu93', '8m7E7G9Z66Tw25IA', '6rg5okO6t45W3q91', 'سعاد', 'غداً', 'برج', 'ابراهيم', 'حيفا', 'فن', '7tyu38kg8TQ1247Z', 'فرح.ماروا@example.com', 'فرح', 'lamia.nasser@example.com', '96OEh99uro82r62a', 'qK05dF6iGX66B745', 'لذا', 'W2M2O7MX6J0q57F4', 'لمياء', 'الواحة', '2Y1R4a11YMz132SE', 'Al-Sayed', 'Ras', 'حميد', '5Jr190L24WzUUS96', 'منى', 'الهلال', 'Al-Olaya', 'Dhabi', 'oY53q55RkW0i3K17', 'sr9A5h7K137n8G6l', '6Q3C0eJg47fn49A3', 'الزهراء', 'يعطي', '98yG06zI836ks2JY', 'عمل', '1B8e46w1L5fXY8M0', 'بنزين', 'souad.zaid@example.com', 'ali.al-mansour@example.com', 'riham.waleed@example.com', 'نو49hU1Vn054fh34', 'jS1HB61b669P6r8t', 'zaki.ayoub@example.com', 'سليم', 'أزهر', 'Al-Qahtani', 'mustafa.saeed@example.com', 'Al-Zahrani', 'H4l1Nc7S8382n4zD', 'qZ8074F3IM90H3cV', 'afaf.eman@example.com', '52s891CZS0X8J4', 'c66Q97UM7y755xdd', 'النصر', 'Aramco', 'رانيا', 'mohammed.al-sayed@example.com', 'القيوين', 'حسين', 'i3S7T4v3o0Ty45u7', 'الفارسي', '5t9bNVd4ro68178D', 'القدس', 'Mohammad', 'Salem.al-zahrani@example.com', 'روسيا', 'dalal.kawthar@example.com', '247Ry0CKk92N8ND5', 'Khaimah', 'جميلة', 'الرياض،', 'أيوب', 'جميل', 'dh29LfN358I5Ts98', 'R20J9u8n496gUK8g', 'المملكة', 'hamid.lubna@example.com', 'فريد', 'الأصيل', 'Al', 'صدقة', 'كريم', 'آل', 'ehsan.leen@example.com', 'سعيد', 'النجم', 'علياء', 'ثامر', 'بالطبع', 'ali', '95', 'HS6C9Ky7p754SJ41', 'إي', 'سامي', 'Al-Harbi', 'ناديه', 'المروج', '4NJu90bVf7l749y8', '6J755E72u8Lqgi5M', 'rahma.souad@example.com', 'Q5R1cIZ6m75h98A9', '4673J0MH9lYH87jR', 'طارق', 'Al-Mansour', 'زيد', 'V64R8b89O9jj08rM', 'مهدي', 'حسن', 'الصرح', 'الراوي', 's612237J0TF0BXGf', 'بريدة', '3', 'بخاصة', 'ناصر', 'أيّ', 'الفيحاء', 'ملاك', 'ريهام', 'Street,', 'djM12a9G9790sb9f', 'noor.sanaa@example.com', 'قاسم', 'وردة', 'uGex454XQ98e25b2', 'خالد', 'زينب', 'شمال', 'التحول-1', 'المنفذ', 'station', 'kareem.mustafa@example.com', 'Z5MG3o2GoA25572p', 'باهر', 'حسان', 'الفجيرة', 'المنورة', '4Dgd5770AfqdH901', 'الربوة', 'الربو', 'Al-Hashemi', 'zuhair.dalal@example.com', 'z78N9t49h8q2Rd7o', 'ibrahim.al-sharif@example.com', 'عاي', 'المروس', 'P0a2Am70C2c70re1', 'المنارة', '1', 'ديزل', 'عجمان', 'ليلى', 'جزيرة', 'الشارقة', 'زياد', 'وليد', 'أسد', 'تبوك', '123', 'DH858xQ2bUA1M192', 'Al-Fursan', 'ahmed', 'hana.mahdi@example.com', 'عزيزي', 'دبي', 'Al-Waha', '91i2qv930Z0BpT7Q', 's2NA2LsY78z0771A', '8G96wBCm4i5152CB', '54r8K2oN3PH3A1H3', '9b2xV05K1E068Vsy', 'سالم', 'saeed.warda@example.com', 'khalid.al-hashemi@example.com', 'الدمام', 'البحرين', 'عمر', 'ET564XL393t0S0ZA', 'sanaa.yasin@example.com', 'شركة', 'Al-Amin', 'عبدالرحمن', 'zein.azhar@example.com', 'أحمد', 'شرق', 'الفرسان', 'Al-Khalidi', 'نبيلة', 'سوبر', '54p0jW7445Ao7Mve', 'أنور', '91', 'ارامكو', 'العربية', 'tariq.zahra@example.com', '6b52akdf0b0r4s39', 'thamer.zaki@example.com', 'الخليج', 'الفجر', '5Xl43A04jx7lSB91', 'm3V81K1457HD6yFV', 'ismail.manal@example.com', 'اللغات', '91qq0UV14DL42IE5', 'البستان', 'i08N207lFgw5i00i', 'eAY64453Enf7Dg14', 'الهدى', '6x4rU4L47Z59V3PQ', 'mona.riham@example.com', 'أطفال', 'p6P16258Ckx7L0Kl'}

# Vector Store 

## Embedding model

In [ ]:
##############################################
# Vector stores - Table Schema and Entities 
##############################################

#################################
"""
Create custom embeding finction using multi-lingual model
"""
#################################
from chromadb import Documents, EmbeddingFunction, Embeddings
from vertexai.language_models import TextEmbeddingModel, TextEmbeddingInput
from langchain.vectorstores import Chroma

class MyEmbeddingFunction(EmbeddingFunction):
    def __init__(self, model_id, task):
        try:
            self.model = TextEmbeddingModel.from_pretrained(model_id)
            self.task = task
            logging.info(f"Initialized embedding model: {model_id} for task: {task}")
        except Exception as e:
            logging.error(f"Error initializing embedding model '{model_id}': {e}")
            raise  # Reraise the exception to avoid using an uninitialized model

    def embed_documents(self, input: Documents) -> Embeddings:
        max_batch_size = 250  # Batch to max limit -> 250
        embeddings = []
        
        try:
            for i in range(0, len(input), max_batch_size):
                batch = input[i:i + max_batch_size]
                inputs = [TextEmbeddingInput(text, self.task) for text in batch]
                
                # Get embeddings for the batch
                embedding_result = self.model.get_embeddings(inputs)
                batch_embeddings = [embedding.values for embedding in embedding_result]
                embeddings.extend(batch_embeddings)

                logging.info(f"Processed batch {i // max_batch_size + 1}: {len(batch)} documents embedded.")
                
            return embeddings
        
        except Exception as e:
            logging.error(f"Error embedding documents: {e}")
            return []  # Return an empty list in case of error

    def embed_query(self, query: str) -> Embeddings:
        try:
            inputs = [TextEmbeddingInput(query, self.task)]
            embedding_result = self.model.get_embeddings(inputs)
            embeddings = [embedding.values for embedding in embedding_result]
            logging.info(f"Processed query: '{query}' -> Embedding obtained.")
            return embeddings[0] if embeddings else []  # Return first embedding or empty
            
        except Exception as e:
            logging.error(f"Error embedding query '{query}': {e}")
            return []  # Return an empty list in case of error

##############################
# Model, task type and creating embedding function
##############################
EMBEDDING_MODEL = "text-multilingual-embedding-002"
EMBEDDING_TASK = "RETRIEVAL_DOCUMENT"

try:
    embedding_function = MyEmbeddingFunction(EMBEDDING_MODEL, EMBEDDING_TASK)
except Exception as e:
    logging.critical(f"Failed to create embedding function: {e}")



## Translated entity vector store for Translator

In [ ]:
# ##############################
# """
# Create vector store retriever for retrieving relevant {en:ar} entity map for user query  
# """
# #############################
# Convert entity translate map to string representation
entity_translate_map_str = [str(doc) for doc in entity_translate_map]

# Attempt to delete the existing collection and handle exceptions
try: 
    entity_translate_vector_store.delete_collection()
    logging.info("Deleted existing collection in vector store.")
except Exception as e:
    logging.warning(f"No need to clean the vector store or error occurred: {e}")

# Create a new vector store from the texts
try:
    entity_translate_vector_store = Chroma.from_texts(
        entity_translate_map_str, 
        embedding=embedding_function,
        collection_name="entity_translate_map_collection",
        persist_directory="entity_translate_map_collection",
    )
    logging.info("Successfully created a new vector store from texts.")
except Exception as e:
    logging.error(f"Error creating vector store: {e}")
    raise  # Reraise the exception for further handling if necessary

# Attempt to persist the collection to disk
try:
    entity_translate_vector_store.persist()
    logging.info("Persisted the collection to disk successfully.")
except Exception as e:
    logging.error(f"Error persisting the collection: {e}")



In [ ]:

def get_entity_translate_retriever(count=8):
    try:
        entity_translate_retriever = entity_translate_vector_store.as_retriever(search_kwargs={'k': count})
        logging.info(f"Created entity translate retriever with retrive count: {count}")
        return entity_translate_retriever
    except Exception as e:
        logging.error(f"Error creating entity translate retriever: {e}")
        return None  # Return None or handle as appropriate

# Specify the directory to load the persisted vector store
translation_persist_directory = 'entity_translate_map_collection'

# Attempt to load the vector store from the persisted data
try:
    entity_translate_vector_store = Chroma(
        collection_name="entity_translate_map_collection", 
        persist_directory=translation_persist_directory, 
        embedding_function=embedding_function
    )
    logging.info("Successfully loaded the vector store from persisted data.")
except Exception as e:
    logging.error(f"Error loading vector store: {e}")
    raise  # Reraise the exception for further handling if necessary

# Use the retriever from the loaded store
entity_translate_retriever = get_entity_translate_retriever()

if entity_translate_retriever is None:
    logging.critical("Failed to create entity translate retriever. Please check the logs for details.")

## Table vector for SQL query generator

In [ ]:
##############################################
# Table Schema vector stores
##############################################
import json

# Attempt to delete the existing collection and handle exceptions
try:
    schema_vector_store.delete_collection()
    logging.info("Deleted existing schema collection in vector store.")
except Exception as e:
    logging.warning(f"No need to clean the vector store or error occurred: {e}")

# Convert child documents to string representation
try:
    child_docs_as_strings = [json.dumps(doc) for doc in child_docs]
    logging.info("Converted child documents to string format.")
except Exception as e:
    logging.error(f"Error converting child documents to strings: {e}")
    raise  # Reraise the exception for further handling if necessary

# Attempt to create a new vector store from the texts
try:
    schema_vector_store = Chroma.from_texts(
        child_docs_as_strings, 
        embedding=embedding_function,
        collection_name="schema_collection",
        persist_directory="schema_collection"
    )
    logging.info("Successfully created schema vector store from texts.")
except Exception as e:
    logging.error(f"Error creating schema vector store: {e}")
    raise  # Reraise the exception for further handling if necessary

# Attempt to persist the collection to disk
try:
    schema_vector_store.persist()
    logging.info("Persisted the schema collection to disk successfully.")
except Exception as e:
    logging.error(f"Error persisting the schema collection: {e}")

In [ ]:
def get_schema_retriever(count=10):
    try:
        schema_retriever = schema_vector_store.as_retriever(search_kwargs={'k': count})
        logging.info(f"Created schema retriever with retrive count: {count}")
        return schema_retriever
    except Exception as e:
        logging.error(f"Error creating schema retriever: {e}")
        return None  # Return None or handle as appropriate

# Specify the directory to load the persisted vector store
schema_persist_directory = 'schema_collection'

# Attempt to load the vector store from the persisted data
try:
    schema_vector_store = Chroma(
        collection_name="schema_collection", 
        persist_directory=schema_persist_directory, 
        embedding_function=embedding_function
    )
    logging.info("Successfully loaded the schema vector store from persisted data.")
except Exception as e:
    logging.error(f"Error loading schema vector store: {e}")
    raise  # Reraise the exception for further handling if necessary

# Use the retriever from the loaded store
schema_retriever = get_schema_retriever()

if schema_retriever is None:
    logging.critical("Failed to create schema retriever. Please check the logs for details.")


In [ ]:
# text = "what are the sales for May 2024"
# res = schema_retriever.get_relevant_documents(text)

In [ ]:
# res
# r = [doc.page_content for doc in res]
# r1 = json.dumps(r[0])
# r1

## Entity vector store for SQL query generator

In [ ]:

##############################################
# Entities vector stores
##############################################

# Attempt to delete the existing noun vector store collection and handle exceptions
try:
    noun_vector_store.delete_collection()
    logging.info("Deleted existing noun collection in vector store.")
except Exception as e:
    logging.warning(f"No need to clean the vector store or an error occurred: {e}")

# Attempt to create a new noun vector store from the texts
try:
    noun_vector_store = Chroma.from_texts(
        all_noun_data,
        embedding=embedding_function,
        collection_name="noun_collection",
        persist_directory="noun_collection"
    )
    logging.info("Successfully created noun vector store from texts.")
except Exception as e:
    logging.error(f"Error creating noun vector store: {e}")
    raise  # Reraise the exception for further handling if necessary

# Attempt to persist the collection to disk
try:
    noun_vector_store.persist()
    logging.info("Persisted the noun collection to disk successfully.")
except Exception as e:
    logging.error(f"Error persisting the noun collection: {e}")

In [ ]:
def get_noun_retriever(count=10):
    try:
        noun_retriever = noun_vector_store.as_retriever(search_kwargs={'k': count})
        logging.info(f"Noun retriever created with retrive cpount {count}.")
        return noun_retriever
    except Exception as e:
        logging.error(f"Error creating noun retriever: {e}")
        raise  # Reraise the exception for further handling if necessary

noun_persist_directory = 'noun_collection'

# Attempt to load the vector store from the persisted data
try:
    noun_vector_store = Chroma(
        collection_name="noun_collection",
        persist_directory=noun_persist_directory,
        embedding_function=embedding_function
    )
    logging.info("Successfully loaded noun vector store from persisted data.")
except Exception as e:
    logging.error(f"Error loading noun vector store: {e}")
    raise  # Reraise the exception for further handling if necessary

# Use the retriever from the loaded store
try:
    noun_retriever = get_noun_retriever()
    logging.info("Successfully retrieved noun retriever.")
except Exception as e:
    logging.error(f"Error retrieving noun retriever: {e}")


# Retriever

In [ ]:

def get_entity_schema(entities):
    noun_rows = []
    
    try:
        conn = connect_db()
        cursor = conn.cursor()
        logging.info("Database connection established successfully.")
        
        for entity in entities:
            for noun_map in noun_maps:
                if entity in noun_map:
                    schema = noun_map[entity]['schema']
                    table_name = noun_map[entity]['table']
                    column_name = noun_map[entity]['column']
                    
                    sql_query = f"""SELECT * FROM "{schema}"."{table_name}" WHERE "{column_name}" = %s"""
                    cursor.execute(sql_query, (entity,))  # Use parameterized query to prevent SQL injection

                    columns = [desc[0] for desc in cursor.description]  # Get column names from cursor
                    for row in cursor.fetchall():
                        temp_dict = {}
                        temp_dict["query"] = sql_query
                        temp_dict["query_result"] = dict(zip(columns, row))  # Map column names to row values
                        noun_rows.append(temp_dict)
        
        logging.info(f"Processed {len(entities)} entities and retrieved their schemas successfully.")
    
    except Exception as e:
        logging.error(f"Error while retrieving entity schema: {e}")
        raise  # Reraise the exception for further handling if necessary
    
    finally:
        # Ensure cursor and connection are closed properly
        if cursor:
            cursor.close()
            logging.info("Cursor closed.")
        if conn:
            conn.close()
            logging.info("Database connection closed.")
    
    return noun_rows

## Entity retriever

In [ ]:
##############################################
# Retriver for Entities 
##############################################

def get_noun_pg_results(question):
    noun_rows = []
    
    try:
        logging.info(f"Retrieving relevant documents for question: {question}")
        results = noun_retriever.get_relevant_documents(question)

        if not results:
            logging.warning("No relevant documents found.")
            return noun_rows  # Return an empty list if no results

        entities = {result.page_content for result in results}
        logging.info(f"Identified entities: {entities}")

        noun_rows = get_entity_schema(entities) 
        logging.info(f"Retrieved schema for {len(entities)} entities.")
    
    except Exception as e:
        logging.error(f"Error while retrieving noun results: {e}")
        raise  # Reraise the exception for further handling if necessary
    
    return noun_rows

In [ ]:
# # question = "amal station is there?"
# question = "What are the employee sales on May 18, 2024?"
# results = get_noun_bq_results(question)  


In [ ]:
# retrieved_noun_docs_str = [str(doc) for doc in results]

In [ ]:

# results1 = top_noun_docs = re_rank_doc(question, retrieved_noun_docs_str, CROSS_ENCODER_MODEL, 5)


In [ ]:
# results

## Entity retriever for tranlsated entity

In [ ]:

def get_schema(entities_list):
    noun_rows = []
    
    try:
        logging.info("Processing entities to retrieve schema.")
        
        # Use a set to avoid duplicates
        entities = set(entities_list)  
        logging.info(f"Identified unique entities: {entities}")

        # Get the entity schema
        noun_rows = get_entity_schema(entities)

        logging.info(f"Retrieved schema for {len(entities)} entities.")
    
    except Exception as e:
        logging.error(f"Error while retrieving schema: {e}")
        raise  # Reraise the exception for further handling if necessary
    
    return noun_rows

In [ ]:
# question = ["Aramco station Amal"]
# r1 = get_entity_schema(question)
# r = get_schema(question)

In [ ]:
# r

## Table schema retriever

In [ ]:
##############################################
# Retriver for table schema - Auto merging retrieval
##############################################

# Create a lookup for column details to table name
column_details_to_table_name_lookup = {json.dumps(doc['column_details']): doc['table_name'] for doc in child_docs}

def get_relevant_parent_docs(question):
    retrieved_tables = []
    
    try:
        logging.info("Retrieving relevant parent documents based on the provided question.")
        
        # Retrieve relevant documents from schema_retriever
        results = schema_retriever.get_relevant_documents(question)
        logging.info(f"Retrieved {len(results)} documents from the schema retriever.")
        
        # Find table names from retrieved results
        retrieved_table_ids = set()
        for result in results:
            # Check if any of the JSON column details from child_docs are in the result's page content
            for column_details_json in column_details_to_table_name_lookup:
                if column_details_json in result.page_content:
                    retrieved_table_ids.add(column_details_to_table_name_lookup[column_details_json])
        
        logging.info(f"Identified table names: {retrieved_table_ids}")
        
        # Retrieve relevant parent documents
        retrieved_tables = [doc for doc in parent_docs if doc['table_name'] in retrieved_table_ids]
        logging.info(f"Retrieved {len(retrieved_tables)} relevant parent documents.")

    except Exception as e:
        logging.error(f"Error while retrieving relevant parent documents: {e}")
        raise  # Reraise the exception for further handling if necessary

    return retrieved_tables


In [ ]:
# # q = "what were the employee sales for the date 18th of May 2024?"
# q= "How much fuel was requested on 1st week of may 2024"
# # q= "Tell me abot Azzi station"

# start_time = time.time()
# a = get_relevant_parent_docs1(q)
# end_time = time.time()
# print(f"Time taken by get_relevant_parent_docs1: {end_time - start_time} seconds")

# start_time = time.time()
# a = get_relevant_parent_docs(q)
# end_time = time.time()
# print(f"Time taken by get_relevant_parent_docs: {end_time - start_time} seconds")

In [ ]:
# a

## Primary and foreign key map

# Summarizer (Not Used)

In [ ]:
# from transformers import pipeline

# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
# summary = summarizer(res_str, max_length=100, min_length=30, do_sample=False,length_penalty=2.0, num_beams=4)

"""
length_penalty: A value >1.0 encourages the model to generate longer sequences, which can result in more detailed summaries.
num_beams: Beam search parameter that can improve the quality of the summary. More beams can lead to a more thorough exploration of possible summaries.
"""

In [ ]:
# summary

# Re-Ranker


In [ ]:
###################################
# Cross Encoder Model for re-ranking
###################################
from sentence_transformers import CrossEncoder

CROSS_ENCODER_MODEL = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
###################################
# ReRanker for retrieved docs
###################################

def re_rank_doc(query, docs, cross_encoder, count=3):
    if not docs:
        logging.warning("No documents provided for re-ranking.")
        return []

    try:
        logging.info("Starting the re-ranking process.")

        # Create pairs of the query and each document
        pairs = [[query, doc] for doc in docs]
        logging.info(f"Generated {len(pairs)} pairs for prediction.")

        # Get scores from the cross encoder
        scores = cross_encoder.predict(pairs)
        logging.info("Scores have been predicted successfully.")

        # Sort indices based on scores
        sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
        logging.info(f"Sorted document indices based on scores.")

        # Select top documents based on the sorted indices
        top_docs = [docs[i] for i in sorted_indices[:count]]
        logging.info(f"Selected top {count} documents for the final output.")

    except Exception as e:
        logging.error(f"Error during the re-ranking process: {e}")
        raise  # Reraise the exception for further handling if necessary

    return top_docs

# Fuzzy search

In [ ]:
from fuzzywuzzy import process

#############################
"""
Perform fuzzy search

IP:
1. Word to be searched
2. list of words to be searched from
3. count of how many relevant search to retrieve
4. score threshold

OP:
Top matches with score higher than 60
"""
#############################
def fuzzy_search(search_word, search_from, limit=3, score=90):
    try:
        logging.info(f"Starting fuzzy search for '{search_word}' with limit {limit} and score threshold {score}.")
        fuzzy_matches = process.extract(search_word, search_from, limit=limit)
        
        top_matches = [match[0] for match in fuzzy_matches if match[1] > score]
        logging.info(f"Found {len(fuzzy_matches)} matches. Filtered down to {len(top_matches)} based on score threshold.")

    except Exception as e:
        logging.error(f"Error during fuzzy search: {e}")
        raise  # Reraise the exception for further handling if necessary

    return top_matches


#############################
"""
Map fuzzy search results to entities in DB

IP:
1. List of word to be searched
2. list of words to be searched from
3. count of how many relevant search to retrieve
4. score threshold

OP:
1. list of fuzzy search results to all input words
"""
#############################
def get_matches(word_list, search_from, limit=3, score=90):
    top_matches_set = set()

    try:
        logging.info(f"Starting to get matches for {len(word_list)} words.")
        for word in word_list:
            logging.info(f"Searching matches for word: '{word}'")
            matches = fuzzy_search(word, search_from, limit, score)
            top_matches_set.update(matches)
        
        top_matches = list(top_matches_set)
        logging.info(f"Found a total of {len(top_matches)} unique matches across all words.")

    except Exception as e:
        logging.error(f"Error while getting matches: {e}")
        raise  # Reraise the exception for further handling if necessary

    return top_matches


In [ ]:
# q = "ما هي الفواتير التي عمل عليها ياسين؟"
# ql = q.split()

In [ ]:
# res = get_matches(ql,all_translated_words)


# ocs = re_rank_doc(q, res, CROSS_ENCODER_MODEL, 3)

# ties = map_fuzzy_to_entity_translator_context(ocs, translated_texts_dict, entity_translate_map )
# str_list = [str(ele) for ele in ties]

In [ ]:
# str_list

# Context 

## Context from entity for SQL query generator 

In [ ]:
#############################
"""
Map fuzzy search results to entities

IP:
1. List of words to map
2. List of dict containg all word to entity maps [{Entity: [words, in, entity]}, ... ]

OP:
1. fuzzy search results mapped to noun maps - [entity1, entity2, ....]
"""
#############################
from collections import defaultdict

def build_reverse_lookup(entity_words_map):
    reverse_map = defaultdict(set)  # A dictionary to map words to entities
    try:
        logging.info("Building reverse lookup map.")
        for entity, words in entity_words_map.items():  # Iterate through the dictionary directly
            for word in words:
                reverse_map[word].add(entity)  # Map word to the corresponding entity
        logging.info("Successfully built the reverse lookup map.")

    except Exception as e:
        logging.error(f"Error while building reverse lookup map: {e}")
        raise  # Reraise the exception for further handling if necessary

    return reverse_map



def map_word_to_entity(word_list, reverse_map):
    entities = set()
    try:
        logging.info(f"Mapping words to entities for word list of size: {len(word_list)}.")
        for word in word_list:
            if word in reverse_map:  # Direct lookup in reverse map
                entities.update(reverse_map[word])
        logging.info(f"Mapping complete. Found {len(entities)} unique entities.")

    except Exception as e:
        logging.error(f"Error while mapping words to entities: {e}")
        raise  # Reraise the exception for further handling if necessary

    return entities

reverse_map = build_reverse_lookup(en_word_entity_map)


In [ ]:
# en_word_entity_map
# # en_all_split_words
# # Abdulaziz
# # Baraka

In [ ]:
# import time

# # start_time = time.time()
# # end_time = time.time()
# # print(f"Time taken by build_reverse_lookup: {end_time - start_time} seconds")

# word_list = ["Abdulaziz", "Baraka"]
# start_time = time.time()
# entities = map_word_to_entity1(word_list, en_word_entity_map)
# end_time = time.time()
# print(f"Time taken by map_word_to_entity1: {end_time - start_time} seconds")


# start_time = time.time()
# entities = map_word_to_entity(word_list, reverse_map)
# end_time = time.time()
# print(f"Time taken by map_word_to_entity: {end_time - start_time} seconds")

In [ ]:
# entities

## Context from enity for translator

In [ ]:
#############################
"""
Map fuzzy search results to entities in DB

IP:
1. List of words to map
2. List of dict containg all word to entity maps {Entity: [words, in, entity], ... }
3. list of dict containg [{en:ar},..] maps

OP:
1. fuzzy search results mapped to {en:ar}
"""
#############################
def build_translation_lookup(translate_map):
    try:
        logging.info("Building translation lookup.")
        translation_lookup = {v: k for ele in translate_map for k, v in ele.items()}
        logging.info("Successfully built the translation lookup.")
    except Exception as e:
        logging.error(f"Error while building translation lookup: {e}")
        raise  # Reraise the exception for further handling if necessary

    return translation_lookup


def map_fuzzy_to_entity_translator_context(word_list, entity_words_map, translation_lookup=translation_lookup):
    if translation_lookup is None:
        logging.warning("No translation lookup provided; proceeding without it.")
        translation_lookup = {}

    try:
        logging.info("Mapping fuzzy words to entities.")
        
        # Step 1: Find matching entities for the word list
        entities = map_word_to_entity(word_list, entity_words_map)
        
        # Step 2: Map entities to their corresponding translations using the lookup
        word_entity_map = [
            {translation_lookup[entity]: entity} for entity in entities if entity in translation_lookup
        ]
        
        logging.info(f"Mapping complete. Found {len(word_entity_map)} translation mappings.")

    except Exception as e:
        logging.error(f"Error while mapping fuzzy words to entities: {e}")
        raise  # Reraise the exception for further handling if necessary

    return word_entity_map


translation_lookup = build_translation_lookup(entity_translate_map)


In [ ]:
# # # translated_texts_dict
# entity_translate_map
# translation_lookup


In [ ]:
# word_list = ["الجزيرة", "الزهراء"]

# start_time = time.time()
# # entities = map_word_to_entity1(word_list, en_word_entity_map)
# as1 = map_fuzzy_to_entity_translator_context1(word_list, translated_texts_dict, entity_translate_map )
# end_time = time.time()
# print(f"Time taken by map_fuzzy_to_entity_translator_context1: {end_time - start_time} seconds")


# start_time = time.time()
# # entities = map_word_to_entity(word_list, reverse_map)
# as2 = map_fuzzy_to_entity_translator_context(word_list, translated_texts_dict, entity_translate_map)
# end_time = time.time()
# print(f"Time taken by map_fuzzy_to_entity_translator_context: {end_time - start_time} seconds")


## Context from query for sql query generator 

In [ ]:
import json


def get_documents(question, entities_list):
    output = ""
    top_tabel_schema_str = ""
    noun_doc_str = ""
    noun_doc_str_prompt = ""

    try:
        #######################################################################################
        ################################# RAG table schema from query #############################
        #######################################################################################
        logging.info("Retrieving relevant parent documents for the table schema.")
        retrieved_parent_docs = get_relevant_parent_docs(question)
        retrieved_tables_str = [json.dumps(doc) for doc in retrieved_parent_docs]
        logging.info(f"Retrieved {len(retrieved_parent_docs)} parent documents.")

        ############################# Re-Rank & filter #############################
        if retrieved_tables_str:
            logging.info("Re-ranking table schema documents.")
            top_tabel_schema = re_rank_doc(question, retrieved_tables_str, CROSS_ENCODER_MODEL, 2)
            logging.debug(f"Length of table schema documents after Re-ranking is {len(top_tabel_schema)}")
            logging.debug(f"Table schema documents after Re-ranking is {top_tabel_schema}")

            logging.info("Filtering table schema documents.")
            filter_retrieved_parent_docs = filter_context(question, top_tabel_schema, CODE_BISON)
            logging.debug(f"Length of table schema documents after filtering is {len(filter_retrieved_parent_docs)}")
            logging.debug(f"Table schema documents after filtering is {filter_retrieved_parent_docs}")

            if filter_retrieved_parent_docs:
                top_tabel_schema_str = '\n\n'.join([str(doc) for doc in filter_retrieved_parent_docs])
            else:
                top_tabel_schema_str = '\n\n'.join([str(doc) for doc in top_tabel_schema])

        #######################################################################################
        ################################# RAG entity from query #############################  
        #######################################################################################
        logging.info("Retrieving noun documents from PostgreSQL.")
        retrieved_noun_docs = get_noun_pg_results(question) 
        retrieved_noun_docs_str = [str(doc) for doc in retrieved_noun_docs]
        logging.info(f"Retrieved {len(retrieved_noun_docs)} noun documents.")

        if entities_list:
            logging.info("Getting schema for entities list.")
            entities_list_docs = get_schema(entities_list)
            entities_list_docs_str = [str(doc) for doc in entities_list_docs]
            combined_noun_docs_set = set(entities_list_docs_str + retrieved_noun_docs_str)
            retrieved_noun_docs_str = list(combined_noun_docs_set)

        ############################# Re-Rank & filter #############################
        if retrieved_noun_docs_str:
            logging.info("Re-ranking noun documents.")
            top_entity_count = 2
            logging.info(f"Max Re-ranked noun documents len {top_entity_count}.")
            top_noun_docs = re_rank_doc(question, retrieved_noun_docs_str, CROSS_ENCODER_MODEL, top_entity_count)
            logging.info(f"Noun documents after Re-ranking is {top_noun_docs}")

            logging.info("Filtering noun documents.")
            filtered_noun_docs_str = filter_context(question, top_noun_docs, CODE_BISON, True)
            logging.info(f"Length of noun documents after filtering is {len(filtered_noun_docs_str)}")
            logging.info(f"Noun documents after filtering is {filtered_noun_docs_str}")

            if filtered_noun_docs_str:
                noun_doc_str = '\n\n'.join([str(doc) for doc in filtered_noun_docs_str])

        if noun_doc_str:
            noun_doc_str_prompt = "\n\n" + "Below is the possible relevant SQL query and query result for the user's question: " + "\n\n" + noun_doc_str
        else:
            noun_doc_str_prompt = ""

        ################ PERFORM FOR ENTITY LIST ###########################
        output = top_tabel_schema_str + noun_doc_str_prompt

    except Exception as e:
        logging.error(f"Error occurred while getting documents: {e}")
        output = "An error occurred while processing your request."

    return output

In [ ]:
# # # query = "tell  me about amal station" # n = 1 (correct) t = 1 station-master

# # #######################################################################################
# # # query =  "What were the employee sales for the date 18th of May 2024?" # n = nan t = 1 daily-sales
# # # query =  "How much fuel was requested on the 1st week of May 2024?" # n = nan, t = 1 fuel req
# # # query =  "What is the employee sales of the 1st 2 weeks of May 2024?" # n = nan, t = 1 daily
# # # query =  "give me a brief summary on transaction done in may 2024" # n = nan
# # # query =  "Give me info about Al-Khaleej." # n = 3 (2 correct, 1 similar)
# query =  "Can you give me some info about Samra station?" # n = 1 (correct)
# # # query =  "What stations have restroom, charging station, and car wash?" # n = 2 (wrong) # cleaning entity vectore store should fix it 
# # """
# # ['{\'query\': "SELECT * FROM `ninth-bonito-424905-g3.my_dataset.janitor-data` WHERE `station_name` = \'Al-Mansour station\'", \'result\': {\'fuel_types\': \'Super 98\', \'location\': \'Khor Fakkan\', \'station_name\': \'Al-Mansour station\', \'restroom\': True, \'parking\': False, \'charging_station\': True, \'air_pump\': True, \'Fueling\': True, \'Oil Services\': True, \'Convenience Store\': False, \'ATM Machines\': False, \'Car Wash\': True, \'Fast Food Restaurants\': False}}', '{\'query\': "SELECT * FROM `ninth-bonito-424905-g3.my_dataset.janitor-data` WHERE `station_name` = \'Al-Sharq station\'", \'result\': {\'fuel_types\': \'Special 95\', \'location\': \'Buraidah\', \'station_name\': \'Al-Sharq station\', \'restroom\': True, \'parking\': False, \'charging_station\': False, \'air_pump\': False, \'Fueling\': False, \'Oil Services\': False, \'Convenience Store\': True, \'ATM Machines\': True, \'Car Wash\': True, \'Fast Food Restaurants\': False}}']
# # """
# # # query =  "Who approved the fuel requested on 19th may 2024?" # n = 0
# # # query =  "What were the total fuel sales for the month of May 2024? provide the cumulative sales amount." # n = 0
# # # query =  "How many transactions were completed on May 25, 2024? " # n = 0
# # # query =  "Can you give the total count of transactions for that day?" # n = 0
# # # query =  "Who were the top three employees in terms of sales for the month of May 2024? Please list their names and sales amounts." # n = 0
# # # query =  "How much revenue was generated from fuel sales in the second week of May 2024? Provide the total revenue for that period." # n = 0
# # # query =  "What is the average transaction amount for May 2024? Include the average value of all transactions for the month." # n = 0
# # # query =  "Can you provide the total number of fuel requests made in May 2024? I need the count of all fuel requests submitted that month." # n = 0
# # #######################################################################################



# # # # # q= "which bill did amal translate?"
# # # # # query = "أخبريني عن الفاتورة التي عالجتها أمل"


# # # retrieved_parent_docs = get_relevant_parent_docs(query)
# # # retrieved_tables_str = [json.dumps(doc) for doc in retrieved_parent_docs]
# # # top_tabel_schema = re_rank_doc(query, retrieved_tables_str, cross_encoder, 4)
# # # filter_retrieved_parent_docs = filter_context(query, top_tabel_schema, AGENT_MODEL)   ## gives good result some times filters out useful content
# # # top_tabel_schema_str = '\n\n'.join([str(doc) for doc in filter_retrieved_parent_docs])

# query = "Give me a summary of the transactions that took place in May 2024"
# ent = [""]
# r = get_documents(query, ent)

In [ ]:
# r

## NLP

In [ ]:
########################################
# Arabic NLP
########################################
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download the stopwords and punkt tokenizer models if not already downloaded
try:
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    logging.info("NLTK stopwords and tokenizer models downloaded successfully.")
except Exception as e:
    logging.error(f"Error downloading NLTK resources: {e}")

# Define the list of Arabic stop words
arabic_stopwords = set(stopwords.words('arabic'))

# Function to remove stop words from a given text
def remove_stopwords(text):
    try:
        # Modify the regex to include Arabic characters and numbers
        text = re.sub(r'[^\u0600-\u06FFa-zA-Z0-9\s]', '', text)
        logging.info("Text cleaned of non-Arabic characters.")

        words = nltk.word_tokenize(text)
        logging.info(f"Tokenized text into {len(words)} words.")

        filtered_words = [word for word in words if word not in arabic_stopwords]
        logging.info(f"Removed stopwords, resulting in {len(filtered_words)} words.")

        return ' '.join(filtered_words)

    except Exception as e:
        logging.error(f"Error occurred while removing stopwords: {e}")
        return text  # Return the original text if an error occurs



# Parsers

In [ ]:
from langchain.schema import BaseOutputParser

#############################
"""
String parser for new translated query chain
"""
#############################

class CustomOutputParser(BaseOutputParser):
    def parse(self, text):
        try:
            # Split the text to extract translated text and entities
            translated_text_part, entities_part = text.split("Entities used:", 1)
            translated_text = translated_text_part.replace("Translated Text:", "").strip()
            entities = [entity.strip() for entity in entities_part.split(",") if entity.strip()]
            logging.info("Parsed translated text and entities successfully.")
            return {"translated_text": translated_text, "entities": entities}
        except Exception as e:
            logging.error(f"Error in CustomOutputParser: {e}")
            return {"translated_text": text, "entities": []}

#############################
"""
Dict parser for NER chain 
"""
#############################

class DictOutputParser(BaseOutputParser):
    def parse(self, output):
        if isinstance(output, str):
            match = re.search(r'{.*}', output, re.DOTALL)
            if match:
                dictionary_str = match.group(0)
                try:
                    dictionary = ast.literal_eval(dictionary_str)
                    logging.info("Parsed dictionary successfully.")
                    return dictionary
                except (SyntaxError, ValueError) as e:
                    logging.error(f"Error parsing dictionary: {e}")
                    return {}
        return {}

    def parse_result(self, result):
        if 'text' in result:
            return self.parse(result['text'])
        return {}

# Helper Chains

## RAG Filter Chain

In [ ]:
#######################################
# Filter RAG documents
######################################
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.chains import LLMChain

def filter_context(question, context, llm, entity_query=False, entity=False, fuzzy_entity=False):
    """
    Filters relevant contexts based on the given question using a specified LLM.

    Parameters:
    - question: The question to evaluate relevance against.
    - context: A list of context strings to filter.
    - llm: The language model to use for processing.
    - entity_query: Flag to indicate if relevance is for SQL queries.
    - entity: Flag to indicate if relevance is for entity presence in context.
    - fuzzy_entity: Flag to indicate if relevance is for entity presence in a question.

    Returns:
    - List of relevant contexts that are deemed relevant to the question.
    """
    
    # Define prompt templates for different scenarios
    prompt_templates = {
        'entity_query': "You are an assistant that determines the relevance of the SQL query and its result to a given question.\n\nQuestion: {question}\n\n{context}\n\nIs this query relevant to the question above? Answer 'yes' or 'no'.",
        'entity': "You are an assistant that determines the presence of the Entity in a given Context.\n\nEntity: {question}\n\nContext: {context}\n\nIs this Entity present in the context above? Answer 'yes' or 'no'.",
        'fuzzy_entity': "You are an assistant that determines the presence of the Entity in a given question.\n\nQuestion: {question}\n\nEntity: {context}\n\nIs this Entity present in the question above? Answer 'yes' or 'no'.",
        'default': "You are an assistant that determines the relevance of table schema to a given question.\n\nQuestion: {question}\n\nTable: {context}\n\nIs this table relevant to the question above? Answer 'yes' or 'no'."
    }
    
    prompt_key = 'entity_query' if entity_query else 'entity' if entity else 'fuzzy_entity' if fuzzy_entity else 'default'
    
    prompt_template = PromptTemplate(
        input_variables=["question", "context"],
        template=prompt_templates[prompt_key]
    )

    filter_chain = LLMChain(llm=llm, prompt=prompt_template)

    try:
        inputs = [{"question": question, "context": ctx} for ctx in context]
        results = filter_chain.apply(inputs)

        # Extract relevant contexts
        relevant_contexts = [ctx["context"] for ctx, res in zip(inputs, results) if 'yes' in res["text"].lower()]
        logging.info(f"Filtered relevant contexts: {len(relevant_contexts)} found.")
        return relevant_contexts

    except Exception as e:
        logging.error(f"Error in filtering context: {e}")
        return []


## AR:EN NER chain # NOT GIVING GOOD RESULT

In [ ]:
# def ar_en_ner_identifier(llm, text, tgt):
#     ner_prompt = PromptTemplate.from_template(
#         """Your are an expert at identifying Entitie and an expert translator.
#         Given the original user query, identify all the possible named entities in the query, translate it to {tgt} and return a dictionary where the key is the entity in the english and the value is the entity in Arabic.

#         Original user query:
#         {text}

#         Result: """
#     )


#     dict_output_parser = DictOutputParser()
#     ar_ner_chain = LLMChain(prompt=ner_prompt, llm=llm, output_parser=dict_output_parser)

#     ner_dict = ar_ner_chain.run({"tgt": {tgt}, "text": text})
#     return ner_dict

In [ ]:
# # # CODE_BISON
# # # INTERPRETOR_MODEL
# # # AGENT_MODEL
# # # CHAT_BISON
# # # TEXT_BISON
# # q = "ما هي الفواتير التي عمل عليها ياسين؟"
# # q = "اعطيني معلومات عن الخليج"
# q = "اعطيني بعض المعلومات عن محطة سمرة"
# res = ar_en_ner_identifier(CHAT_BISON, q, "en")
# res

## Adv Translator chain

In [ ]:
def translator_with_rag(llm, text, tgt, rag_result, translated_query):
    """
    Translates the user query based on rough translation and possible entities using a Language Model (LLM).
    
    Parameters:
    - llm: The language model used for translation.
    - text: The original user query.
    - tgt: The target language for translation.
    - rag_result: Retrieved documents or entities used for translation.
    - translated_query: The initial rough translation of the query.
    
    Returns:
    - result: A more accurate translated query based on the given inputs.
    """

    logging.info("Starting the translation process with RAG")
    logging.debug(f"Input text: {text}")
    logging.debug(f"Target language: {tgt}")
    logging.debug(f"RAG result: {rag_result}")
    logging.debug(f"Rough translation: {translated_query}")

    try:
        # Define the prompt template for LLM
        prompt = PromptTemplate.from_template(
            """You are an expert Translator especially in English and Arabic.
            The query can be related to Bills, Daily Sales, Dispenser Configurations, Fuel Requests, Fuel Types, Locations, Nozzles, Ports, Regions, Shifts, Fuel Stations, Tenants, Transactions, Users.
            
            Below provided are the original user query, query translated roughly to {tgt}, possible entities in the query and its translations in {tgt}. 
            Return more accurate translated query.
            
            Instructions:
            - Use the translation of an entity only if there is a good match. 
            - Do not alter or combine entities. Use them as they are.
            - Do not use entities if there is no good match.

            Original user query:
            {text}
            
            Roughly translated query:
            {translated_query}

            Possible entities:
            {rag_result}

            Result:
            """
        )

        # Initialize the output parser and LLM chain
        output_parser = CustomOutputParser()
        chain = LLMChain(prompt=prompt, llm=llm, output_parser=output_parser)

        # Run the chain to get the translation result
        result = chain.run({
            "tgt": tgt,
            "text": text,
            "rag_result": rag_result,
            "translated_query": translated_query
        })

        logging.info("Translation process completed successfully")
        logging.debug(f"Translation result: {result}")

        return result

    except Exception as e:
        logging.error(f"An error occurred during translation: {e}")
        return {"error": f"Failed to translate query: {str(e)}"}


In [ ]:
# # t = "اسمي مادوسودان"
# t ="اسمي أمل"
# r = translator_with_rag(AGENT_MODEL, t, "en", "{'Aramco station Amal': 'محطة أرامكو أمل'}" )

In [ ]:
# r

## NER chain

In [ ]:
##################################
# Identify entities in the user query 
##################################

def ner_identifier(llm, query):
    """
    Identifies named entities in the given user query using a Language Model (LLM).
    
    Parameters:
    - llm: The language model used for named entity recognition (NER).
    - query: The user query where named entities are to be identified.
    
    Returns:
    - entity_list: A comma-separated string of identified entities.
    """

    logging.info("Starting NER identification process")
    logging.debug(f"User query: {query}")

    try:
        # Define the NER prompt template for LLM
        ner_prompt = PromptTemplate.from_template(
            """You are an expert at identifying Entities.
            Given the original user query, identify the possible named entities in the query and return all entities (names, locations, addresses, etc.).

            Original user query:
            {text}

            Result (just the entities separated by commas):
            """
        )

        # Initialize the output parser and LLM chain
        output_parser = CommaSeparatedListOutputParser()
        chain = LLMChain(prompt=ner_prompt, llm=llm, output_parser=output_parser)

        # Run the chain to get the identified entities
        result = chain.invoke({"text": query})

        # Extract and return the list of entities
        entity_list = result['text']
        logging.info("NER identification completed successfully")
        logging.debug(f"Identified entities: {entity_list}")

        return entity_list

    except Exception as e:
        logging.error(f"An error occurred during NER identification: {e}")
        return {"error": f"Failed to identify entities: {str(e)}"}


In [ ]:
# q = "Can you give me some info about Samra station?"
# q = "Give me info about Al-Khaleej."
# # q = "اعطيني معلومات عن الخليج"
# # q = "هل يمكنك أن تعطيني بعض المعلومات عن محطة سمرة ؟"
# # q = "في أي وقت يفتح مادو؟"
# # q = "محطة المزيدة مفتوحة"
# rse = ner_identifier(CHAT_BISON, q)
# rse

In [ ]:
##################################
# Identify entities in the user query 
##################################

def ner_translator(llm, query):
    """
    Translates an Arabic entity name into English, focusing on pronunciation rather than meaning.
    
    Parameters:
    - llm: The language model used for translation.
    - query: The Arabic entity name to be translated.
    
    Returns:
    - entity_list: A string representing the translated entity name, or 'None' if it cannot be translated.
    """
    
    logging.info("Starting Arabic-to-English entity translation")
    logging.debug(f"Arabic entity provided for translation: {query}")

    try:
        # Define the translation prompt template for LLM
        ner_tran_prompt = PromptTemplate.from_template(
            """Given an entity name in Arabic, translate it to English. Do not translate its meaning; translate its pronunciation.
            If you can't, just return 'None'.

            Arabic entity:
            {text}

            Result (just the translated Arabic entity):
            """
        )

        # Initialize the output parser and LLM chain
        output_parser = CommaSeparatedListOutputParser()
        chain = LLMChain(prompt=ner_tran_prompt, llm=llm, output_parser=output_parser)

        # Run the chain to get the translated entity
        result = chain.invoke({"text": query})

        # Extract and return the translation result
        entity_list = result['text']
        logging.info("Translation process completed successfully")
        logging.debug(f"Translated entity: {entity_list}")

        return entity_list

    except Exception as e:
        logging.error(f"An error occurred during entity translation: {e}")
        return "None"

In [ ]:
# ress = ner_translator(CHAT_BISON, rse[0])
# ress


## Get SQL query chain

In [ ]:
# Convert dictionary to JSON formatted string
def _dict_to_json(x: dict) -> str:
    return "```\n" + json.dumps(x, indent=4) + "\n```"

# Clean SQL query by removing unwanted formatting
def clean_query(query: str) -> str:
    return query.replace("`sql\n", "").replace("```sql", "").replace("```", "").replace("``", "")

# Format SQL or Error prompt
def format_prompt(template, context=None, chat_history=None, question=None, error=None, result=None, query=None, entities=None, answer=None):
    """Helper function to format the SQL or Error prompt."""
    return template.format(
        context=context or '',
        chat_history=chat_history or '',
        question=question or '',
        error=error or '',
        query=query or '',
        result=result or '',
        entities=entities or '',
        answer=answer or ''
    )

In [ ]:
# Execute SQL query
def execute_query(query, fetch_all=True):
    connection = None
    result = None
    try:
        logging.info(f"Executing query: {query}")
        connection = connect_db() 
        cursor = connection.cursor()
        cursor.execute(query)

        if fetch_all:
            result = cursor.fetchall()
        else:
            result = cursor.fetchone()

        logging.info("Query executed successfully.")
    except Exception as e:
        logging.error(f"Error executing query: {e}")
        raise
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

    return result

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import CommaSeparatedListOutputParser

SQL_PROMPT = """You are a PostgreSQL expert.

Create a PostgreSQL query based on the following schema and user input.

Schema (JSON format):
{context}

Conversation so far:
{chat_history}

User query:
{question}

Instructions:
- Return only the PostgreSQL code.
- Use standard PostgreSQL syntax.
- Do not use backticks or any markup.
- Use double quotes for schema names and table names.
- Use double quotes for column names.
- Use single quotes for values.

SQL query:
"""

#######################################################################
ERROR_PROMPT = """Here are the PostgreSQL queries and their errors:

{error}

Modify the query to fix the errors using the schema and context below.

Schema (JSON format):
{context}

Conversation so far:
{chat_history}

User query:
{question}

Instructions:
- Return only the PostgreSQL code.
- Use standard PostgreSQL syntax.
- Do not use backticks or any markup.
- Use double quotes for schema names and table names.
- Use double quotes for column names.
- Use single quotes for values.

Corrected SQL query:
"""

# Generate SQL query using LLM and retry if errors occur
def sql_query_generator(input, config, palm_max_attempts=3, gemini_max_attempts=2):
    """
    Generates a PostgreSQL query using an LLM model and retries corrections based on errors.
    
    Parameters:
    - input: Input dictionary containing the user's query, entities, and chat history.
    - config: Configuration parameters.
    - palm_max_attempts: Max attempts using the first model (LLM).
    - gemini_max_attempts: Max attempts using the second model (fallback model).

    Returns:
    - A dictionary with the final query and results, or error information if failed.
    """
    try:
        docs = get_documents(input['input'], input['entities'])
        question = itemgetter("input")(input)
        chat_history = itemgetter("chat_history")(input)
        formatted_prompt = format_prompt(template=SQL_PROMPT, context=docs, chat_history=chat_history, question=question)

        max_attempts = palm_max_attempts + gemini_max_attempts
        attempts = 0
        error_list = []

        while attempts <= max_attempts:
            logging.info(f"Attempt {attempts+1} of {max_attempts}")
            
            if attempts < palm_max_attempts:
                logging.debug(f"Using Chat Code Bison for sql query generation")
                logging.debug(f"Prompt: {formatted_prompt}")
                query = CODE_BISON.invoke(formatted_prompt).content
            else:
                logging.debug(f"Using Gemini Pro for sql query generation")
                logging.debug(f"Prompt: {formatted_prompt}")
                query = GEMINI_PRO(formatted_prompt)
            
            cleaned_query = clean_query(query)
            logging.debug(f"Cleaned query: {cleaned_query}")
            
            try:
                # Attempt to execute the query
                query_result = execute_query(cleaned_query)
                logging.info(f"Query executed successfully: {cleaned_query}")
                return {"query": cleaned_query, "result": query_result, "context": docs, "question": question}
            
            except Exception as e:
                logging.error(f"Error executing query: {e}")
                error_message = f"Query: {cleaned_query}\nError: {str(e)}"
                error_list.append(error_message)
                errors_so_far = "\n".join(error_list)
                
                attempts += 1
                if attempts < palm_max_attempts:
                    # Generate a corrected query using the error message
                    formatted_error_prompt = format_prompt(ERROR_PROMPT, docs, chat_history, question, error=errors_so_far)
                    formatted_prompt = formatted_error_prompt
                elif attempts == palm_max_attempts:
                    # Retry with initial prompt
                    formatted_prompt = format_prompt(template=SQL_PROMPT, context=docs, chat_history=chat_history, question=question)
                elif attempts <= max_attempts:
                    # Use the error prompt for the fallback model
                    formatted_error_prompt = format_prompt(ERROR_PROMPT, docs, chat_history, question, error=errors_so_far)
                    formatted_prompt = formatted_error_prompt
                else:
                    logging.error("Failed to generate a correct query after multiple attempts.")
                    return {
                        "error": "Failed to generate a correct query after multiple attempts.",
                        "query": cleaned_query,
                        "error_messages": error_list,
                        "query_result": None
                    }
        return {"error": "Max attempts reached, no valid query.", "query_result": None}

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return {"error": "An unexpected error occurred.", "query_result": None}


## Interpret the result

In [ ]:
INTERPRET_PROMPT = """You are a PostgreSQL expert and skilled in extracting data from CSV.

A user asked:
{question}

The SQL query run was:
{query}

The query result in CSV format:
{result}

Based on this result, provide a brief answer to the user's question.

Instructions:
- Provide only the answer, without explaining how it was obtained.
- Extract the answer solely from the query result.
- Do not include the user question in your response.
- If the answer cannot be found, respond with "I cannot find the answer" and include a brief explanation with relevant details.
"""

def interpret_data(input, config):
    """
    Interprets data based on the input query and context.

    Parameters:
    - input: Dictionary containing the query, context, question, and result.
    - config: Configuration parameters (if any).

    Returns:
    - interpreted_result: The interpreted result of the BigQuery data.
    """

    try:
        logging.info("Interpreting data...")

        # Extract query and context from the input
        query = itemgetter("query")(input)
        context = itemgetter("context")(input)
        question = itemgetter("question")(input)
        result = itemgetter("result")(input)

        logging.info(f"Extracted input values: Query: {query}, Question: {question}")
        logging.debug(f"Query: {query}")
        logging.debug(f"Question: {question}")
        logging.debug(f"Context : {context}")


        # Format the prompt for data interpretation
        prompt_template = format_prompt(template=INTERPRET_PROMPT, question=question, result=result, query=query)
        logging.debug(f"Prompt: {prompt_template}")

        # Get the interpreted result
        interpreted_result = INTERPRETOR_MODEL.invoke(prompt_template)
        logging.info("Interpreted result retrieved successfully.")

        return interpreted_result.content

    except Exception as e:
        logging.error(f"Error interpreting data: {e}")
        return {"error": "Failed to interpret data", "details": str(e)}



## Translator English to Arabic Chain

In [ ]:
def translate_en_to_ar(query, query_response, entities_list, config):
    """
    Translates an English query result and response into Arabic, considering entity mappings.

    Parameters:
    - query: A dictionary containing the question, query, and query result.
    - query_response: The result to be translated.
    - entities_list: A list of possible entities that need special handling for translation.
    - config: Configuration object containing additional parameters (e.g., translation models).

    Returns:
    - translation_result.content: Translated response in Arabic.
    """
    try:
        # Define the translation prompt template
        TRANSLATE_PROMPT = """You are an expert English to Arabic Translator.

        A user asked this question:
        {question}

        To find the answer, the following SQL query was run in BigQuery:
        ```
        {query}
        ```

        The result of that query was the following table in CSV format:
        ```
        {result}
        ```

        {entities}

        This is the brief answer to the user question:
        ```
        {answer}
        ```

        Using all the above information, translate the answer from English to Arabic.

        Follow these restrictions strictly:
        - Use only the given information to translate.
        - If the entities in the answer are present in the given possible English : Arabic entity translation, use the Arabic entity as is.
        - Just write the answer, omit the question from your answer, this is a chat, just provide the answer.
        """

        logging.info("Starting translate_en_to_ar function.")

        # Extract relevant fields from the query
        question = query.get("question", "")
        sql_query = query.get("query", "")
        query_result = query.get("result", "")
        answer = query_response

        # Handle entity mappings if available
        entities = None
        if entities_list:
            entity_maps = []
            for entity in entities_list:
                for entity_map in entity_translate_map:
                    if entity in entity_map:
                        entity_maps.append({entity: entity_map[entity]})

            # Prepare the entity translation mapping
            if entity_maps:
                entity_map_str = '\n'.join([f"{en}: {ar}" for mapping in entity_maps for en, ar in mapping.items()])
                entities = f"\n\nHere are some entities in English and their Arabic translations that might exist in the answer:\n```\n{entity_map_str}\n```"
        
        logging.debug(f"Entity mappings: {entities}")

        # Format the prompt for translation
        prompt_template = format_prompt(
            template=TRANSLATE_PROMPT,
            question=question,
            result=query_result,
            query=sql_query,
            entities=entities,
            answer=answer
        )
        logging.debug(f"Formatted prompt: {prompt_template}")

        # Invoke the translation model
        translation_result = CHAT_BISON.invoke(prompt_template)
        logging.info("Translation result successfully retrieved.")
        
        return translation_result.content

    except Exception as e:
        logging.error(f"Error during translation: {e}", exc_info=True)
        return {"error": "Failed to translate the query result.", "details": str(e)}


## Test

In [ ]:
# # # # # question = {"input": "what were the total employee sales for the date 18th of May 2024", "chat_history": "", "entities": []}
# question = {"input": "Give me a summary of the operations that took place in May 2024", "entities" : [""], "chat_history": ""}
# # # question = {"input": "what were the employee sales for the date 18th of May 2024?", "entities" : [""], "chat_history": ""}

# # config={} # 'callbacks': [ConsoleCallbackHandler()]
# query = sql_query_generator(question, config)
# query_response = interpret_data(query, config=config)
# # entities_list = []

# translated_result = translate_en_to_ar(query, query_response, entities_list, config)


In [ ]:
# translated_result

In [ ]:

# # # question = {"input": "what were the total employee sales for the date 18th of May 2024", "chat_history": "", "entities": []}
# question = {"input": " Give me a summary of the operations that took place in May 2024", "entities" : [""], "chat_history": ""}

# config={} # 'callbacks': [ConsoleCallbackHandler()]

# query = sql_query_generator(question, config)


In [ ]:
# query

In [ ]:
# # # question = {"input": "what were the total employee sales for the date 18th of May 2024", "chat_history": "", "entities": []}
# question = {"input": " Give me a summary of the operations that took place in May 2024", "entities" : [""], "chat_history": ""}

# config={} # 'callbacks': [ConsoleCallbackHandler()]

# query = sql_query_generator(question, config)

# query_response, query_result = interpret_data(query, config=config)

In [ ]:
# query_response
# # query_result

In [ ]:
# # # question = {"input": "what were the total employee sales for the date 18th of May 2024", "chat_history": "", "entities": []}
# question = {"input": "Give me a summary of the operations that took place in May 2024", "entities" : [""], "chat_history": ""}
# question = {"input": "what were the employee sales for the date 18th of May 2024?", "entities" : [""], "chat_history": ""}

# config={} # 'callbacks': [ConsoleCallbackHandler()]
# query, query_result = sql_query_generator(question, config)

# query_response = interprect_data(query, query_result, config=config)

# translated_result = translate_en_to_ar(query, query_response, query_result, config)


In [ ]:
# # translated_result
# query
# # query_result

# Tools

## Math tool

In [ ]:
from langchain import LLMMathChain
from langchain.tools import Tool

math_chain = LLMMathChain.from_llm(llm=AGENT_MODEL)
math_tool = Tool(
  name="Calculator",
  description="Useful for when you need to answer questions about math.",
  func=math_chain.run,
  coroutine=math_chain.arun)

# Advanced translator

In [ ]:
def replace_words(text, replacements):
    words = text.split()
    new_words = [replacements.get(word, word) for word in words]
    return ' '.join(new_words)

# Define the dictionary for replacements
replacements = {
    "operations": "transactions",
    "sessions": "transactions"
}

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import re
from concurrent.futures import ThreadPoolExecutor

#############################
# Translator with RAG
#############################

do_print = False
def translate_text_v2(llm, text: str, tgt: str):
    """
    Translates a given text into the target language, with entity recognition and RAG (Retrieve and Generate) support.
    
    Parameters:
    - llm: Large language model used for translation tasks.
    - text: The input text to be translated.
    - tgt: The target language code.
    
    Returns:
    - new_query: The translated text or the original text if no translation was needed.
    - translated_entities_used: List of entities translated during the process.
    """
    try:
        translate_client = translate.Client()

        if isinstance(text, bytes):
            text = text.decode("utf-8")
        
        detected_source_language = translate_client.detect_language(text)["language"]
        logging.info(f"Detected source language: {detected_source_language}")

        logging.debug(f"Source languages: {detected_source_language}.")
        logging.debug("Target languages: {tgt}.")
        logging.debug("Text: {text}.")

        translated_entities_used = []
        if detected_source_language == tgt:
            logging.info("Source and target languages are the same. No translation needed.")



            return text, translated_entities_used

        # Translate the text if necessary
        translated_query = translate_text(text, tgt)
        logging.info("Basic translation sucessful.")
        logging.debug("Basic translated text: {translated_query}.")


        # Initialize variables for storing NER results and entity matching
        ar_ner_res, en_ner_res, query_match_entities_list = [], [], []
        ar_ner_res_list, en_ner_res_list, ner_match_entities_list = [], [], []

        #####################################################################################
        ############################# Query RAG and Filtering ##############################
        #####################################################################################

        query_rag = entity_translate_retriever.get_relevant_documents(text)
        query_rag_list = list({doc.page_content for doc in query_rag})

        if query_rag_list:
            query_re_rank = re_rank_doc(text, query_rag_list, CROSS_ENCODER_MODEL, 2)
            query_filter = filter_context(text, query_rag_list, INTERPRETOR_MODEL)

            logging.debug(f"Re-ranked query: {query_re_rank}")
            logging.debug(f"Filtered query: {query_filter}")

        #####################################################################################
        ############################# Fuzzy Search for Entities ############################
        #####################################################################################

        query_cleaned = remove_stopwords(text)
        query_top_match = get_matches(query_cleaned.split(), all_translated_words)

        if query_top_match:
            query_re_ranked_match = re_rank_doc(text, query_top_match, CROSS_ENCODER_MODEL, 3)
            query_re_ranked_match_entities = map_fuzzy_to_entity_translator_context(query_re_ranked_match, translated_texts_dict, entity_translate_map)
            query_filtered_match = filter_context(text, query_top_match, INTERPRETOR_MODEL, fuzzy_entity=True)

            query_match_entities = map_fuzzy_to_entity_translator_context(query_filtered_match, translated_texts_dict, entity_translate_map)
            query_match_entities_list = [str(entity) for entity in query_match_entities]
            logging.debug(f"Fuzzy matched entities: {query_match_entities_list}")

        #####################################################################################
        ############################# Named Entity Recognition (NER) ########################
        #####################################################################################

        ar_ner = ner_identifier(CHAT_BISON, text)
        if ar_ner:
            en_ner = [translated for ner in ar_ner for translated in ner_translator(CHAT_BISON, ner)]
            logging.debug(f"Arabic NER: {ar_ner}")
            logging.debug(f"English NER: {en_ner}")

            # # Handle Arabic NER
            # for ner_item in ar_ner:
            #     process_ner_entities(ner_item, ar_ner_res, ar_ner_res_list, "Arabic")

            # # Handle English NER
            # for ner_item in en_ner:
            #     process_ner_entities(ner_item, en_ner_res, en_ner_res_list, "English")

        entity_count=1
        if ar_ner:
            entity_count = len(ar_ner)
        
            ner_res = []
            ner = []
            ner = ar_ner + en_ner
            # ner.append(en_ner)
            start_time = time.time()
            with ThreadPoolExecutor() as executor:
                ar_future = executor.submit(process_ner, ar_ner, 'ar')
                en_future = executor.submit(process_ner, en_ner, 'en')
                
                ar_ner_res_list = ar_future
                en_ner_res_list = en_future
                
                logging.debug(f"Filtered Arabic NER: {ar_ner_res_list}")
                logging.debug(f"Filtered English NER: {en_ner_res_list}")

                
                
        #####################################################################################
        ############################# Fuzzy Search for NER Entities #########################
        #####################################################################################

        ner_split_flattened = [item for sublist in [remove_stopwords(ele).split() for ele in ar_ner] for item in sublist]
        if 'محطة' in ner_split_flattened:
            ner_split_flattened.remove('محطة')

        ner_top_matches = get_matches(ner_split_flattened, all_translated_words, 5)
        if ner_top_matches:
            process_fuzzy_matches(ner_top_matches, ar_ner, en_ner_res_list, ar_ner_res_list)

        #####################################################################################
        ############################# Final RAG and Fuzzy Result ############################
        #####################################################################################

        entities_list = list(set(query_filter + query_match_entities_list + en_ner_res_list + ar_ner_res_list + ner_match_entities_list))
        logging.info(f"Final list of entities: {entities_list}")

        ############################# LLM Chain for Translation #############################
        
        result = translator_with_rag(CHAT_BISON, text, tgt, entities_list, translated_query)
        new_query = replace_words(result['translated_text'], replacements)
        logging.debug(f"Translation with adv translator with out replacer: {result}")
        logging.debug(f"Translation with replacer: {new_query}")

        translated_entities_used = result['entities']
        
        logging.info("Translation completed successfully.")
        return new_query, translated_entities_used

    except Exception as e:
        logging.error(f"An error occurred during translation: {e}", exc_info=True)
        return {"error": "Translation failed.", "details": str(e)}


# def process_ner_entities(ner_item, ner_res, ner_res_list, lang_type):
#     """
#     Helper function to process NER entities and apply RAG and filtering.
    
#     Parameters:
#     - ner_item: The named entity to process.
#     - ner_res: List to store filtered NER results.
#     - ner_res_list: List to store final NER results.
#     - lang_type: String indicating whether the NER is 'Arabic' or 'English'.
#     """
#     try:
#         ner_rag = entity_translate_retriever.get_relevant_documents(ner_item)
#         ner_rag_list = list({doc.page_content for doc in ner_rag})

#         if ner_rag_list:
#             ner_re_rank = re_rank_doc(ner_item, ner_rag_list, CROSS_ENCODER_MODEL)
#             ner_filter = filter_context(ner_item, ner_rag_list, INTERPRETOR_MODEL, entity_query=False, entity=True)
#             ner_res.append(list(ner_filter))
#             ner_res_list.extend(sum(ner_res, []))

#         logging.debug(f"{lang_type} NER re-ranked: {ner_re_rank}")
#         logging.debug(f"{lang_type} NER filtered: {ner_filter}")

#     except Exception as e:
#         logging.error(f"Error processing {lang_type} NER entity: {e}", exc_info=True)


def process_ner(ner, language):
    ner_res = []
    start_time = time.time()

    for ele in ner:
        ############# RAG #############
        ner_rag = entity_translate_retriever.get_relevant_documents(ele)
        ner_rag_list = list({doc.page_content for doc in ner_rag})  # Unique results
        
        ############# Re-Rank and filter #############
        if ner_rag_list:
            # ner_re_rank = re_rank_doc(ele, ner_rag_list, CROSS_ENCODER_MODEL)
            ner_filter = filter_context(ele, ner_rag_list, interpret_data_model, entity_query=False, entity=True)
            ner_res.append(list(ner_filter))

    ner_res_future = ner_res.result()
    ner_res_list = sum(ner_res_future, [])

    return ner_res_list


def process_fuzzy_matches(ner_top_matches, ar_ner, en_ner_res_list, ar_ner_res_list):
    """
    Helper function to process fuzzy matches for NER entities.
    
    Parameters:
    - ner_top_matches: The top matches from the fuzzy search.
    - ar_ner: Arabic NER results.
    - en_ner_res_list: List to store English NER results.
    - ar_ner_res_list: List to store Arabic NER results.
    """
    try:
        ner_re_rank_match = re_rank_doc(ar_ner, ner_top_matches, CROSS_ENCODER_MODEL, len(ar_ner) * 2)
        ner_rr_match_entities = map_fuzzy_to_entity_translator_context(ner_re_rank_match, translated_texts_dict, entity_translate_map)
        ner_filtered_match = filter_context(ar_ner, ner_top_matches, INTERPRETOR_MODEL, fuzzy_entity=True)
        ner_match_entities = map_fuzzy_to_entity_translator_context(ner_filtered_match, translated_texts_dict, entity_translate_map)

        en_ner_res_list.extend(ner_rr_match_entities)
        ar_ner_res_list.extend(ner_match_entities)

        logging.debug(f"Fuzzy NER matches processed successfully.")

    except Exception as e:
        logging.error(f"Error processing fuzzy NER matches: {e}", exc_info=True)


In [ ]:
# ### q = "اعطيني معلومات عن الخليج"
# # q = "هل يمكنك أن تعطيني بعض المعلومات عن محطة سمرة ؟"
# # q = "في أي وقت يفتح مادو؟"
# # q = "محطة المزيدة مفتوحة"
# # opt, ent = 
# # q = "أعطني ملخصًا موجزًا للمعاملات التي تمت في مايو 2024"
# # q = "عطني ملخص العمليات التي تمت في شهر مايو سنة 2024"
# q = "اعطيني معلومات عن الخليج"
# a,b = translate_text_v2(CHAT_BISON, q, "en")
# # print(opt)
# # print(ent)

In [ ]:
# b

In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# import re
# from concurrent.futures import ThreadPoolExecutor
# #############################
# # Translator with RAG
# #############################

# do_print = False
# def translate_text_v2(llm, text: str, tgt: str): #-> dict:
    
#     translate_client = translate.Client()

#     if isinstance(text, bytes):
#         text = text.decode("utf-8")
    
#     detected_source_language = translate_client.detect_language(text)["language"] # time taken 0.117 sec
    
#     translated_entities_used = []
#     if detected_source_language == tgt:
#         new_query = text
#     else:
#         translated_query = translate_text(text, tgt) # time taken 0.188 sec

#         ar_ner_res = []
#         en_ner_res = []
#         query_filter = []
#         query_match_entities_list = []
#         en_ner_res_list = [] 
#         ar_ner_res_list = []
#         ner_match_entites_list = []
#         #####################################################################################
#         ############################# query RAG #############################
#         #####################################################################################
#         start_time = time.time()
        
#         query_rag = entity_translate_retriever.get_relevant_documents(text)
#         query_rag_list = list({doc.page_content for doc in query_rag})
        
#         ############# Re-Rank and filter ############# 
#         if query_rag_list:
#             query_re_rank = re_rank_doc(text, query_rag_list, CROSS_ENCODER_MODEL, 2)

#             query_filter = filter_context(text, query_rag_list, interpret_data_model)
            
#             if do_print:
#                 print("query_re_rank !!!!!!!!!!!!!!!!")
#                 print(query_re_rank)
#                 print("query_filter !!!!!!!!!!!!!!!!")
#                 print(query_filter)

#         end_time = time.time()
#         print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TIME !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#         print(f"Time taken by query RAG: {end_time - start_time} seconds")
        
#         #####################################################################################
#         ############################# Fuzzy search query ############################# # time taken 0.244
#         #####################################################################################

#         query_match_entites_list = ""
#         query_cleaned = remove_stopwords(text)
#         query_split = query_cleaned.split()
#         query_top_match = get_matches(query_split, all_translated_words)

#         if query_top_match:
#             if 'محطة' in query_top_match:
#                 query_top_match.remove('محطة')
#             if do_print:
#                 print("query_top_match #####################################################################################")
#                 print(query_top_match)
#             ###################################### Re-Rank ##############################################

#             query_re_ranked_match = re_rank_doc(text, query_top_match, CROSS_ENCODER_MODEL, 3)
#             query_re_ranked_match_entites = map_fuzzy_to_entity_translator_context(query_re_ranked_match, translated_texts_dict, entity_translate_map )
#             query_re_ranked_match_entites_list = [str(ele) for ele in query_re_ranked_match_entites]
#             if do_print:
#                 print("query_re_ranked_match #####################################################################################")
#                 print(query_re_ranked_match)

#                 print("query_re_ranked_match_entites #####################################################################################")
#                 print(query_re_ranked_match_entites)
            
#             ###################################### Filter ##############################################
#             query_filtered_match = filter_context(text, query_top_match, interpret_data_model, fuzzy_entity=True)

#             query_match_entities = map_fuzzy_to_entity_translator_context(query_filtered_match, translated_texts_dict, entity_translate_map )
#             if do_print:
#                 print("query_filtered_match #####################################################################################")
#                 print(query_filtered_match)

#                 print("query_match_entities #####################################################################################")
#                 print(query_match_entities)
            
#             query_match_entities_list = [str(ele) for ele in query_match_entities]

#         #####################################################################################
#         ############################# NER Chain ############################# # Time taken 1.767 sec
#         #####################################################################################
        
#         ar_ner = ner_identifier(translation_model, text)

#         if ar_ner:
#             # en_ner = [ner_translator(translation_model, ner) for ner in ar_ner]
#             en_ner = [translated for ner in ar_ner for translated in ner_translator(translation_model, ner)]
#             if do_print:
#                 print("Arabic NER !!!!!!!!!!!!!!!!")
#                 print(ar_ner)
#                 print("English NER !!!!!!!!!!!!!!!!")
#                 print(en_ner)

#         #####################################################################################
#         ############################# ar+en NER #############################
#         #####################################################################################
#         entity_count=1
#         if ar_ner:
#             entity_count = len(ar_ner)
        
#             ner_res = []
#             ner = []
#             ner = ar_ner + en_ner
#             # ner.append(en_ner)
#             start_time = time.time()
#             with ThreadPoolExecutor() as executor:
#                 ar_future = executor.submit(process_ner, ar_ner, 'ar')
#                 en_future = executor.submit(process_ner, en_ner, 'en')
                
#                 ar_ner_res = ar_future
#                 en_ner_res = en_future

#                 print(ar_ner_res_list)
#                 print(en_ner_res_list)
                
                
#             end_time = time.time()
#             print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TIME !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#             print(f"Time taken by ar + en NER: {end_time - start_time} seconds")
        
# #         #####################################################################################
# #         ############################# ar NER #############################
# #         #####################################################################################

# #         if ar_ner:
# #             start_time = time.time()
            

# #             for ele in ar_ner:
# #                 ############# RAG ############# 
# #                 ar_ner_rag = entity_translate_retriever.get_relevant_documents(ele)
# #                 ar_ner_rag_list = list({doc.page_content for doc in ar_ner_rag})  # Unique results
# #                 if do_print:
# #                     print("ar_ner_rag_list #####################################################################################")
# #                     print(f"ele : {ele}")
# #                     print(ar_ner_rag_list)

# #                 ############# Re-Rank and filter ############# 
# #                 if ar_ner_rag_list:
# #                     ar_ner_re_rank = re_rank_doc(ele, ar_ner_rag_list, CROSS_ENCODER_MODEL)
                    
# #                     ar_ner_filter = filter_context(ele, ar_ner_rag_list, interpret_data_model, entity_query= False, entity=True)
# #                     ar_ner_res.append(list(ar_ner_filter))
# #                     ar_ner_res_list = sum(ar_ner_res, [])

# #                     # if do_print:
# #                     print("ar_ner_re_rank #####################################################################################")
# #                     print(ar_ner_re_rank)

# #                     print("ar_ner_filter #####################################################################################")
# #                     print(ar_ner_filter)

# #             end_time = time.time()
# #             print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TIME !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
# #             print(f"Time taken by ar NER: {end_time - start_time} seconds")
# #         #####################################################################################
# #         ############################# en NER #############################
# #         #####################################################################################
# #             start_time = time.time()

# #             for ele in en_ner:
# #                 ############# RAG ############# 
# #                 en_ner_rag = entity_translate_retriever.get_relevant_documents(ele)
# #                 en_ner_rag_list = list({doc.page_content for doc in en_ner_rag})
# #                 if do_print:
# #                     print("en_ner_rag_list #####################################################################################")
# #                     print(f"ele : {ele}")
# #                     print(en_ner_rag_list)

# #                 # all_en_ner_rag_res.append(en_ner_rag_list)

# #                 ############# Re-Rank ############# 
# #                 if en_ner_rag_list:
# #                     en_ner_re_rank = re_rank_doc(ele, en_ner_rag_list, CROSS_ENCODER_MODEL)

# #                     en_ner_filter = filter_context(ele, en_ner_rag_list, interpret_data_model, entity_query= False, entity=True)
# #                     en_ner_res.append(list(en_ner_filter))
# #                     en_ner_res_list = sum(en_ner_res, [])
                    
# #                     # if do_print:
# #                     print("en_ner_re_rank #####################################################################################")
# #                     print(en_ner_re_rank)
# #                     print("en_ner_filter #####################################################################################")
# #                     print(en_ner_filter)
# #             end_time = time.time()
# #             print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TIME !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
# #             print(f"Time taken by en NER: {end_time - start_time} seconds")
#         #####################################################################################
#         ############################# Fuzzy search ar entities ############################# # time take 0.17057 sec
#         #####################################################################################

#             ner_cleaned = [remove_stopwords(ele) for ele in ar_ner] 
#             ner_split = [ele.split() for ele in ner_cleaned]
#             ner_split_flattened = [item for sublist in ner_split for item in sublist]
#             if 'محطة' in ner_split_flattened:
#                 ner_split_flattened.remove('محطة')

            
#             ner_top_matches = get_matches(ner_split_flattened, all_translated_words, 5)
#             if do_print:
#                 print(ner_split_flattened)
#                 print("ner_top_matches #####################################################################################")
#                 print(ner_top_matches)

#             if ner_top_matches:
#                 ner_re_rank_match = re_rank_doc(text, ner_top_matches, CROSS_ENCODER_MODEL, entity_count*2)
#                 ner_rr_match_entites = map_fuzzy_to_entity_translator_context(ner_re_rank_match, translated_texts_dict, entity_translate_map )
#                 ner_rr_match_entites_list = [str(ele) for ele in ner_rr_match_entites]   
                
#                 ner_filtered_match = filter_context(text, ner_top_matches, interpret_data_model, fuzzy_entity=True)
#                 ner_match_entites = map_fuzzy_to_entity_translator_context(ner_filtered_match, translated_texts_dict, entity_translate_map )
#                 ner_match_entites_list = [str(ele) for ele in ner_match_entites]        
                
#                 if do_print:
#                     print("ner_match_entites #####################################################################################")
#                     print(ner_match_entites)

#                     print("ner_re_rank_match #####################################################################################")
#                     print(ner_re_rank_match)

#         #####################################################################################
#         ############################# Final RAG and fuzzy result #############################
#         #####################################################################################

#         entities_set = set(query_filter + query_match_entities_list + en_ner_res_list + ar_ner_res_list + ner_match_entites_list)
#         entities_list = list(entities_set)
#         if do_print:
#             # query_filter
#             # query_match_entities_list
#             # en_ner_res
#             # ar_ner_res
#             # ner_match_entites_list
#             print(query_filter)
#             print(query_match_entities_list)
#             print(en_ner_res_list)
#             print(ar_ner_res_list)
#             print(ner_match_entites_list)
#         print(entities_list)

#         ############################# LLM Chain ############################# # time taken 0.345 sec
        
#         result = translator_with_rag(translation_model, text, tgt, entities_list, translated_query)
#         print(result['translated_text'])
#         new_query = replace_words(result['translated_text'], replacements)
#         translated_entities_used = result['entities']

#     return new_query, translated_entities_used



# def process_ner(ner, language):
#     ner_res = []
#     start_time = time.time()

#     for ele in ner:
#         ############# RAG #############
#         ner_rag = entity_translate_retriever.get_relevant_documents(ele)
#         ner_rag_list = list({doc.page_content for doc in ner_rag})  # Unique results
        
#         if do_print:
#             print(f"{language}_ner_rag_list #####################################################################################")
#             print(f"ele : {ele}")
#             print(ner_rag_list)

#         ############# Re-Rank and filter #############
#         if ner_rag_list:
#             # ner_re_rank = re_rank_doc(ele, ner_rag_list, CROSS_ENCODER_MODEL)
#             ner_filter = filter_context(ele, ner_rag_list, interpret_data_model, entity_query=False, entity=True)
#             ner_res.append(list(ner_filter))

#             # if do_print:
#             # print(f"{language}_ner_re_rank #####################################################################################")
#             # print(ner_re_rank)
#             print(f"{language}_ner_filter #####################################################################################")
#             print(ner_filter)

#     ner_res_future = ner_res.result()
#     ner_res_list = sum(ner_res_future, [])
#     end_time = time.time()
#     print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TIME !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#     print(f"Time taken by {language} NER: {end_time - start_time} seconds")
#     return ner_res_list


# Agent

In [ ]:
AGENT_PROMPT = """You are a very powerful assistant that can answer questions. 

ALWAYS USE user_question_tool to answer fuel, fuel stations, station locations, transcations and bills related questions.

You can invoke the tool Calculator if you need to do mathematical operations.

Always use the tools to try to answer the questions. Use the chat history for context. Never try to use any other external information.

Assume that the user may write with misspellings, fix the spelling of the user before passing the question to any tool.

Don't mention what tool you have used in your answer.
"""

In [ ]:
# from langchain.tools import tool
# from langchain.callbacks.tracers import ConsoleCallbackHandler

# @tool
# def user_question_tool(input) -> str:
#     """Answers natural language questions related to fuel, fuel stations, station locations, transcations and bills."""
    
#     config = {'callbacks': [ConsoleCallbackHandler()]}
    
#     try:
#         # Fetch user memory
#         user_memory = user_memories.get(user_id)
#         if not user_memory:
#             raise ValueError("User memory not found.")
#         memory = user_memory.buffer_as_str.strip()

#         # Translate input to English
#         try:
#             translated_text, entities_list = translate_text_v2(CHAT_BISON, input, "en")
#             logger.info(f"Translated text: {translated_text}, Entities: {entities_list}")
#         except Exception as e:
#             logger.error(f"Error during translation: {str(e)}")
#             raise

#         # Prepare query
#         question = {"input": translated_text, "chat_history": memory, "entities": entities_list}
        
#         # Generate SQL query
#         try:
#             query = sql_query_generator(question, config)
#             logger.info(f"Generated SQL query: {query}")
#         except Exception as e:
#             logger.error(f"Error in SQL query generation: {str(e)}")
#             raise

#         # Interpret query response
#         try:
#             query_response = interpret_data(query, config)
#             logger.info(f"Query response: {query_response}")
#         except Exception as e:
#             logger.error(f"Error interpreting query response: {str(e)}")
#             raise

#         # Translate query response back to Arabic
#         try:
#             translated_result = translate_en_to_ar(query, query_response, entities_list, config)
#             logger.info(f"Translated result: {translated_result}")
#         except Exception as e:
#             logger.error(f"Error translating result to Arabic: {str(e)}")
#             raise

#         # Save conversation context
#         try:
#             user_memory.save_context({"input": translated_text}, {"output": translated_result.strip()})
#         except Exception as e:
#             logger.error(f"Error saving user memory: {str(e)}")
#             raise

#         return translated_result.strip()

#     except Exception as e:
#         logger.error(f"An error occurred in user_question_tool: {str(e)}")
#         return "An error occurred while processing your request."



In [ ]:
from langchain.tools import tool
from langchain.callbacks.tracers import ConsoleCallbackHandler
from pydantic import BaseModel, Field


class UserQuestionToolInput(BaseModel):
    input: int = Field(description="User natural language questions to be answered.")
    user_id: int = Field(description="User Id")


@tool("user-question-tool", args_schema= UserQuestionToolInput)
def user_question_tool(input: str, user_id: str ) -> str:
    """Answers natural language questions related to fuel, fuel stations, station locations, transcations and bills using data from database"""
    
    config = {'callbacks': [ConsoleCallbackHandler()]}
    
    try:
        logger.info(f"Using user_question_tool for user: {user_id}")
        logger.info(f"Using user_question_tool for user question: {input}")


        # Fetch user memory
        user_memory = user_memories.get(user_id)
        if not user_memory:
            raise ValueError("User memory not found.")
        memory = user_memory.buffer_as_str.strip()

        # Translate input to English
        try:
            translated_text, entities_list = translate_text_v2(CHAT_BISON, input, "en")
            logger.info(f"Translated text: {translated_text}, Entities: {entities_list}")
        except Exception as e:
            logger.error(f"Error during translation: {str(e)}")
            raise

        # Prepare query
        question = {"input": translated_text, "chat_history": memory, "entities": entities_list}
        
        # Generate SQL query
        try:
            query = sql_query_generator(question, config)
            logger.info(f"Generated SQL query: {query}")
        except Exception as e:
            logger.error(f"Error in SQL query generation: {str(e)}")
            raise

        # Interpret query response
        try:
            query_response = interpret_data(query, config)
            logger.info(f"Query response: {query_response}")
        except Exception as e:
            logger.error(f"Error interpreting query response: {str(e)}")
            raise

        # Translate query response back to Arabic
        try:
            translated_result = translate_en_to_ar(query, query_response, entities_list, config)
            logger.info(f"Translated result: {translated_result}")
        except Exception as e:
            logger.error(f"Error translating result to Arabic: {str(e)}")
            raise

        # Save conversation context
        try:
            user_memory.save_context({"input": translated_text}, {"output": translated_result.strip()})
        except Exception as e:
            logger.error(f"Error saving user memory: {str(e)}")
            raise

        return translated_result.strip()

    except Exception as e:
        logger.error(f"An error occurred in user_question_tool: {str(e)}")
        return "An error occurred while processing your request."



In [ ]:
from collections import defaultdict
from langchain.memory import ConversationBufferWindowMemory

user_memories = defaultdict(lambda: ConversationBufferWindowMemory(memory_key="chat_history", k=50, return_messages=True))
user_agents = {}

In [27]:
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/#initialize-tools

from langchain.agents import AgentType, initialize_agent, AgentExecutor

    
agent_kwgards = {"system_message": AGENT_PROMPT}
agent_tools = [math_tool, user_question_tool]  # List of tools used by the agent

def get_or_create_agent(user_id):
    try:
        # Check if the user agent already exists
        if user_id in user_agents:
            logging.info(f"Reusing existing agent for user_id: {user_id}")
            return user_agents[user_id]

        logging.info(f"Creating new agent for user_id: {user_id}")
        # Create/Get the memory for the user (here its creating)
        agent_memory = user_memories.get(user_id)

        if not agent_memory:
            logging.error(f"No memory found for user_id: {user_id}")
            raise ValueError(f"Memory for user_id {user_id} not initialized.")

        # Initialize a new agent with the required tools, memory, and configuration
        agent = initialize_agent(
            tools=agent_tools,
            llm=AGENT_MODEL,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            memory=agent_memory,
            agent_kwgards=agent_kwgards,
            max_iterations=5,
            early_stopping_method='generate',
            verbose=True
        )

        # Store the newly created agent for future use
        user_agents[user_id] = agent
        logging.info(f"Agent successfully created for user_id: {user_id}")

        return agent

    except Exception as e:
        logging.error(f"Failed to create or retrieve agent for user_id: {user_id}. Error: {e}")
        raise e



SyntaxError: invalid syntax (2867925098.py, line 1)

In [ ]:
# from langchain.prompts.chat import ChatPromptTemplate #chatGPT
from langchain_core.prompts import ChatPromptTemplate # https://python.langchain.com/api_reference/langchain/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool


# Define the agent's system prompt
AGENT_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "You are a very powerful assistant that can answer questions using postgres."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# List of tools available for the agent
agent_tools = [math_tool, user_question_tool]

def get_or_create_agent(user_id):
    try:
        # Check if the user agent already exists
        if user_id in user_agents:
            logging.info(f"Reusing existing agent for user_id: {user_id}")
            return user_agents[user_id]

        logging.info(f"Creating new agent for user_id: {user_id}")
        # Get the memory associated with the user
        agent_memory = user_memories.get(user_id)

        if not agent_memory:
            logging.error(f"No memory found for user_id: {user_id}")
            raise ValueError(f"Memory for user_id {user_id} not initialized.")

        # Create a new agent using create_tool_calling_agent
        agent = create_tool_calling_agent(
            llm=AGENT_MODEL,
            tools=agent_tools,
            prompt=AGENT_PROMPT
        )

        # Wrap the agent in an executor with settings like max iterations and verbose logging
        agent_executor = AgentExecutor(
            agent=agent,
            tools=agent_tools,
            memory=agent_memory,
            max_iterations=5,
            early_stopping_method='generate',
            verbose=True
        )

        # Store the agent executor for future use
        user_agents[user_id] = agent_executor
        logging.info(f"Agent successfully created for user_id: {user_id}")

        return agent_executor

    except Exception as e:
        logging.error(f"Failed to create or retrieve agent for user_id: {user_id}. Error: {e}")
        raise e


In [ ]:
import jwt
import json
from jwt import DecodeError, ExpiredSignatureError

# Decoding JWT (without verification)
def decode_jwt(token):
    try:
        # Decode the token to view its payload and header without verifying the signature
        decoded = jwt.decode(token, options={"verify_signature": False})
        
        return decoded
    except DecodeError as e:
        return f"Failed to decode token: {e}"


# Verifying JWT Signature
def verify_jwt(token, pub_key):
    try:
        # Verifying the token with the public key (for RS256)
        verified = jwt.decode(token, pub_key, algorithms=["RS256"])
        print("Verified JWT Payload: ", verified)
        print("Verified JWT Header: ", verified.header)
        return verified
    except ExpiredSignatureError:
        return "Token has expired!"
    except DecodeError:
        return "Invalid token signature!"

In [ ]:
# def process_user_query(user_id: str, query: str) -> str:
#     """Process the user query and return the response."""
#     agent = get_or_create_agent(user_id)
#     response = agent({"input": query})
#     output = response['output']
#     return output

def process_user_query(token: str, query: str) -> str: 
    """Process the user query and return the response."""
    try:
        if not token or not query:
            logging.error("Token or query is missing.")
            raise ValueError("Token and query are required.")
        
        # Decode JWT to get user data
        user_data = decode_jwt(token)
        
        if 'TenantID' not in user_data or 'user_id' not in user_data:
            logging.error("Invalid user data received from JWT.")
            raise ValueError("Invalid user data.")
        
        tenant_id = user_data['TenantID']
        user_id = user_data['user_id']
        
        logging.info(f"Processing query for user_id: {user_id} with tenant_id: {tenant_id}")

        # Get or create the agent for the user
        agent = get_or_create_agent(user_id)
        
        # Process the query using the agent
        response = agent({"input": query})

        if 'output' not in response:
            logging.error(f"No output found in response for user_id: {user_id}")
            raise ValueError("No output found in the response.")
        
        output = response['output']
        logging.info(f"Query processed successfully for user_id: {user_id}")
        
        return output

    except Exception as e:
        logging.error(f"Error processing user query for token: {token}. Error: {e}")
        raise  # Re-raise the exception after logging

In [ ]:
user_memories.clear()
user_agents.clear()
user_agents
user_memories

# Query

In [ ]:
# query = "ماهو متوسط عدد العمليات في شهر خمسة سنة 2024؟ متوسط قيمة جميع العمليات للشهر." # What is the average transaction amount for May 2024? Include the average value of all transactions for the month.
user_id = "user12"  # Example user ID
# query = "هل يمكنك أن تعطيني بعض المعلومات عن محطة سمرة ؟" # samara
query = "ما هي مبيعات الموظفين بتاريخ 18 مايو 2024؟"  # "what were the employee sales for the date 18th of May 2024?"
# query =  "كم عدد المعاملات التي تم الانتهاء منها في 23 مايو 2024 ؟" # How many transactions were completed on May 25, 2024? 
# query = "2024 كم عدد طلبات الوقود في الأسبوع الأول من شهر مايو" # How much fuel was requested on the 1st week of May 2024?
# query = "no, give me average value of all transactions for the whole month, not only 18th"

response = process_user_query(user_id, query)

In [ ]:
# Example usage
user_id = "user12"  # Example user ID

# query = "Which stations have charging?"
# query = "ما هي المحطات التي لديها الشحن؟"

# query = "Which stations have restrooms?"
# query = "Can you give me name of 2 stations?"

# query = "what were the employee sales for the date 18th of May 2024?"
# query = "ما هي مبيعات الموظفين بتاريخ 18 مايو 2024؟"

# query = "what is the employee sales of the 1st 2 week of may 2024"
# query = "How much fuel was requested on 1st week of may 2024"
# query = "give me a brief summary on the transactions done in may 2024"
# query = "Who approved the fuel requested on 19th may 2024"

# query = "Tell me about the bill processed by Amal"
# query = "أخبريني عن الفاتورة التي عالجتها أمل"


# query = "What were the employee sales for the date 18th of May 2024?"
# query = "عطني ملخص العمليات التي تمت في شهر مايو سنة 2024"
# query = "كم مبيعات الموظفين في يوم 18 مايو سنة 2024"


In [ ]:
li = ["كم مبيعات الموظفين في يوم 18 مايو سنة 2024", "ما هي مبيعات الموظفين بتاريخ 18 مايو 2024 ؟", "كم كانت مبيعات الموظفين في تاريخ 18/05/2024", "كم عدد الوقود الذي تم طلبه في اول أسبوع من شهر مايو", "ما مقدار الوقود المطلوب في الأسبوع الأول من مايو 2024 ؟", "كم عدد طلبات الوقود في الأسبوع الأول من شهر مايو", "ماهي مبيعات الموظفين في اول أسبوعين من شهر مايو سنة 2024", "ما هي مبيعات الموظفين في الأسابيع الاثنين الأولى من مايو 2024 ؟", "كم كانت مبيعات الموظفين في الأسبوعين الأولى من شهر مايو ", "عطني ملخص العمليات التي تمت في شهر مايو سنة 2024", "أعطني ملخصًا موجزًا للمعاملات التي تمت في مايو 2024", "اعطيني ملخص العمليات اللي صارت في شهر مايو", "اعطيني معلومات عن الخليج", "زودني بمعلومات عن الخليج", "عطني معلومات عن الخليج ", "اعطيني بعض المعلومات عن محطة سمرة", "هل يمكنك أن تعطيني بعض المعلومات عن محطة سمرة ؟", "تقدر تعطيني بعض الملعومات عن محطة سمرة", "ما هي المحطات التي تحتوي على دورة مياه ومحطة شحن وغسيل سيارات ؟", "ايش هي المحطات التي تحتوي على دورة مياه ومحطة شحن وغسيل سيارات ؟", "وش هي المحطات اللي فيها دورات مياه وشاحن كهربائي ومغسلة سيارات ؟", "مين وافق على طلب الوقود في يوم 19 شهر 5 سنة 2024", "من وافق على الوقود المطلوب في 19 مايو 2024 ؟", "من اعتمد طلبات الوقود في 19 مايو 2024 ؟", "كم عدد مبيعات الوقود في شهر 5 سنة 2024؟ اعطيني تراكمي المبيعات ", "ما هو إجمالي مبيعات الوقود لشهر مايو 2024 ؟ مبلغ المبيعات التراكمي.", "كم كان اجمالي مبيعات الوقود في شهر مايو 2024 ؟ عطني اجمالي قيمة المبيعات ", "كم عدد الحوالات المكتملة في يوم 25 شهر 5 سنة 2024؟", "كم عدد المعاملات التي تم الانتهاء منها في 25 مايو 2024 ؟", "كم كانت العمليات اللي تمت في 25 مايو سنة 2024", "اعطيني اجمالي الحولات في هذاك اليوم", "هل يمكنك إعطاء إجمالي عدد المعاملات لذلك اليوم ؟", "تقدر تعطيني اجمالي العمليات في ذاك اليوم ", "من هم أفضل ثلاثة موظفين من حيث المبيعات لشهر 5 سنة 2024 ؟ يرجى ذكر أسمائهم ومبالغ مبيعاتهم.", "مين افضل ثلاثة موظفين من حيث المبيعات لشهر مايو سنة 2024؟ اذكر أسمائهم ومبالغ مبيعاتهم", "مين اعلى ثلاثة عمال في المبيعات لشهر مايو سنة 2024؟ عطني أسمائهم مع المبيعات ", "ما مقدار الإيرادات التي تم تحقيقها من مبيعات الوقود في الأسبوع الثاني من مايو 2024 ؟ توفير إجمالي الإيرادات لتلك الفترة.", "كم عدد الإيرادات من مبيعات الوقود في الأسبوع الثاني من شهر خمسة سنة 2024؟ اعطيني اجمالي الإيرادات في ذيك الفترة", "كم كان دخل مبيعات الوقود في الأسبوع الثاني من شهر 5 سنة 2024؟ عطني اجمالي الدخل في هذيك الفترة ", "ماهو متوسط عدد العمليات في شهر خمسة سنة 2024؟ متوسط قيمة جميع العمليات للشهر.", "ما هو متوسط مبلغ الصفقة لشهر مايو 2024 ؟ تضمين متوسط قيمة جميع المعاملات للشهر.", "وش متوسط عدد العمليات في مايو 2024؟ اضف متوسط قيمة كل العمليات للشهر", "هل يمكنك تقديم العدد الإجمالي لطلبات الوقود المقدمة في مايو 2024 ؟ أحتاج إلى إحصاء جميع طلبات الوقود المقدمة في ذلك الشهر.", "اعطيني مجموع طلبات الوقود في شهر 5 سنة 2024؟ احتاج جميع طلبات الوقود في ذلك الشهر", "كم مجموع طلبات الوقود في شهر مايو 2024؟ ابي عدد كل الطلبات المعتمدة في هذا الشهر"]
li_en = [ "How much are employee sales on May 18, 2024?", "What are employee sales on May 18, 2024?", ]


In [ ]:
user_memories.clear()
user_agents.clear()
user_agents
user_memories

In [ ]:

response = process_user_query(user_id, li[4])

In [ ]:
print(response)
li_res = ["بلغت مبيعات الموظفين في 18 مايو 2024 مبلغ 85140.05." , "إجمالي مبيعات الموظفين في 18 مايو 2024 هو 85140.05.", "", "", "إجمالي الوقود المطلوب في الأسبوع الأول من شهر مايو 2024 هو 400000.", ]

# Test

In [ ]:
q = "How many operations took place on May 25, 2024?"
# q = "what were the total employee sales for the date 18th of May 2024"
q ="Give me a summary of the operations that took place in May 2024"
result = replace_words(q, replacements)
print(result)

In [ ]:
# # question = {"input": "what were the total employee sales for the date 18th of May 2024", "chat_history": "", "entities": []}
# q = "How many transactions took place on May 25, 2024?"
# q = "How many fuel orders are there in the first week of May 2024? and how much of each fuel, give fuel name not id?"
q = "How much revenue will be generated from fuel sales in the second week of May 2024? Give me the total revenues in that period"
question = {"input": q , "entities" : [""], "chat_history": ""}

config={} # 'callbacks': [ConsoleCallbackHandler()]

query = sql_query_generator(question, config)

query_response, query_result = interpret_data(query, config=config)

# translated_result = translate_en_to_ar(query, query_response, query_result, config)

In [ ]:
print(query) 

In [ ]:

li = ["كم مبيعات الموظفين في يوم 18 مايو سنة 2024", "ما هي مبيعات الموظفين بتاريخ 18 مايو 2024 ؟", "كم كانت مبيعات الموظفين في تاريخ 18/05/2024", "كم عدد الوقود الذي تم طلبه في اول أسبوع من شهر مايو", "ما مقدار الوقود المطلوب في الأسبوع الأول من مايو 2024 ؟", "كم عدد طلبات الوقود في الأسبوع الأول من شهر مايو", "ماهي مبيعات الموظفين في اول أسبوعين من شهر مايو سنة 2024", "ما هي مبيعات الموظفين في الأسابيع الاثنين الأولى من مايو 2024 ؟", "كم كانت مبيعات الموظفين في الأسبوعين الأولى من شهر مايو ", "عطني ملخص العمليات التي تمت في شهر مايو سنة 2024", "أعطني ملخصًا موجزًا للمعاملات التي تمت في مايو 2024", "اعطيني ملخص العمليات اللي صارت في شهر مايو", "اعطيني معلومات عن الخليج", "زودني بمعلومات عن الخليج", "عطني معلومات عن الخليج ", "اعطيني بعض المعلومات عن محطة سمرة", "هل يمكنك أن تعطيني بعض المعلومات عن محطة سمرة ؟", "تقدر تعطيني بعض الملعومات عن محطة سمرة", "ما هي المحطات التي تحتوي على دورة مياه ومحطة شحن وغسيل سيارات ؟", "ايش هي المحطات التي تحتوي على دورة مياه ومحطة شحن وغسيل سيارات ؟", "وش هي المحطات اللي فيها دورات مياه وشاحن كهربائي ومغسلة سيارات ؟", "مين وافق على طلب الوقود في يوم 19 شهر 5 سنة 2024", "من وافق على الوقود المطلوب في 19 مايو 2024 ؟", "من اعتمد طلبات الوقود في 19 مايو 2024 ؟", "كم عدد مبيعات الوقود في شهر 5 سنة 2024؟ اعطيني تراكمي المبيعات ", "ما هو إجمالي مبيعات الوقود لشهر مايو 2024 ؟ مبلغ المبيعات التراكمي.", "كم كان اجمالي مبيعات الوقود في شهر مايو 2024 ؟ عطني اجمالي قيمة المبيعات ", "كم عدد الحوالات المكتملة في يوم 25 شهر 5 سنة 2024؟", "كم عدد المعاملات التي تم الانتهاء منها في 25 مايو 2024 ؟", "كم كانت العمليات اللي تمت في 25 مايو سنة 2024", "اعطيني اجمالي الحولات في هذاك اليوم", "هل يمكنك إعطاء إجمالي عدد المعاملات لذلك اليوم ؟", "تقدر تعطيني اجمالي العمليات في ذاك اليوم ", "من هم أفضل ثلاثة موظفين من حيث المبيعات لشهر 5 سنة 2024 ؟ يرجى ذكر أسمائهم ومبالغ مبيعاتهم.", "مين افضل ثلاثة موظفين من حيث المبيعات لشهر مايو سنة 2024؟ اذكر أسمائهم ومبالغ مبيعاتهم", "مين اعلى ثلاثة عمال في المبيعات لشهر مايو سنة 2024؟ عطني أسمائهم مع المبيعات ", "ما مقدار الإيرادات التي تم تحقيقها من مبيعات الوقود في الأسبوع الثاني من مايو 2024 ؟ توفير إجمالي الإيرادات لتلك الفترة.", "كم عدد الإيرادات من مبيعات الوقود في الأسبوع الثاني من شهر خمسة سنة 2024؟ اعطيني اجمالي الإيرادات في ذيك الفترة", "كم كان دخل مبيعات الوقود في الأسبوع الثاني من شهر 5 سنة 2024؟ عطني اجمالي الدخل في هذيك الفترة ", "ماهو متوسط عدد العمليات في شهر خمسة سنة 2024؟ متوسط قيمة جميع العمليات للشهر.", "ما هو متوسط مبلغ الصفقة لشهر مايو 2024 ؟ تضمين متوسط قيمة جميع المعاملات للشهر.", "وش متوسط عدد العمليات في مايو 2024؟ اضف متوسط قيمة كل العمليات للشهر", "هل يمكنك تقديم العدد الإجمالي لطلبات الوقود المقدمة في مايو 2024 ؟ أحتاج إلى إحصاء جميع طلبات الوقود المقدمة في ذلك الشهر.", "اعطيني مجموع طلبات الوقود في شهر 5 سنة 2024؟ احتاج جميع طلبات الوقود في ذلك الشهر", "كم مجموع طلبات الوقود في شهر مايو 2024؟ ابي عدد كل الطلبات المعتمدة في هذا الشهر"]


In [ ]:
for ele in li[37:-1]:
    translated_text, entities = translate_text_v2(AGENT_MODEL, ele, "en") # , entities_list
    print(translated_text) # , entities_list
    print(ele) # , entities_list


In [ ]:
# t = "زودني بمعلومات عن الخليج"

t = "كم عدد الوقود الذي تم طلبه في اول أسبوع من شهر مايو"
translated_text, entities = translate_text_v2(AGENT_MODEL, t, "en") # , entities_list
print(translated_text)

In [ ]:
entities

In [ ]:

# query = "كم مبيعات الموظفين في يوم 18 مايو سنة 2024"  # What are the employee sales on May 18, 2024?
# query = "ما هي مبيعات الموظفين بتاريخ 18 مايو 2024 ؟"  # What are the employee sales on May 18, 2024?
# query = "كم كانت مبيعات الموظفين في تاريخ 18/05/2024"   # How much were the employee sales on May 18, 2024?

# query = "How much fuel was requested on the 1st week of May 2024?"
# query = "كم عدد الوقود الذي تم طلبه في اول أسبوع من شهر مايو"  # How much fuel was ordered in the first week of May?
# query = "ما مقدار الوقود المطلوب في الأسبوع الأول من مايو 2024 ؟"  # How much fuel is required in the first week of May 2024?
# query = "كم عدد طلبات الوقود في الأسبوع الأول من شهر مايو"  # How many fuel orders are there in the first week of May?

# query = "ماهي مبيعات الموظفين في اول أسبوعين من شهر مايو سنة 2024"  # What are the employee sales in the first two weeks of May 2024?
# query = "ما هي مبيعات الموظفين في الأسابيع الاثنين الأولى من مايو 2024 ؟"  # What are the employee sales in the first two weeks of May 2024?
# query = "كم كانت مبيعات الموظفين في الأسبوعين الأولى من شهر مايو "  # How much were the sales of employees in the first two weeks of May?

# query = "عطني ملخص العمليات التي تمت في شهر مايو سنة 2024" # Give me a summary of the operations that took place in May 2024.
# query = "أعطني ملخصًا موجزًا للمعاملات التي تمت في مايو 2024" # Provide a concise summary of the transactions that occurred in May 2024.
# query = "اعطيني ملخص العمليات اللي صارت في شهر مايو" # Give me a summary of the operations that took place in May.

# query = "اعطيني معلومات عن الخليج"  # Give me information about the Persian Gulf. # 
# query = "زودني بمعلومات عن الخليج"  # Provide me with information about the Gulf. 
# query = ""

# query = "اعطيني بعض المعلومات عن محطة سمرة"  # Give me some information about Samra petrol station.
# query = "هل يمكنك أن تعطيني بعض المعلومات عن محطة سمرة ؟" # Can you provide me with some details about Samra petrol station? # WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '؟']
# query = "تقدر تعطيني بعض الملعومات عن محطة سمرة"  # Can you provide me with some information about Samra petrol station?

# ما هي المحطات التي تحتوي على دورة مياه ومحطة شحن وغسيل سيارات ؟  #  Which stations have a restroom, a charging station, and a car wash?
# ايش هي المحطات التي تحتوي على دورة مياه ومحطة شحن وغسيل سيارات ؟  #  Which stations have a restroom, a charging station, and a car wash?
# وش هي المحطات اللي فيها دورات مياه وشاحن كهربائي ومغسلة سيارات ؟  #  Which stations have restrooms, an electric charger, and a car wash?

# مين وافق على طلب الوقود في يوم 19 شهر 5 سنة 2024  #  Who approved the fuel request on May 19, 2024?
# مين وافق على طلب الوقود في يوم 19 شهر 5 سنة 2024  #  Who approved the fuel request on May 19, 2024?
# من وافق على الوقود المطلوب في 19 مايو 2024 ؟  #  Who approved the fuel required on May 19, 2024?
# من اعتمد طلبات الوقود في 19 مايو 2024 ؟  #  Who approved fuel requests on May 19, 2024?
# كم عدد مبيعات الوقود في شهر 5 سنة 2024؟ اعطيني تراكمي المبيعات   #  How many fuel sales in May 2024? Give me your cumulative sales.
# ما هو إجمالي مبيعات الوقود لشهر مايو 2024 ؟ مبلغ المبيعات التراكمي.  #  What are the total fuel sales for May 2024? Cumulative sales amount.
# كم كان اجمالي مبيعات الوقود في شهر مايو 2024 ؟ عطني اجمالي قيمة المبيعات   #  What were the total fuel sales in May 2024? Provide the total sales value.
# كم عدد الحوالات المكتملة في يوم 25 شهر 5 سنة 2024؟  #  How many transfers were completed on May 25, 2024?
# كم عدد المعاملات التي تم الانتهاء منها في 25 مايو 2024 ؟  #  How many transactions were completed on May 25, 2024?

# كم كانت العمليات اللي تمت في 25 مايو سنة 2024  #  How many operations were there on May 25, 2024?
# اعطيني اجمالي الحولات في هذاك اليوم  #  Give me the total transfers on that day
# هل يمكنك إعطاء إجمالي عدد المعاملات لذلك اليوم ؟  #  Can you provide the total number of transactions for that day?
# تقدر تعطيني اجمالي العمليات في ذاك اليوم   #  Can you give me the total number of operations on that day?
# من هم أفضل ثلاثة موظفين من حيث المبيعات لشهر 5 سنة 2024 ؟ يرجى ذكر أسمائهم ومبالغ مبيعاتهم.  #  Who are the top three employees in terms of sales for May 2024? Please provide their names and sales amounts.
# مين افضل ثلاثة موظفين من حيث المبيعات لشهر مايو سنة 2024؟ اذكر أسمائهم ومبالغ مبيعاتهم  #  Who are the top three employees in terms of sales for May 2024? Please provide their names and sales amounts.

# مين اعلى ثلاثة عمال في المبيعات لشهر مايو سنة 2024؟ عطني أسمائهم مع المبيعات   #  Who are the top three sales workers for May 2024? Provide their names and sales figures.
# ما مقدار الإيرادات التي تم تحقيقها من مبيعات الوقود في الأسبوع الثاني من مايو 2024 ؟ توفير إجمالي الإيرادات لتلك الفترة.  #  How much revenue was generated from fuel sales during the second week of May 2024? Provide the total revenue for that period.
# كم عدد الإيرادات من مبيعات الوقود في الأسبوع الثاني من شهر خمسة سنة 2024؟ اعطيني اجمالي الإيرادات في ذيك الفترة  #  How much revenue will be generated from fuel sales in the second week of May 2024? Give me the total revenues in that period.

# كم كان دخل مبيعات الوقود في الأسبوع الثاني من شهر 5 سنة 2024؟ عطني اجمالي الدخل في هذيك الفترة   #  What was the fuel sales revenue in the second week of May 2024? Give me the total revenue during that period.
# ماهو متوسط عدد العمليات في شهر خمسة سنة 2024؟ متوسط قيمة جميع العمليات للشهر.  #  What is the average number of operations in May 2024? Average value of all operations for the month.
# ما هو متوسط مبلغ الصفقة لشهر مايو 2024 ؟ تضمين متوسط قيمة جميع المعاملات للشهر.  #  What is the average deal amount for May 2024? Include the average value of all transactions for the month.
# وش متوسط عدد العمليات في مايو 2024؟ اضف متوسط قيمة كل العمليات للشهر  #  What is the average number of operations in May 2024? Also, add the average value of all operations for the month.

# هل يمكنك تقديم العدد الإجمالي لطلبات الوقود المقدمة في مايو 2024 ؟ أحتاج إلى إحصاء جميع طلبات الوقود المقدمة في ذلك الشهر.  #  Can you provide the total number of fuel orders placed in May 2024? I need to count all fuel orders placed that month.
# اعطيني مجموع طلبات الوقود في شهر 5 سنة 2024؟ احتاج جميع طلبات الوقود في ذلك الشهر  #  Give me the total fuel orders in May 2024. I need all fuel orders for that month.

In [ ]:
listt = [{'Sharq': 'شرق'},
{'Riyadh, KSA': 'الرياض، المملكة العربية السعودية'},
{'Fujairah': 'الفجيرة'},
{'Dubai': 'دبي'},
{'Al-Khaleej station': 'محطة الخليج'},
{'Umm Al Quwain': 'ام القيوين'},
{'Al-Khaleej': 'الخليج'},
{'Sharjah': 'الشارقة'}]

ent = '\n'.join([str(doc) for doc in listt])

# Lanagchain set_llm_cache

In [ ]:
CODE_BISON = ChatVertexAI(model_name="codechat-bison", max_output_tokens=1024)
INTERPRETOR_MODEL = ChatVertexAI(max_output_tokens=1024)
AGENT_MODEL = ChatVertexAI(max_output_tokens=1024)
CHAT_BISON = ChatVertexAI(model_name="chat-bison@002", max_output_tokens=1024)

In [7]:
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache

set_llm_cache(InMemoryCache())

In [13]:
%%time
res = INTERPRETOR_MODEL.invoke("what is ML")
res

CPU times: user 727 µs, sys: 115 µs, total: 842 µs
Wall time: 729 µs


AIMessage(content=' Machine learning (ML) is a subfield of artificial intelligence (AI) that gives computers the ability to learn without being explicitly programmed. ML algorithms are able to learn from and make decisions or predictions based on data. These algorithms are typically trained on large datasets and can be used for a variety of tasks, such as image recognition, natural language processing, and predictive analytics.')

In [12]:
%%time
res = INTERPRETOR_MODEL.invoke("Tell me about ML")
res

CPU times: user 9.71 ms, sys: 856 µs, total: 10.6 ms
Wall time: 2.6 s


AIMessage(content=' Machine learning (ML) is a subfield of artificial intelligence (AI) that gives computers the ability to learn without being explicitly programmed. ML algorithms are able to learn from and make decisions or predictions based on data.\n\nThere are many different types of ML algorithms, each with its own strengths and weaknesses. Some of the most common types of ML algorithms include:\n\n* **Supervised learning:** In supervised learning, the ML algorithm is trained on a dataset of labeled data. This means that each data point in the dataset is associated with a known output. The ML algorithm learns to map input data to output data by finding patterns in the labeled data.\n* **Unsupervised learning:** In unsupervised learning, the ML algorithm is trained on a dataset of unlabeled data. This means that each data point in the dataset does not have a known output. The ML algorithm learns to find patterns in the unlabeled data without being explicitly told what to look for.

In [14]:
%%time
res = INTERPRETOR_MODEL.invoke("Tell me about ML")
res

CPU times: user 847 µs, sys: 135 µs, total: 982 µs
Wall time: 869 µs


AIMessage(content=' Machine learning (ML) is a subfield of artificial intelligence (AI) that gives computers the ability to learn without being explicitly programmed. ML algorithms are able to learn from and make decisions or predictions based on data.\n\nThere are many different types of ML algorithms, each with its own strengths and weaknesses. Some of the most common types of ML algorithms include:\n\n* **Supervised learning:** In supervised learning, the ML algorithm is trained on a dataset of labeled data. This means that each data point in the dataset is associated with a known output. The ML algorithm learns to map input data to output data by finding patterns in the labeled data.\n* **Unsupervised learning:** In unsupervised learning, the ML algorithm is trained on a dataset of unlabeled data. This means that each data point in the dataset does not have a known output. The ML algorithm learns to find patterns in the unlabeled data without being explicitly told what to look for.

In [24]:
!pip install langchainhub
from langchain import hub

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.19
    Uninstalling urllib3-1.26.19:
      Successfully uninstalled urllib3-1.26.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.3 which is incompatible.


In [25]:
prompt = hub.pull("hwchase17/react")

/opt/conda/lib/python3.10/site-packages/langchain/hub.py:80: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  resp: str = client.pull(owner_repo_commit)
/opt/conda/lib/python3.10/site-packages/langchainhub/client.py:326: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = self.pull_repo(owner_repo_commit)


In [26]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

# CONTEXT_PROMPT

In [ ]:
CONTEXT_PROMPT = """You are a helpful assistant. Your task is to determine if the current user question is a follow-up to any previous questions.

Current question:
{question}

Chat history:
{chat_history}

If the current question is indeed a follow-up, please formulate a new question that highlights the current question while incorporating relevant context from the related question and its answer.
If the current question is not a  follow-up, return the current question as is.

New question:
"""

# " and its answer" is optional

# question adjusted for possible follow up 
adujusted_question = GEMINI_PRO(CONTEXT_PROMPT)

# Agent executer

In [ ]:
# https://js.langchain.com/docs/integrations/chat/google_vertex_ai/
# https://python.langchain.com/docs/integrations/chat/google_generative_ai/
# https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm/

# https://python.langchain.com/docs/how_to/tool_calling/      # Tool calling 
# https://python.langchain.com/docs/concepts/#agents           # ReAct
# https://python.langchain.com/docs/how_to/custom_tools/      # Create custom tools
# https://python.langchain.com/docs/concepts/#tools          # Tools
# https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/     # Tools/function calling
# https://python.langchain.com/docs/how_to/agent_executor/#concepts        # will tell AgentExecutor is not the best For more flexibility and give more info on chat history management
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/    # create_tool_calling_agent      
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/           # agent_types
# https://python.langchain.com/api_reference/google_vertexai/chat_models/langchain_google_vertexai.chat_models.ChatVertexAI.html


#Batch - https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html


# Guard Rails

In [ ]:
def calling_dummy_api(user_id):
    return
    
def table_access_check():
    # call API
    permited_tables = calling_dummy_api(user_id)

    # From RAG get table id, if restricted id are there remove that table
    # from SQL query generated, check if table_id in query, if is there re generate query with restricted_prompt
    # get the id of the user from tenant_id table, see how you can add filter to make sure only selected tenant and stations are seleted, 
        # Maybe use a function to that adds to WHERE clause if it exists if not, adds the WHERE clause   
    # Get the sql result, make sure labels are there (wont work if aggregation are performed)
        # and turn it into  DF,
        # use it to filter out the ID  
    